# Initializations

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

import os
import sys
import argparse
import csv
import json
import time
import types, copy, pprint
from datetime import datetime
for p in ['../','../../..']:
    if p not in sys.path:
        print(f"insert {p}")
        sys.path.insert(0, p)
print(sys.path)
import pickle
import pathlib
import copy
# import io

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
# import s3fs
from datetime import datetime
import scipy
print(scipy.__version__)

# from utils_cellpainting import (read_profiles_from_local, cat_columns,  get_cols_like,
                                 # disp_stats, check_values, )
# from KevinsRoutines.utils.utils_cellpainting import (disp_metadata_file, read_profiles_from_local, cat_columns,  get_cols_like,
#                                  disp_stats, check_values, )

# from KevinsRoutines.utils.utils_general  import  list_namespace 
# from scipy.stats import median_absolute_deviation
from pycytominer import normalize, feature_select

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)
pd.options.display.width = 132
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')

pio.renderers.default = "png"  # Set to "svg" or "png" for static plots or "notebook_connected" for interactive plots
pd.set_option('display.width', 140)

insert ../
insert ../../..
['../../..', '../', '/home/kevin/WSL-shared/cellpainting/cj-datasets/Notebooks-Dataprep', '/home/kevin/miniforge3/envs/cp311/lib/python311.zip', '/home/kevin/miniforge3/envs/cp311/lib/python3.11', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/lib-dynload', '', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg']
1.11.4


In [2]:
# time.strftime(' %x%X')
# datetime.now().strftime('%X.%f')
# time.strftime('%X %x %Z')
print(datetime.now().strftime('%D-%X.%f'))
print(datetime.now().strftime('%Y-%M-%d %H:%m:%S.%f' ))

import logging 
logLevel = os.environ.get('LOG_LEVEL', 'INFO').upper()
FORMAT = '%(asctime)s - %(levelname)s: - %(message)s'
logging.basicConfig(level="INFO", format= FORMAT)
logging.getLogger("imported_module").setLevel(logging.CRITICAL)

12/20/24-18:58:32.319532
2024-58-20 18:12:32.319852


In [3]:
# import dask
# import dask.array as da
# import dask.dataframe as dd
# from dask import delayed
# from dask.distributed import Client
# from dask.distributed import LocalCluster
# from dask_cuda import LocalCUDACluster
# import dask_ml.model_selection as dcv
 

In [4]:
from KevinsRoutines.utils.utils_cellpainting import (disp_metadata_file, read_profiles_from_local, cat_columns,  get_cols_like,
                                 disp_stats, check_values, check_df_for_nans )



/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning:

Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.



In [5]:
from src.utils.jcp_datasets import project_datasets, display_project_datasets
# ds = project_datasets(output_path = './output_10162024/')
ds = project_datasets(output_path = '../output_11102023/')
display_project_datasets(ds)

 Path and Prefixes : 
 ------------------- 
 prefix                                      : 
 prefix_lc                                   : 
 metadata path                               : ./metadata/
 input path                                  : ./input/
 output path                                 : ../output_11102023/

 Input & Metadata Datasets : 
 --------------------------- 
 Compound Metadata Input File                : ./metadata/compound.csv
 Plate Metadata Input File                   : ./metadata/plate_new.csv
 Well Metadata Input File                    : ./metadata/well.csv

 Profiles Metadata File                      : ./metadata/profile_metadata.pkl
 Parquet  Metadata File                      : ./metadata/parquet_columns.pkl

 Compound Metadata : 
 ------------------- 
 Metadata + Smiles                           : ../output_11102023/compound_metadata_smiles.csv
          + TPSA                             : ../output_11102023/compound_metadata_tpsa.csv
          + TPSA

# Load Metadata Files

<!-- This notebook loads a small number of plates with precomputed features and the metadata information. -->

- Order Extended Metadata Data set by Source, Batch, Plate

The following files contain the metadata information for the entire dataset.
The schema is [here](metadata/README.md).

## Profiles Metadata file

In [117]:
print(f" Profile Metadata file:  {ds.profileMetadataFile}")

with open(ds.profileMetadataFile, 'rb') as f: 
     pickle_data = pickle.load(f)

 Profile Metadata file:  ./metadata/profile_metadata.pkl


In [118]:
print(f" Profile metadata dict  keys : {pickle_data.keys()}")

MetadataColumns =  pickle_data['metadata_columns']['MetadataNames']
MetadataNames   =  pickle_data['metadata_columns']['MetadataNames']
ParquetColumns = pickle_data['parquet_columns']
MetadataCategoricalColumns = pickle_data['metadata_columns']['MetadataCategoricalColumns']
MetadataFloatColumns       = pickle_data['metadata_columns']['MetadataFloatColumns']
MetadataIntegerColumns     = pickle_data['metadata_columns']['MetadataIntegerColumns']
WRITE_COMPOUND_PROFILE_COLUMNS = MetadataColumns + sorted(list(pickle_data['selected_columns']['selected']))
disp_metadata_file(pickle_data, WRITE_COMPOUND_PROFILE_COLUMNS)

 Profile metadata dict  keys : dict_keys(['all_profile_columns', 'metadata_columns', 'selected_columns', 'parquet_columns'])
--------------------------------------------------------------------------------
 all_profile_columns & COMPOUND_PROFILE_COLUMNS
--------------------------------------------------------------------------------
  Len all_profile_columns        : 3672
  Len COMPOUND_PROFILE_COLUMNS   : 1482

  all_profile_columns[:10]       : ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA']
  COMPOUND_PROFILE_COLUMNS[:10]  : ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA']

  all_profile_columns[10:15]     : ['Metadata_Permiation', 'Cells_AreaShape_Area', 'Cells_AreaShape_BoundingBoxArea', 'Cells_AreaShape_BoundingBo

### Parquet Profile Columns Metadata

   Parquet files column metadata

In [218]:
SelectedColumns = pickle_data['selected_columns']
print(f" Parquet_columns keys : {ParquetColumns.keys()}")
print(f" Selected_columns keys: {SelectedColumns.keys()}")

parquetAllColumns = ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei'] | ParquetColumns['Image'] | ParquetColumns['Metadata']
parquetNumericColumns  =  ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei']
total_dropped = 0 
for k in ['dropped_correlation', 'dropped_variance', 'dropped_nans', 'dropped_image_features', 'dropped_metadata']:
    total_dropped += len(SelectedColumns[k])
    
print(total_dropped)
print(f"\n")
print(f" Parquet      - Metadata columns:                                             {len(ParquetColumns['Metadata']):7d}")
print(f" File         - Image feature columns:                                        {len(ParquetColumns['Image']):7d}")
print(f" Pre-Drop     - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cells']):7d}")
print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cytoplasm']):7d}")
print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Nuclei']):7d}")
print(f"                                               total columns:                 {len(parquetAllColumns):7d}")
print()
print(f" Dropped      - Correlation:                                                  {len(SelectedColumns['dropped_correlation']):7d}")
print(f" Features     - Variance:                                                     {len(SelectedColumns['dropped_variance']):7d}")
print(f"              - NaN values:                                                   {len(SelectedColumns['dropped_nans']):7d}")
print(f"              - Image Feature Columns:                                        {len(SelectedColumns['dropped_image_features']):7d}")
print(f"              - Metadata Columns:                                             {len(SelectedColumns['dropped_metadata']):7d}\n")
print(f"              - TOTAL Dropped Columns:                                        {total_dropped:7d}")

# print()
# print(f" Selected     - Metadata columns:                                             {len(ParquetColumns['Metadata']):7d}")
# print(f" Features     - Image feature columns:                                        {len(ParquetColumns['Image']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cells']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cytoplasm']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Nuclei']):7d}\n")
# print(f"              Parquet file total columns:                                     {len(parquetAllColumns):7d}")
# print()
# print(f"              - Selected numeric columns (Cells + Cytoplasm + Nuclei):        {len(parquetAllColumns) - total_dropped:7d}")
# print(f"              - Selected Numeric columns:                                     {len(SelectedColumns['selected']):7d}")
# print()
# print(f" Selected     - Metadata columns:                                             {len(ParquetColumns['Metadata']):7d}")
# print(f" Columns      - Image feature columns:                                        {len(ParquetColumns['Image']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cells']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cytoplasm']):7d}")
# print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Nuclei']):7d}\n")
# print(f"              Parquet file total columns:                                     {len(parquetAllColumns):7d}")

##-----------------------------------------------------------------------
## Remove numeric columns that contains bad values
##-----------------------------------------------------------------------
print()
for key in ParquetColumns.keys():
    before = len(ParquetColumns[key])
    for x in pickle_data['selected_columns']['dropped_nans']: 
        if x in ParquetColumns[key] :
            del ParquetColumns[key][x]
            # print(f" \t FOUND in ParquetColumns['{key}'], deleted -- {x:35s}")
        # else:
            # print(f"\t NOT FOUND in ParquetColumns['{key}'], skipped -- {x:35s}")
            # pass
    # print(f"ParquetColumns['{key}'] : \t ttl columns before / after dropping Nans Columns : {before:5d}/{len(ParquetColumns[key]):5d}")
 
parquetAllColumns = ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei'] | ParquetColumns['Image'] | ParquetColumns['Metadata']
parquetNumericColumns  =  ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei']

READ_PARQUET_COLUMNS = list(ParquetColumns['Metadata'] | parquetNumericColumns)
READ_SELECTED_PARQUET_COLUMNS_ONLY = list(ParquetColumns['Metadata'].keys()) +  sorted(SelectedColumns['selected'])
JOIN_COLUMNS = MetadataColumns + list(parquetNumericColumns)

print()
print(f" Parquet      - Metadata columns:                                             {len(ParquetColumns['Metadata']):7d}")
print(f" File         - Image feature columns:                                        {len(ParquetColumns['Image']):7d}")
print(f" Post-Drop    - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cells']):7d}")
print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Cytoplasm']):7d}")
print(f"              - Numeric columns (Cells + Cytoplasm + Nuclei):                 {len(ParquetColumns['Nuclei']):7d}")
print(f"                                               total columns:                 {len(parquetAllColumns):7d}")
print()
print(f"Selected      - Selected Numeric columns:                                     {len(SelectedColumns['selected']):7d}")
print(f"Columns      ")
print()
print(f" PARQUET COLUMNS : ")
print("-"*60)
print(f"   Metadata Columns : {len(ParquetColumns['Metadata']):4d}    : {ParquetColumns['Metadata']}")
print(f"   Numeric Columns  : {len(parquetNumericColumns):4d}")
# print(f"   Numeric columns :")
print(f"   ------------------")
col_dict = cat_columns(parquetNumericColumns)
print()
print(f"   Total (READ_PARQUET_COLUMNS)     : {len(READ_PARQUET_COLUMNS):4d}  : {READ_PARQUET_COLUMNS[:5]} ...")
print('\n')

print(f" JOIN_COLUMNS : join between parquet and compound extended metadata ")
print("-"*60)
print(f"   Metadata Columns              : {len(MetadataColumns):5d}")
print(f"   profile Numneric Columns      : {len(parquetNumericColumns):5d}")
print(f"   all Columns                   : {len(JOIN_COLUMNS):5d}")   
print('\n')

 Parquet_columns keys : dict_keys(['Metadata', 'Cells', 'Cytoplasm', 'Nuclei', 'Image'])
 Selected_columns keys: dict_keys(['selected', 'dropped_correlation', 'dropped_variance', 'dropped_nans', 'dropped_image_features', 'dropped_metadata'])
3294


 Parquet      - Metadata columns:                                                   3
 File         - Image feature columns:                                           1089
 Pre-Drop     - Numeric columns (Cells + Cytoplasm + Nuclei):                    1249
              - Numeric columns (Cells + Cytoplasm + Nuclei):                    1240
              - Numeric columns (Cells + Cytoplasm + Nuclei):                    1172
                                               total columns:                    4753

 Dropped      - Correlation:                                                     2190
 Features     - Variance:                                                           0
              - NaN values:                                   

In [133]:
##-----------------------------------------------------------------------
## Conversion dictionary for reading CSV files 
##-----------------------------------------------------------------------
profileConversionDict = dict()
## Add numeric features 
## Add Metadata float columns to conversionDict:
## Add Integer metadata columns to conversion Dict
## Add Categorical metadata columns to conversion Dict
profileConversionDict |= {x:y  for x,y in MetadataFloatColumns.items()}
profileConversionDict |= {x:y  for x,y in MetadataIntegerColumns.items()}
profileConversionDict |= {x:y  for x,y in MetadataCategoricalColumns.items()}
profileConversionDict |= {x: np.float32 for x in parquetNumericColumns.keys()} 
print(f" conversion dict length (after addition of Float,Integer,Categorical and Numeric columns): {len(profileConversionDict)}")
## one column is read with no conversion

 conversion dict length (after addition of Float,Integer,Categorical and Numeric columns): 3671


# Sort in Compound order and write to output

In [ ]:
# del df_profiles, ext_metadata, grouped_metadata, df_normalized, df_profile, df_join, df_profiles_list

In [147]:
x = 1
x = 1482  ## 1471 features, 11 metadata 
inputFile = profileInputFile.format(x)
outputFile = profileOutputFile.format(x) 
print(f" Read from {inputFile}")
print(f" Write  to {outputFile}")

 Read from ./output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv
 Write  to ./output_11102023/3_sample_profiles/3sample_profiles_1482_CompoundOrder.csv


In [148]:
df_profiles =  pd.read_csv(inputFile, 
                           low_memory=True
                           # nrows = 20,
                           # skiprows = 0, 
                           # header = 0, 
                           # names = JOIN_COLUMNS,
                           # dtype = conversionDict,
                           # usecols = ['Metadata_JCP2022', 'Metadata_permiation', 'Cells_AreaShape_Area']
                          )

In [149]:
df_profiles.shape
df_profiles.columns
# df_profiles.info(verbose=True)
# df_profiles.memory_usage(deep=True).sum()
# df_profiles.memory_usage().sum()

(346542, 1482)

Index(['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin',
       'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA',
       ...
       'Nuclei_Texture_SumAverage_DNA_10_01_256', 'Nuclei_Texture_SumAverage_ER_10_01_256', 'Nuclei_Texture_SumAverage_Mito_10_01_256',
       'Nuclei_Texture_SumAverage_RNA_10_01_256', 'Nuclei_Texture_SumEntropy_DNA_10_03_256', 'Nuclei_Texture_SumVariance_AGP_10_03_256',
       'Nuclei_Texture_SumVariance_DNA_10_03_256', 'Nuclei_Texture_SumVariance_ER_10_01_256', 'Nuclei_Texture_SumVariance_Mito_10_03_256',
       'Nuclei_Texture_SumVariance_RNA_10_01_256'],
      dtype='object', length=1482)

In [150]:
df_profiles.head(25)
df_profiles.tail(25)

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,0.443768,0.062235,-0.648585,0.308744,0.073046,-0.245728,0.099481,-0.107570,-0.558575,0.174041
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,1.705322,3.242936,-1.327198,1.111929,2.102121,1.878933,3.540586,2.529848,-1.022145,0.753343
2,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,-0.408875,-0.214203,-0.092149,-0.658746,-0.453523,5.058665,-0.418815,-0.455386,0.699883,-0.680918
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.438654,-0.627846,-0.707734,-0.504685,-0.398547,-0.303720,-0.389571,0.922415,-0.724165,-0.202291
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,0.683232,-0.350120,0.300466,-0.384838,0.766679,-0.162195,-0.055417,-0.199495,4.481964,-0.165720
5,source_10,2021_08_23_U2OS_48_hr_run18,Dest210823-174733,I13,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.385001,-0.620632,-0.703596,-0.348666,-0.692979,0.238075,-0.492878,-0.662979,-0.251163,-0.245822
6,source_1,Batch6_20221102,UL000587,O34,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-1.358390,-0.948076,-1.779347,-1.717747,-1.499519,-0.395883,-0.842981,-0.349826,-0.925387,-0.687635
7,source_6,p211109CPU2OS48hw384exp035JUMP,110000296167,H17,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-0.309675,0.356243,-0.550914,-0.129928,-0.289333,0.247451,-0.187486,-0.203569,-0.008298,-0.340577
8,source_3,CP_26_all_Phenix1,J12455b,H17,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,0.270542,-0.265563,-0.349706,-0.242253,0.212501,-0.225538,-0.052160,-0.232956,-0.158716,-0.185053
9,source_2,20210712_Batch_5,1053597875,K10,JCP2022_038591,-9222975330808494972,4,51.22,3.936130,1.709440,...,0.919958,1.159541,0.540352,0.923058,1.803371,0.638770,1.548086,0.621993,-0.278182,0.397897


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
346517,source_5,JUMPCPE-20210806-Run17_20210807_000211,APTJUM301,C22,JCP2022_113137,9221606593339848526,14,71.51,4.269837,1.854367,...,0.628857,0.472722,0.481285,0.743271,0.669582,-0.239123,0.302745,0.383771,1.101200,0.556036
346518,source_6,p210831CPU2OS48hw384exp024JUMP,110000294904,M05,JCP2022_025173,9221783401606620876,12,62.30,4.131961,1.794488,...,1.071991,-0.173546,0.755830,0.908355,1.578703,-0.011290,1.301099,-0.642553,-0.249592,-0.021833
346519,source_3,CP_26_all_Phenix1,J12428c,M05,JCP2022_025173,9221783401606620876,12,62.30,4.131961,1.794488,...,0.033905,-0.088639,-0.593034,-0.047083,0.158208,-0.517159,-0.120764,-0.679147,-0.638654,-0.721445
346520,source_1,Batch3_20221010,UL001667,Z09,JCP2022_025173,9221783401606620876,12,62.30,4.131961,1.794488,...,1.045126,1.168534,1.161564,1.304712,0.996569,0.829639,1.094599,1.103649,1.049664,1.324145
346521,source_2,20210621_Batch_3,1053599619,G21,JCP2022_074903,9222053201753317661,13,96.75,4.572130,1.985651,...,0.042882,-0.767861,-0.335751,-0.839283,-0.088811,-0.267524,-0.072567,0.645874,-0.276915,-0.767783
346522,source_1,Batch2_20221006,UL001677,W43,JCP2022_074903,9222053201753317661,13,96.75,4.572130,1.985651,...,0.059740,-0.420108,-0.984086,-0.841276,0.041166,0.002753,-0.163884,-0.599887,-0.472253,-0.685085
346523,source_3,CP_25_all_Phenix1,C13495aW,L22,JCP2022_074903,9222053201753317661,13,96.75,4.572130,1.985651,...,0.161756,-0.323060,0.229337,-0.404724,0.181975,-0.113182,-0.069909,-0.073722,-0.127787,-0.170992
346524,source_11,Batch1,EC000028,J22,JCP2022_017743,9222186318791160018,2,76.08,4.331785,1.881271,...,-2.271280,-1.895477,-1.192394,-2.003391,-0.698224,-0.931657,-1.136927,-0.792447,-0.192490,-1.126262
346525,source_5,JUMPCPE-20210706-Run06_20210706_235916,APTJUM212,H15,JCP2022_017743,9222186318791160018,2,76.08,4.331785,1.881271,...,-0.333781,0.316921,-1.758729,-1.375661,-0.184179,2.044889,-0.032444,0.485405,-0.526663,-0.863440
346526,source_9,20211013-Run14,GR00003290,H15,JCP2022_017743,9222186318791160018,2,76.08,4.331785,1.881271,...,-0.952216,-0.164309,-0.884471,-0.663352,-1.239603,-0.159411,-1.260185,-0.662479,-1.084258,-1.112597


In [136]:
# import operator 
# df_profiles['Metadata_Bin'] = df_profiles['Metadata_Hash'].map(lambda x: operator.mod(x,16))

In [138]:
df_profiles['Metadata_Bin'].value_counts()

Metadata_Bin
12    22014
15    21927
7     21834
2     21804
10    21783
0     21780
4     21756
3     21714
13    21654
11    21645
14    21633
9     21606
1     21435
6     21351
5     21315
8     21291
Name: count, dtype: int64

In [119]:
df_profiles.index

RangeIndex(start=0, stop=346542, step=1)

In [120]:
df_profiles.set_index(['Metadata_JCP2022'], drop=False, append = False, inplace=True )
# df_profiles.reset_index(drop = True, inplace=True)

In [121]:
# df_profiles.shape
# df_profiles.columns
df_profiles.head(7)
df_profiles.tail(7)

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
Metadata_JCP2022,,,,,,,,,,,,,,,,,,,,,
JCP2022_000082,source_1,Batch1_20221004,UL001641,AA40,JCP2022_000082,-1920040449369071690,10,64.89,4.172694,1.812178,...,0.438577,-0.051575,0.279409,0.767113,0.143125,0.498506,-0.147239,-0.072483,0.059082,0.547492
JCP2022_000137,source_1,Batch1_20221004,UL001641,Z22,JCP2022_000137,-9070688483616626497,11,60.28,4.099000,1.780173,...,0.522905,0.206919,-0.737207,0.050593,0.889717,-0.436981,0.109206,-0.024600,-0.490239,-0.262859
JCP2022_000428,source_1,Batch1_20221004,UL001641,Z32,JCP2022_000428,4317235005617559002,2,94.53,4.548917,1.975570,...,0.258570,0.235806,0.430072,0.565591,0.588245,-0.338327,-0.014979,0.087962,-0.005913,0.229194
JCP2022_000586,source_1,Batch1_20221004,UL001641,A26,JCP2022_000586,9012499819759238838,6,74.90,4.316154,1.874482,...,-0.034145,0.245875,1.385014,0.838357,0.164862,0.387234,-0.024273,0.050859,1.224053,0.311790
JCP2022_000727,source_1,Batch1_20221004,UL001641,E35,JCP2022_000727,6430107330365911187,11,41.90,3.735286,1.622214,...,-0.866072,0.814530,1.890379,0.465849,-0.799093,0.974528,-0.240034,0.443196,1.119420,-0.073996
JCP2022_000931,source_1,Batch1_20221004,UL001641,J29,JCP2022_000931,-4737815026204311369,3,100.68,4.611947,2.002943,...,-0.308211,1.256083,-0.406539,0.040415,-0.542983,-0.046594,-0.114935,0.724475,-0.301127,-0.073744
JCP2022_000940,source_1,Batch1_20221004,UL001641,AE32,JCP2022_000940,3151155857995480151,11,81.63,4.402197,1.911850,...,-0.527949,-0.366357,0.274680,-0.185940,-0.938016,0.183939,-0.484898,-0.327941,0.009168,-0.016833


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
Metadata_JCP2022,,,,,,,,,,,,,,,,,,,,,
JCP2022_114424,source_9,20211103-Run16,GR00004421,X06,JCP2022_114424,-2136046343981961576,0,90.17,4.501697,1.955062,...,-1.930686,0.447849,0.672740,0.425575,0.797944,0.333247,0.262444,0.074133,0.371755,0.182448
JCP2022_114628,source_9,20211103-Run16,GR00004421,E09,JCP2022_114628,-6711676118285912590,2,64.19,4.161847,1.807467,...,-2.010728,1.020253,0.434322,0.826195,-0.189718,-0.562614,0.263729,-0.050986,-0.208017,0.046498
JCP2022_114732,source_9,20211103-Run16,GR00004421,H15,JCP2022_114732,4002782965987436773,1,32.26,3.473828,1.508664,...,0.077809,3.444471,0.527644,3.230805,-1.427530,2.646495,-0.324206,1.528266,0.789272,2.473024
JCP2022_114814,source_9,20211103-Run16,GR00004421,Y11,JCP2022_114814,6486431468446205044,4,57.53,4.052307,1.759894,...,1.816665,-0.837454,-0.638099,-0.844296,-1.024990,-0.588304,-1.261337,-0.676002,-0.640375,-0.803725
JCP2022_115139,source_9,20211103-Run16,GR00004421,B33,JCP2022_115139,-6572166729817227893,7,121.19,4.797360,2.083467,...,1.558703,0.477761,0.702035,-0.067837,0.345086,-0.351046,-0.030078,0.324098,0.209365,-0.442027
JCP2022_115733,source_9,20211103-Run16,GR00004421,I42,JCP2022_115733,-748165371697138316,4,72.11,4.278193,1.857995,...,0.757216,0.120178,0.019669,-0.161075,1.020779,0.525285,0.573911,0.326046,0.205970,-0.207513
JCP2022_116507,source_9,20211103-Run16,GR00004421,L46,JCP2022_116507,7615022959335138959,11,56.59,4.035832,1.752740,...,1.501378,0.820614,0.719263,0.774854,-0.044516,-0.224126,-0.099275,0.130524,-0.024717,0.237543


In [122]:
df_profiles.sort_index(inplace = True)

In [123]:
df_profiles.head(10)
df_profiles.tail(10)

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
Metadata_JCP2022,,,,,,,,,,,,,,,,,,,,,
JCP2022_000001,source_1,Batch5_20221030,UL001783,C29,JCP2022_000001,-8640665944756343816,4,63.49,4.150882,1.802705,...,-0.458914,-1.371064,-0.544489,1.860768,0.053360,-1.044631,-0.164694,-0.447111,-0.210514,1.771579
JCP2022_000001,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-144945,C07,JCP2022_000001,-8640665944756343816,4,63.49,4.150882,1.802705,...,-0.329434,0.519854,-1.321279,1.215553,-0.326494,1.145019,-0.621820,-0.006298,-1.016483,1.013539
JCP2022_000001,source_6,p210928CPU2OS48hw384exp030JUMP,110000296383,B15,JCP2022_000001,-8640665944756343816,4,63.49,4.150882,1.802705,...,-0.345951,-0.269666,-0.389647,0.363030,-0.798532,0.097936,-0.059971,-0.878109,-0.705732,-0.311763
JCP2022_000002,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-163143,D21,JCP2022_000002,-2058530355611931119,1,35.83,3.578786,1.554247,...,1.398717,-0.649733,-0.842234,-0.056985,1.385063,0.594396,0.093356,-0.339999,-0.068153,-0.048143
JCP2022_000002,source_8,J1,A1170405,B22,JCP2022_000002,-2058530355611931119,1,35.83,3.578786,1.554247,...,-2.263589,-1.022479,0.150967,-0.583586,-2.196386,-0.575999,-0.620191,-0.547518,0.237101,-0.556583
JCP2022_000002,source_1,Batch3_20221010,UL000087,D43,JCP2022_000002,-2058530355611931119,1,35.83,3.578786,1.554247,...,-0.974689,-0.236813,0.778883,-0.350895,-0.754805,-0.498228,-0.275179,-0.580250,0.398299,-0.316116
JCP2022_000004,source_1,Batch3_20221010,UL001785,Q42,JCP2022_000004,2222818783381135156,4,67.23,4.208120,1.827563,...,-1.288501,-1.473039,-1.041408,-2.025411,-1.574590,1.824743,0.374160,0.126741,-0.384062,-0.050030
JCP2022_000004,source_6,p210928CPU2OS48hw384exp030JUMP,110000296367,I21,JCP2022_000004,2222818783381135156,4,67.23,4.208120,1.827563,...,0.749900,1.189327,-0.543651,0.270924,0.928818,-0.271433,0.033352,0.266131,-0.299621,0.353198
JCP2022_000004,source_10,2021_06_21_U2OS_48_hr_run7,Dest210621-135001,A20,JCP2022_000004,2222818783381135156,4,67.23,4.208120,1.827563,...,1.718655,1.964404,-0.003414,1.244441,1.634077,3.687617,2.235745,2.035113,1.812285,2.155622


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
Metadata_JCP2022,,,,,,,,,,,,,,,,,,,,,
JCP2022_116750,source_9,20211102-Run15,GR00004396,U19,JCP2022_116750,9161564570943414536,8,66.32,4.194492,1.821645,...,-0.608053,-0.257768,-1.021150,-0.322636,-0.425168,-0.431935,-0.335616,-0.220644,-0.432304,-0.385689
JCP2022_116751,source_8,J1,A1170428,J13,JCP2022_116751,5024933905542715644,0,85.30,4.446174,1.930949,...,-0.034675,-0.432163,-0.617461,-0.746341,0.249512,-0.918198,-0.111235,-0.160633,-0.610097,-0.502493
JCP2022_116751,source_1,Batch6_20221102,UL000599,S26,JCP2022_116751,5024933905542715644,0,85.30,4.446174,1.930949,...,0.143227,0.154577,0.085366,-0.228974,-0.404697,-0.080999,-0.101874,0.428692,0.098301,-0.104493
JCP2022_116751,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152634,C04,JCP2022_116751,5024933905542715644,0,85.30,4.446174,1.930949,...,-0.617530,0.218408,1.534000,-0.544757,-0.632089,0.225475,-0.579470,-0.196968,1.447412,-0.589395
JCP2022_116752,source_2,20210614_Batch_1,1053600759,B15,JCP2022_116752,8043583690001075349,9,71.95,4.275971,1.857031,...,-0.066296,0.025305,-0.038387,0.212261,-0.144381,0.112191,-0.077720,-0.036034,-0.067276,0.278774
JCP2022_116752,source_8,J2,A1170487,I09,JCP2022_116752,8043583690001075349,9,71.95,4.275971,1.857031,...,0.336270,0.077642,-0.514690,-0.023691,0.506268,-0.213596,-0.117489,-0.129574,-0.182804,-0.052572
JCP2022_116752,source_10,2021_06_14_U2OS_48_hr_run5,Dest210614-163244,B15,JCP2022_116752,8043583690001075349,9,71.95,4.275971,1.857031,...,-0.224899,0.625847,-1.067718,0.712468,-0.088796,0.643967,-0.304517,0.258705,-0.343265,0.400791
JCP2022_116753,source_11,Batch5,EC000101,M17,JCP2022_116753,-9112886486787329276,4,54.29,3.994340,1.734720,...,-0.035762,-0.435662,-0.447610,-0.421927,-0.029684,-0.132829,-0.062805,-0.127680,-0.092514,-0.086078
JCP2022_116753,source_9,20210918-Run12,GR00004382,AE42,JCP2022_116753,-9112886486787329276,4,54.29,3.994340,1.734720,...,0.747758,1.080333,0.006025,1.046426,0.215379,0.560407,0.695249,0.714755,0.024474,1.070501


In [124]:
df_profiles.reset_index(drop = True, inplace = True)

In [139]:
df_profiles.head(10)
df_profiles.tail(10)

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,source_1,Batch5_20221030,UL001783,C29,JCP2022_000001,-8640665944756343816,8,63.49,4.150882,1.802705,...,-0.458914,-1.371064,-0.544489,1.860768,0.053360,-1.044631,-0.164694,-0.447111,-0.210514,1.771579
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-144945,C07,JCP2022_000001,-8640665944756343816,8,63.49,4.150882,1.802705,...,-0.329434,0.519854,-1.321279,1.215553,-0.326494,1.145019,-0.621820,-0.006298,-1.016483,1.013539
2,source_6,p210928CPU2OS48hw384exp030JUMP,110000296383,B15,JCP2022_000001,-8640665944756343816,8,63.49,4.150882,1.802705,...,-0.345951,-0.269666,-0.389647,0.363030,-0.798532,0.097936,-0.059971,-0.878109,-0.705732,-0.311763
3,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-163143,D21,JCP2022_000002,-2058530355611931119,1,35.83,3.578786,1.554247,...,1.398717,-0.649733,-0.842234,-0.056985,1.385063,0.594396,0.093356,-0.339999,-0.068153,-0.048143
4,source_8,J1,A1170405,B22,JCP2022_000002,-2058530355611931119,1,35.83,3.578786,1.554247,...,-2.263589,-1.022479,0.150967,-0.583586,-2.196386,-0.575999,-0.620191,-0.547518,0.237101,-0.556583
5,source_1,Batch3_20221010,UL000087,D43,JCP2022_000002,-2058530355611931119,1,35.83,3.578786,1.554247,...,-0.974689,-0.236813,0.778883,-0.350895,-0.754805,-0.498228,-0.275179,-0.580250,0.398299,-0.316116
6,source_1,Batch3_20221010,UL001785,Q42,JCP2022_000004,2222818783381135156,4,67.23,4.208120,1.827563,...,-1.288501,-1.473039,-1.041408,-2.025411,-1.574590,1.824743,0.374160,0.126741,-0.384062,-0.050030
7,source_6,p210928CPU2OS48hw384exp030JUMP,110000296367,I21,JCP2022_000004,2222818783381135156,4,67.23,4.208120,1.827563,...,0.749900,1.189327,-0.543651,0.270924,0.928818,-0.271433,0.033352,0.266131,-0.299621,0.353198
8,source_10,2021_06_21_U2OS_48_hr_run7,Dest210621-135001,A20,JCP2022_000004,2222818783381135156,4,67.23,4.208120,1.827563,...,1.718655,1.964404,-0.003414,1.244441,1.634077,3.687617,2.235745,2.035113,1.812285,2.155622
9,source_9,20211103-Run16,GR00004417,I20,JCP2022_000005,-5338746385487734035,13,84.89,4.441356,1.928857,...,1.230853,0.513702,-0.154136,0.339097,1.042908,0.828966,1.182196,0.585454,0.251206,0.221291


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
346532,source_9,20211102-Run15,GR00004396,U19,JCP2022_116750,9161564570943414536,8,66.32,4.194492,1.821645,...,-0.608053,-0.257768,-1.021150,-0.322636,-0.425168,-0.431935,-0.335616,-0.220644,-0.432304,-0.385689
346533,source_8,J1,A1170428,J13,JCP2022_116751,5024933905542715644,12,85.30,4.446174,1.930949,...,-0.034675,-0.432163,-0.617461,-0.746341,0.249512,-0.918198,-0.111235,-0.160633,-0.610097,-0.502493
346534,source_1,Batch6_20221102,UL000599,S26,JCP2022_116751,5024933905542715644,12,85.30,4.446174,1.930949,...,0.143227,0.154577,0.085366,-0.228974,-0.404697,-0.080999,-0.101874,0.428692,0.098301,-0.104493
346535,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152634,C04,JCP2022_116751,5024933905542715644,12,85.30,4.446174,1.930949,...,-0.617530,0.218408,1.534000,-0.544757,-0.632089,0.225475,-0.579470,-0.196968,1.447412,-0.589395
346536,source_2,20210614_Batch_1,1053600759,B15,JCP2022_116752,8043583690001075349,5,71.95,4.275971,1.857031,...,-0.066296,0.025305,-0.038387,0.212261,-0.144381,0.112191,-0.077720,-0.036034,-0.067276,0.278774
346537,source_8,J2,A1170487,I09,JCP2022_116752,8043583690001075349,5,71.95,4.275971,1.857031,...,0.336270,0.077642,-0.514690,-0.023691,0.506268,-0.213596,-0.117489,-0.129574,-0.182804,-0.052572
346538,source_10,2021_06_14_U2OS_48_hr_run5,Dest210614-163244,B15,JCP2022_116752,8043583690001075349,5,71.95,4.275971,1.857031,...,-0.224899,0.625847,-1.067718,0.712468,-0.088796,0.643967,-0.304517,0.258705,-0.343265,0.400791
346539,source_11,Batch5,EC000101,M17,JCP2022_116753,-9112886486787329276,4,54.29,3.994340,1.734720,...,-0.035762,-0.435662,-0.447610,-0.421927,-0.029684,-0.132829,-0.062805,-0.127680,-0.092514,-0.086078
346540,source_9,20210918-Run12,GR00004382,AE42,JCP2022_116753,-9112886486787329276,4,54.29,3.994340,1.734720,...,0.747758,1.080333,0.006025,1.046426,0.215379,0.560407,0.695249,0.714755,0.024474,1.070501
346541,source_9,20210918-Run12,GR00004385,O42,JCP2022_116753,-9112886486787329276,4,54.29,3.994340,1.734720,...,-0.038226,-0.980228,0.634307,-1.404200,-0.483003,0.618864,-0.455687,-0.091167,0.472225,-0.289596


In [140]:
print(f" Write to {outputFile}")

 Write to ./output_11102023/3_sample_profiles/3sample_profiles_1482_CompoundOrder.csv


In [141]:
df_profiles.to_csv(outputFile, mode = 'w', index = False, header = True, )

# Sort in Hash order and write to output

In [ ]:
# del df_profiles, ext_metadata, grouped_metadata, df_normalized, df_profile, df_join, df_profiles_list

In [63]:
print(f" CompoundProfiles3SampleFile   :{CompoundProfiles3SampleFile}")
print(f" CompoundProfiles2SampleFile   :{CompoundProfiles2SampleFile}")
file_id= 1482
input_file_suffix = ''
input_file_suffix = '_Unordered'
output_file_suffix = '_HashOrder'

INPUT_FILE = CompoundProfiles2SampleFile.format(file_id, input_file_suffix)
OUTPUT_FILE = CompoundProfiles2SampleFile.format(file_id, output_file_suffix)+'.gz' 

print()
print(f" Read from {INPUT_FILE}")
print(f" Write  to {OUTPUT_FILE}")
print()

 CompoundProfiles3SampleFile   :./output_11102023/3_sample_profiles/3sample_profiles_{0:03d}{1:s}.csv
 CompoundProfiles2SampleFile   :./output_11102023/2_sample_profiles/2sample_profiles_{0:03d}{1:s}.csv

 Read from ./output_11102023/2_sample_profiles/2sample_profiles_1482_Unordered.csv
 Write  to ./output_11102023/2_sample_profiles/2sample_profiles_1482_HashOrder.csv.gz



In [64]:
df_profiles =  pd.read_csv(INPUT_FILE, 
                           low_memory=True,
                           # nrows = 20,
                           # skiprows = 0, 
                           # header = 0, 
                           # names = JOIN_COLUMNS,
                           # dtype = conversionDict,
                           # usecols = ['Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_Permiation']
                          )

/tmp/ipykernel_1001857/363442140.py:1: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



In [65]:
df_profiles.shape
df_profiles.columns
# df_profiles.loc[0]
df_profiles.head(5)
df_profiles.tail(10)
# df_profiles.info(verbose=True)
# HEADER_LIST = df_profiles.columns.to_list()
# df_profiles.memory_usage().sum()

(231028, 1482)

Index(['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin',
       'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA',
       ...
       'Nuclei_Texture_SumAverage_DNA_10_01_256', 'Nuclei_Texture_SumAverage_ER_10_01_256', 'Nuclei_Texture_SumAverage_Mito_10_01_256',
       'Nuclei_Texture_SumAverage_RNA_10_01_256', 'Nuclei_Texture_SumEntropy_DNA_10_03_256', 'Nuclei_Texture_SumVariance_AGP_10_03_256',
       'Nuclei_Texture_SumVariance_DNA_10_03_256', 'Nuclei_Texture_SumVariance_ER_10_01_256', 'Nuclei_Texture_SumVariance_Mito_10_03_256',
       'Nuclei_Texture_SumVariance_RNA_10_01_256'],
      dtype='object', length=1482)

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,source_1,Batch1_20221004,UL001641,Z22,JCP2022_000137,-9070688483616626497,11,60.28,4.099000,1.780173,...,0.492849,0.217898,-0.792035,0.023729,0.907163,-0.398245,0.102155,-0.020685,-0.447971,-0.258863
1,source_1,Batch1_20221004,UL001641,Z32,JCP2022_000428,4317235005617559002,2,94.53,4.548917,1.975570,...,0.230296,0.248118,0.506423,0.523815,0.588579,-0.313709,-0.029310,0.084373,-0.008286,0.172546
2,source_1,Batch1_20221004,UL001641,E35,JCP2022_000727,6430107330365911187,11,41.90,3.735286,1.622214,...,-0.886761,0.853556,2.130836,0.426961,-0.877505,0.811267,-0.267559,0.415926,1.013326,-0.093276
3,source_1,Batch1_20221004,UL001641,J29,JCP2022_000931,-4737815026204311369,3,100.68,4.611947,2.002943,...,-0.332662,1.315491,-0.424206,0.013846,-0.606859,-0.063725,-0.135126,0.678454,-0.276289,-0.093055
4,source_1,Batch1_20221004,UL001641,AE32,JCP2022_000940,3151155857995480151,11,81.63,4.402197,1.911850,...,-0.550919,-0.381841,0.333567,-0.205954,-1.024313,0.133817,-0.526778,-0.303803,0.005405,-0.043158


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
231018,source_9,20211103-Run16,GR00004421,B29,JCP2022_107421,7851312155050824172,4,84.56,4.437461,1.927165,...,-0.423043,1.044793,1.801521,1.714509,-1.007930,0.554405,-1.046780,0.388307,1.249818,1.307371
231019,source_9,20211103-Run16,GR00004421,D40,JCP2022_107842,6143457798268220942,2,78.68,4.365389,1.895864,...,0.482047,-0.123022,-0.520065,-0.126249,-0.297010,0.121759,-0.692837,-0.055510,-0.323743,0.027401
231020,source_9,20211103-Run16,GR00004421,K16,JCP2022_108112,7024548821384581315,7,74.76,4.314283,1.873669,...,0.432738,-0.888831,-0.750901,-0.696978,0.162575,0.031390,0.448707,-0.155849,-0.393599,-0.156753
231021,source_9,20211103-Run16,GR00004421,L30,JCP2022_108881,4283081041926708068,8,75.67,4.326382,1.878924,...,-1.340185,-0.829631,-0.649975,-0.523039,0.596309,-0.379410,0.432306,-0.133647,-0.458986,0.106424
231022,source_9,20211103-Run16,GR00004421,AE17,JCP2022_110628,3504912177193634472,0,98.65,4.591578,1.994097,...,-0.778065,-1.191664,-1.686158,-1.361991,-0.773829,-0.822787,-0.294651,-0.406894,-1.113406,-0.570094
231023,source_9,20211103-Run16,GR00004421,AF07,JCP2022_112989,6737580529358542085,5,66.04,4.190261,1.819807,...,-0.584118,1.775437,1.915860,1.645939,-1.048144,-0.080104,-0.575364,0.549139,0.997264,0.600715
231024,source_9,20211103-Run16,GR00004421,AA05,JCP2022_113080,2755737618301067333,5,77.97,4.356324,1.891928,...,-0.028573,0.716662,1.779197,0.879557,-0.891598,0.158234,-1.098460,0.012578,0.969010,0.061043
231025,source_9,20211103-Run16,GR00004421,C08,JCP2022_113175,-4635564294545534988,0,32.78,3.489819,1.515609,...,-0.074594,1.474628,3.092740,2.298495,-1.437355,1.279717,-0.648594,0.120241,2.370742,1.455255
231026,source_9,20211103-Run16,GR00004421,E09,JCP2022_114628,-6711676118285912590,2,64.19,4.161847,1.807467,...,-2.092308,1.190517,0.498109,0.891899,-0.193603,-0.547733,0.306252,-0.059638,-0.209431,0.046675
231027,source_9,20211103-Run16,GR00004421,Y11,JCP2022_114814,6486431468446205044,4,57.53,4.052307,1.759894,...,1.858309,-0.883518,-0.593790,-0.826004,-1.028037,-0.571216,-1.505228,-0.771989,-0.668453,-0.786138


In [66]:
df_profiles.index

RangeIndex(start=0, stop=231028, step=1)

In [67]:
df_profiles.set_index(['Metadata_Hash'], drop=False, append = False, inplace=True )
# df_profiles.reset_index(drop = True, inplace=True)

In [69]:
df_profiles.sort_index(inplace = True)

In [70]:
df_profiles.head(10)
df_profiles.tail(10)

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
Metadata_Hash,,,,,,,,,,,,,,,,,,,,,
-9223347314827979542,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,2,71.18,4.265212,1.852358,...,1.809368,3.337135,-1.276324,1.156037,2.196125,1.834885,3.639664,3.459966,-1.205182,0.779201
-9223347314827979542,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,2,71.18,4.265212,1.852358,...,0.443403,0.047363,-0.681948,0.270548,0.060409,-0.246590,0.098544,-0.101091,-0.565178,0.147804
-9223229703754907651,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,1,58.65,4.071588,1.768268,...,0.583367,-0.318395,0.254350,-0.353821,0.697691,-0.171918,-0.042680,-0.176172,4.075710,-0.147697
-9223229703754907651,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,1,58.65,4.071588,1.768268,...,-0.474088,-0.658752,-0.811990,-0.603970,-0.386900,-0.288997,-0.410041,1.141289,-0.925920,-0.239292
-9223005963369866647,source_1,Batch6_20221102,UL000587,O34,JCP2022_051160,-9223005963369866647,5,68.54,4.227418,1.835944,...,-1.455121,-0.997568,-1.942567,-1.938624,-1.667607,-0.420883,-1.188958,-0.372468,-1.183223,-0.887965
-9223005963369866647,source_6,p211109CPU2OS48hw384exp035JUMP,110000296167,H17,JCP2022_051160,-9223005963369866647,5,68.54,4.227418,1.835944,...,-0.335024,0.306608,-0.508667,-0.206817,-0.302789,0.610406,-0.216382,-0.258501,-0.078076,-0.372267
-9222975330808494972,source_3,CP_26_all_Phenix1,J12428b,N06,JCP2022_038591,-9222975330808494972,0,51.22,3.936130,1.709440,...,-0.160295,-0.396355,-0.639134,-0.419577,0.157756,0.008083,-0.210333,-0.259159,-0.521891,-0.461423
-9222975330808494972,source_6,p210831CPU2OS48hw384exp024JUMP,110000294903,N06,JCP2022_038591,-9222975330808494972,0,51.22,3.936130,1.709440,...,-0.638048,-1.481849,-0.649141,-1.298332,-0.566365,-0.445617,-0.083087,0.362087,0.590038,-0.669180
-9222864758624474277,source_10,2021_08_20_U2OS_48_hr_run17,Dest210810-175644,O07,JCP2022_100039,-9222864758624474277,3,63.30,4.147885,1.801404,...,-0.015065,-0.299711,-0.179571,-0.891356,-0.062856,-0.592561,0.602241,1.183228,-0.610894,-0.713007


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
Metadata_Hash,,,,,,,,,,,,,,,,,,,,,
9222285918691659081,source_5,JUMPCPE-20210908-Run28_20210909_072022,AETJUM105,P15,JCP2022_060479,9222285918691659081,9,102.95,4.634243,2.012626,...,-1.678282,-0.742371,-0.036795,-0.478807,-1.937646,-0.817725,-1.418260,-0.599040,-0.835329,-0.519536
9222285918691659081,source_11,Batch4,EC000107,K10,JCP2022_060479,9222285918691659081,9,102.95,4.634243,2.012626,...,0.053443,-0.191211,0.078778,-0.021335,0.456247,-0.280813,0.253166,-0.415770,-0.029218,-0.318301
9222418181275619136,source_10,2021_08_20_U2OS_48_hr_run17,Dest210810-180316,C20,JCP2022_110705,9222418181275619136,0,38.33,3.646233,1.583539,...,1.258486,0.454688,1.983555,1.520577,1.733437,0.567238,1.873856,0.175452,1.701109,1.345346
9222418181275619136,source_2,20210712_Batch_5,1053597806,C20,JCP2022_110705,9222418181275619136,0,38.33,3.646233,1.583539,...,0.423443,0.987636,0.819246,0.981055,0.402290,0.454137,0.405991,0.672437,-0.146229,0.697928
9222499262440709243,source_11,Batch5,EC000075,O21,JCP2022_017353,9222499262440709243,11,62.06,4.128102,1.792812,...,-0.193315,-0.160149,1.103814,-0.235727,0.125977,-0.147652,-0.264310,-0.100797,0.275493,-0.158203
9222499262440709243,source_9,20210914-Run9,GR00003303,AA45,JCP2022_017353,9222499262440709243,11,62.06,4.128102,1.792812,...,-0.506650,-0.648240,0.634524,-0.268596,-1.124347,-0.133345,-0.424215,-0.189925,-0.022563,-0.121902
9222528730267146426,source_9,20211103-Run16,GR00004410,R43,JCP2022_040466,9222528730267146426,6,98.00,4.584967,1.991226,...,-0.459075,0.588972,1.021280,0.713711,-0.482989,0.300435,-0.202340,0.368767,0.914307,0.692342
9222528730267146426,source_9,20211103-Run16,GR00004412,B19,JCP2022_040466,9222528730267146426,6,98.00,4.584967,1.991226,...,0.418346,0.977722,-0.177036,0.187285,2.219393,-0.290201,1.143182,1.478179,0.186903,1.058180
9223213570900537648,source_6,p211109CPU2OS48hw384exp035JUMP,110000296165,N13,JCP2022_102583,9223213570900537648,4,56.14,4.027849,1.749272,...,0.485312,1.927512,1.005826,1.016068,0.422560,-0.429561,-0.173206,-0.351267,1.396463,0.020549


In [71]:
df_profiles.reset_index(drop = True, inplace = True)

In [72]:
df_profiles.head(10)
df_profiles.tail(10)

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,2,71.18,4.265212,1.852358,...,1.809368,3.337135,-1.276324,1.156037,2.196125,1.834885,3.639664,3.459966,-1.205182,0.779201
1,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,2,71.18,4.265212,1.852358,...,0.443403,0.047363,-0.681948,0.270548,0.060409,-0.246590,0.098544,-0.101091,-0.565178,0.147804
2,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,1,58.65,4.071588,1.768268,...,0.583367,-0.318395,0.254350,-0.353821,0.697691,-0.171918,-0.042680,-0.176172,4.075710,-0.147697
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,1,58.65,4.071588,1.768268,...,-0.474088,-0.658752,-0.811990,-0.603970,-0.386900,-0.288997,-0.410041,1.141289,-0.925920,-0.239292
4,source_1,Batch6_20221102,UL000587,O34,JCP2022_051160,-9223005963369866647,5,68.54,4.227418,1.835944,...,-1.455121,-0.997568,-1.942567,-1.938624,-1.667607,-0.420883,-1.188958,-0.372468,-1.183223,-0.887965
5,source_6,p211109CPU2OS48hw384exp035JUMP,110000296167,H17,JCP2022_051160,-9223005963369866647,5,68.54,4.227418,1.835944,...,-0.335024,0.306608,-0.508667,-0.206817,-0.302789,0.610406,-0.216382,-0.258501,-0.078076,-0.372267
6,source_3,CP_26_all_Phenix1,J12428b,N06,JCP2022_038591,-9222975330808494972,0,51.22,3.936130,1.709440,...,-0.160295,-0.396355,-0.639134,-0.419577,0.157756,0.008083,-0.210333,-0.259159,-0.521891,-0.461423
7,source_6,p210831CPU2OS48hw384exp024JUMP,110000294903,N06,JCP2022_038591,-9222975330808494972,0,51.22,3.936130,1.709440,...,-0.638048,-1.481849,-0.649141,-1.298332,-0.566365,-0.445617,-0.083087,0.362087,0.590038,-0.669180
8,source_10,2021_08_20_U2OS_48_hr_run17,Dest210810-175644,O07,JCP2022_100039,-9222864758624474277,3,63.30,4.147885,1.801404,...,-0.015065,-0.299711,-0.179571,-0.891356,-0.062856,-0.592561,0.602241,1.183228,-0.610894,-0.713007
9,source_2,20211003_Batch_13,1086289730,O07,JCP2022_100039,-9222864758624474277,3,63.30,4.147885,1.801404,...,1.815162,1.684873,0.413203,1.194111,1.223029,-0.068906,1.363251,-0.012265,0.332452,-0.450587


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
231018,source_5,JUMPCPE-20210908-Run28_20210909_072022,AETJUM105,P15,JCP2022_060479,9222285918691659081,9,102.95,4.634243,2.012626,...,-1.678282,-0.742371,-0.036795,-0.478807,-1.937646,-0.817725,-1.418260,-0.599040,-0.835329,-0.519536
231019,source_11,Batch4,EC000107,K10,JCP2022_060479,9222285918691659081,9,102.95,4.634243,2.012626,...,0.053443,-0.191211,0.078778,-0.021335,0.456247,-0.280813,0.253166,-0.415770,-0.029218,-0.318301
231020,source_10,2021_08_20_U2OS_48_hr_run17,Dest210810-180316,C20,JCP2022_110705,9222418181275619136,0,38.33,3.646233,1.583539,...,1.258486,0.454688,1.983555,1.520577,1.733437,0.567238,1.873856,0.175452,1.701109,1.345346
231021,source_2,20210712_Batch_5,1053597806,C20,JCP2022_110705,9222418181275619136,0,38.33,3.646233,1.583539,...,0.423443,0.987636,0.819246,0.981055,0.402290,0.454137,0.405991,0.672437,-0.146229,0.697928
231022,source_11,Batch5,EC000075,O21,JCP2022_017353,9222499262440709243,11,62.06,4.128102,1.792812,...,-0.193315,-0.160149,1.103814,-0.235727,0.125977,-0.147652,-0.264310,-0.100797,0.275493,-0.158203
231023,source_9,20210914-Run9,GR00003303,AA45,JCP2022_017353,9222499262440709243,11,62.06,4.128102,1.792812,...,-0.506650,-0.648240,0.634524,-0.268596,-1.124347,-0.133345,-0.424215,-0.189925,-0.022563,-0.121902
231024,source_9,20211103-Run16,GR00004410,R43,JCP2022_040466,9222528730267146426,6,98.00,4.584967,1.991226,...,-0.459075,0.588972,1.021280,0.713711,-0.482989,0.300435,-0.202340,0.368767,0.914307,0.692342
231025,source_9,20211103-Run16,GR00004412,B19,JCP2022_040466,9222528730267146426,6,98.00,4.584967,1.991226,...,0.418346,0.977722,-0.177036,0.187285,2.219393,-0.290201,1.143182,1.478179,0.186903,1.058180
231026,source_6,p211109CPU2OS48hw384exp035JUMP,110000296165,N13,JCP2022_102583,9223213570900537648,4,56.14,4.027849,1.749272,...,0.485312,1.927512,1.005826,1.016068,0.422560,-0.429561,-0.173206,-0.351267,1.396463,0.020549
231027,source_1,Batch4_20221012,UL000093,AB26,JCP2022_102583,9223213570900537648,4,56.14,4.027849,1.749272,...,-0.588762,-0.007227,-1.030003,0.078356,-0.759970,0.004432,-0.490696,-0.681372,-1.272495,-0.567439


In [73]:
print(f" Write to {OUTPUT_FILE}")

 Write to ./output_11102023/2_sample_profiles/2sample_profiles_1482_HashOrder.csv.gz


In [74]:
df_profiles.to_csv(OUTPUT_FILE, mode = 'w', index = False, header = True, )

In [75]:
del df_profiles

# Sort in Bin # order and write in Bin files 

In [ ]:
# del df_profiles, ext_metadata, grouped_metadata, df_normalized, df_profile, df_join, df_profiles_list

In [107]:
NUMBER_OF_BINS = 12
output_files = []
for i in range(NUMBER_OF_BINS):
    outputFile = os.path.join('./output_11102023/', '3_sample_profiles', f"3sample_profiles_1482_bin_{i:03d}.csv")
    output_files.append(outputFile)
for i in range(NUMBER_OF_BINS):
    print(f"  -->    {output_files[i]:65s}")
# print(output_files)

  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_000.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_001.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_002.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_003.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_004.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_005.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_006.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_007.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_008.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_009.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_010.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_011.csv


In [108]:
print(f" CompoundProfiles3SampleFile   :{CompoundProfiles3SampleFile}")
print(f" CompoundProfiles2SampleFile   :{CompoundProfiles2SampleFile}")
file_id= 1482
input_file_suffix = ''
input_file_suffix = '_CompoundOrder' ## 'Unordered'
output_file_suffix = '_HashOrder'

INPUT_FILE = CompoundProfiles3SampleFile.format(file_id, input_file_suffix)

print()
print(f" Read from {INPUT_FILE}")


 CompoundProfiles3SampleFile   :./output_11102023/3_sample_profiles/3sample_profiles_{0:03d}{1:s}.csv
 CompoundProfiles2SampleFile   :./output_11102023/2_sample_profiles/2sample_profiles_{0:03d}{1:s}.csv

 Read from ./output_11102023/3_sample_profiles/3sample_profiles_1482_CompoundOrder.csv


In [109]:
df_profiles =  pd.read_csv(INPUT_FILE, 
                           low_memory=True,
                           # nrows = 20,
                           # skiprows = 0, 
                           # header = 0, 
                           # names = JOIN_COLUMNS,
                           # dtype = conversionDict,
                           # usecols = ['Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_Permiation']
                          )

FileNotFoundError: [Errno 2] No such file or directory: './output_11102023/3_sample_profiles/3sample_profiles_1482_CompoundOrder.csv'

In [106]:
df_profiles.shape
df_profiles.columns
df_profiles.head(5)
df_profiles.tail(5)

(346542, 1482)

Index(['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin',
       'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA',
       ...
       'Nuclei_Texture_SumAverage_DNA_10_01_256', 'Nuclei_Texture_SumAverage_ER_10_01_256', 'Nuclei_Texture_SumAverage_Mito_10_01_256',
       'Nuclei_Texture_SumAverage_RNA_10_01_256', 'Nuclei_Texture_SumEntropy_DNA_10_03_256', 'Nuclei_Texture_SumVariance_AGP_10_03_256',
       'Nuclei_Texture_SumVariance_DNA_10_03_256', 'Nuclei_Texture_SumVariance_ER_10_01_256', 'Nuclei_Texture_SumVariance_Mito_10_03_256',
       'Nuclei_Texture_SumVariance_RNA_10_01_256'],
      dtype='object', length=1482)

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,source_11,Batch5,EC000101,K15,JCP2022_086595,-9222767843993592896,0,42.01,3.737908,1.623353,...,-0.085906,-0.246525,-0.611051,-0.293267,-0.048620,-0.256795,-0.115376,-0.382004,-0.304168,-0.365506
1,source_9,20210918-Run12,GR00004385,E40,JCP2022_086595,-9222767843993592896,0,42.01,3.737908,1.623353,...,-0.191645,0.298424,0.086420,0.091461,0.195888,-0.585053,-0.185089,0.111404,-0.084773,0.199419
2,source_5,JUMPCPE-20210806-Run17_20210807_000211,ATSJUM231,E16,JCP2022_086595,-9222767843993592896,0,42.01,3.737908,1.623353,...,0.941055,0.539791,-0.072088,-0.306165,0.778215,-0.099456,-0.041682,0.723210,-0.234527,-0.112753
3,source_8,J1,A1170421,P05,JCP2022_002068,-9218894441428193744,0,74.53,4.311202,1.872331,...,-0.100050,-0.628107,0.331514,-0.092686,-0.160660,-0.347643,0.762522,-1.214884,-0.270841,-0.348170
4,source_6,p210927CPU2OS48hw384exp029JUMP,110000296309,P05,JCP2022_002068,-9218894441428193744,0,74.53,4.311202,1.872331,...,1.237008,-0.254410,-0.395405,0.569814,1.222061,-0.554041,0.698668,-0.191568,-0.119092,0.982290


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
346537,source_11,Batch2,EC000062,M04,JCP2022_092539,9213911692466032495,15,57.50,4.051785,1.759668,...,-0.685322,-0.138697,0.642430,-0.083706,-0.451505,-0.078741,-0.275593,0.047866,0.016174,-0.028026
346538,source_9,20211013-Run14,GR00003282,B18,JCP2022_092539,9213911692466032495,15,57.50,4.051785,1.759668,...,-0.627854,-0.153829,-0.326598,-1.053974,-0.255164,-0.096937,0.301954,0.026118,-0.224778,-0.350575
346539,source_10,2021_07_05_U2OS_48_hr_run10,Dest210705-144624,H18,JCP2022_036272,9219096116385562239,15,111.71,4.715906,2.048092,...,0.104377,-0.544124,0.949310,0.351205,0.084198,-1.277857,-0.088098,-0.685468,0.403097,0.489222
346540,source_3,CP_27_all_Phenix1,J12450d,L20,JCP2022_036272,9219096116385562239,15,111.71,4.715906,2.048092,...,-0.024512,-0.584158,-0.267737,-0.830045,-0.180191,-0.382974,-0.091091,-0.189499,-0.743165,-0.322569
346541,source_2,20210719_Batch_6,1086293898,H18,JCP2022_036272,9219096116385562239,15,111.71,4.715906,2.048092,...,-0.151475,-0.340775,0.520895,-0.099295,-0.619773,-0.117696,-0.737597,-0.275580,0.270665,-0.011800


In [93]:
df_profiles.index

RangeIndex(start=0, stop=346542, step=1)

In [74]:
df_profiles.set_index(['Metadata_Bin','Metadata_Hash'], drop=False, append = False, inplace=True )
# df_profiles.reset_index(drop = True, inplace=True)

In [95]:
# df_profiles.head(12)
# df_profiles.tail(12)

In [77]:
df_profiles.sort_index(inplace = True)

In [96]:
# df_profiles.head(10)
# df_profiles.tail(10)

In [80]:
df_profiles.reset_index(drop = True, inplace = True)

In [97]:
# df_profiles.head(10)
# df_profiles.tail(10)

## Group by Bin Number

In [85]:
df_grouped_profiles = df_profiles.groupby(by=['Metadata_Bin'])

In [99]:
for ctr, (key_tuple, group_data) in enumerate(df_grouped_profiles):
    key = key_tuple[0]
    logging.info(f" Write bin {key} to  {output_files[key]}  - {group_data.shape[0]} rows")
    # print(f"{ctr:5d}     {key}   {group_data.shape}") 
    # print(group_data.info())
    # group_data.head(16)
    # group_data.tail(16)
    group_data.to_csv(output_files[key], index = False, header = False)
    logging.info(f" Writing bin {key} profiles file ... complete") 

2024-04-23 23:01:18,432 - INFO: -  Write bin 0 to  ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_000.csv  - 21780 rows
2024-04-23 23:02:04,874 - INFO: -  Writing bin 0 profiles file ... complete
2024-04-23 23:02:04,877 - INFO: -  Write bin 1 to  ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_001.csv  - 21435 rows
2024-04-23 23:02:50,850 - INFO: -  Writing bin 1 profiles file ... complete
2024-04-23 23:02:50,853 - INFO: -  Write bin 2 to  ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_002.csv  - 21804 rows
2024-04-23 23:03:37,416 - INFO: -  Writing bin 2 profiles file ... complete
2024-04-23 23:03:37,419 - INFO: -  Write bin 3 to  ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_003.csv  - 21714 rows
2024-04-23 23:04:23,981 - INFO: -  Writing bin 3 profiles file ... complete
2024-04-23 23:04:23,984 - INFO: -  Write bin 4 to  ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_004.csv  - 21756 rows
2024-04-23 23:05:10,5

### Read Each bin and verify count

In [105]:
# BIN_READ = 2

for BIN_READ in  range(16) : ## [7, 8, 9 , 10, 11]:
    # logging.info(f" Read from  {output_files[BIN_READ]}  for bin {BIN_READ} ")
    
    grp =  pd.read_csv(output_files[BIN_READ],
                       names = WRITE_COMPOUND_PROFILE_COLUMNS,
                       # usecols = ['Metadata_JCP2022', 'Metadata_Bin'],
                      )
    logging.info(f" Total rows in bin {BIN_READ} :      {grp.shape}")
    # grp.npartitions  
    # grp.head()
    # ss = grp.map_partitions(len).compute()
    # ss


2024-04-23 23:20:34,949 - INFO: -  Total rows in bin 0 :      (21780, 1482)
2024-04-23 23:20:42,036 - INFO: -  Total rows in bin 1 :      (21435, 1482)
2024-04-23 23:20:49,589 - INFO: -  Total rows in bin 2 :      (21804, 1482)
2024-04-23 23:20:56,806 - INFO: -  Total rows in bin 3 :      (21714, 1482)
2024-04-23 23:21:04,081 - INFO: -  Total rows in bin 4 :      (21756, 1482)
2024-04-23 23:21:11,082 - INFO: -  Total rows in bin 5 :      (21315, 1482)
2024-04-23 23:21:18,497 - INFO: -  Total rows in bin 6 :      (21351, 1482)
2024-04-23 23:21:25,917 - INFO: -  Total rows in bin 7 :      (21834, 1482)
2024-04-23 23:21:33,246 - INFO: -  Total rows in bin 8 :      (21291, 1482)
2024-04-23 23:21:40,591 - INFO: -  Total rows in bin 9 :      (21606, 1482)
2024-04-23 23:21:47,927 - INFO: -  Total rows in bin 10 :      (21783, 1482)
2024-04-23 23:21:55,329 - INFO: -  Total rows in bin 11 :      (21645, 1482)
2024-04-23 23:22:02,860 - INFO: -  Total rows in bin 12 :      (22014, 1482)
2024-04-2

# Read Sample Profile file and get TPSA Permiation counts

Metadata_Permiation

    - '0'                      305214

    - '1'                       41328

In [ ]:
file_reader_1 =  pd.read_csv('./output_11102023/3_sample_profiles/3sample_profiles_1471_HashOrder.csv', 
                        # names = COMPOUND_PROFILE_COLUMNS,
                        header = 0,
                        skiprows = 0,
                        # nrows = 1,
                        # dtype = Xy_columns_dtype,
                        usecols = ['Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin',],
                        # iterator = True,
                        # chunksize = 1000,
                        )

file_reader_1.value_counts()

In [1]:
df_fp = pd.read_pickle(compoundPharmacophoreFile)
df_fp.info()
df_fp.head(10)
df_fp.tail(10)

file_reader_2.value_counts()

NameError: name 'pd' is not defined

# Read Sample Profile file and check for `nan`s

In [ ]:
input_file = './output_11102023/3_sample_profiles/3sample_profiles_1471_HashOrder.csv'
input_file = './output_11102023/3_sample_profiles/3sample_profiles_1471_CompoundOrder.csv'

In [ ]:
file_reader =  pd.read_csv(input_file,
                        # names = COMPOUND_PROFILE_COLUMNS,
                        header = 0,
                        skiprows = 0,
                        # nrows = self.numrows,
                        # dtype = Xy_columns_dtype,
                        # usecols = Xy_columns,
                        iterator = True,
                        chunksize = 10000,
                        )

In [119]:
# del na_rows, rows_read
df_profiles_list = []
# na_rows  = 0 
# rows_read = 0
group_id = 0
ttl_rows = 0

ttl_rows_read = 0
ttl_na_rows = 0 
ttl_bad_columns = set()
df_Nans = pd.DataFrame()

In [120]:
for batch in file_reader:
    print(f" group: {group_id}  size: {len(batch)}")
    
    # ttl_rows_read, ttl_na_rows, ttl_bad_columns, df_Nans = check_df_for_nans(batch, group_id, ttl_rows_read , ttl_na_rows, ttl_bad_columns, df_Nans)    
    batch.head(2)['Metadata_JCP2022']
    batch.tail(2)['Metadata_JCP2022']
    ttl_rows  += len(batch)

    group_id += 1
    # break
    # if ttl_rows > 20000:
        # break
# df_profiles = pd.concat(df_profiles_list)

print("\n")
print(f" # of groups: {group_id} \n")
print(f" ttl_rows    - total number of rows: {ttl_rows}")
print(f" ttl_rows_read - total rows read   : {ttl_rows_read}") 
 
print(f" ttl_na_rows - total invalid rows  : {ttl_na_rows}")
print(f"\n Bad columns: {ttl_bad_columns}")

 group: 0  size: 10000


0    JCP2022_000082
1    JCP2022_000137
Name: Metadata_JCP2022, dtype: object

9998    JCP2022_100349
9999    JCP2022_100383
Name: Metadata_JCP2022, dtype: object

 group: 1  size: 10000


10000    JCP2022_100431
10001    JCP2022_100737
Name: Metadata_JCP2022, dtype: object

...

339998    JCP2022_112791
339999    JCP2022_112950
Name: Metadata_JCP2022, dtype: object

 group: 34  size: 6542


340000    JCP2022_112954
340001    JCP2022_112964
Name: Metadata_JCP2022, dtype: object

346540    JCP2022_115733
346541    JCP2022_116507
Name: Metadata_JCP2022, dtype: object



 # of groups: 35 

 ttl_rows    - total number of rows: 346542
 ttl_rows_read - total rows read   : 0
 ttl_na_rows - total invalid rows  : 0

 Bad columns: set()


In [60]:
# df_Nans_t = df_Nans.transpose()

In [ ]:
# ttl_bad_columns = {'Cells_Neighbors_AngleBetweenNeighbors_5', 'Cells_Neighbors_FirstClosestDistance_5', 'Nuclei_Neighbors_SecondClosestDistance_1', 'Cells_Neighbors_SecondClosestDistance_Adjacent', 
#                    'Nuclei_Neighbors_AngleBetweenNeighbors_1', 'Nuclei_Neighbors_FirstClosestDistance_1'}

In [61]:
print(f"{datetime.now().strftime('%Y%m%d_%H%M%S')}  epoch {starting_epoch+1:4d} of {epochs:4d}")
show_every = 1

for epoch in range(starting_epoch,epochs):
    
    train_loss  = model.epoch_train( train_loader, epoch, factor = snnl_factor, verbose = False)
    model.model_history('train', train_loss )

    val_loss  =  model.epoch_validate( val_loader, epoch, factor = snnl_factor, verbose = False)
    model.model_history('val', val_loss )

    display_epoch_metrics(model, epoch, epochs)
    
    # model.scheduler.step(model.train_loss[-1])

20240429_143130  epoch    2 of    2
 training : Acc: 0.88000  F1: 0.00000   Prec: 0.00000   Recall: 0.00000   fbeta: 0.00000  pos lables: 33.0  preds: 3
 training : Acc: 0.87333  F1: 0.00000   Prec: 0.00000   Recall: 0.00000   fbeta: 0.00000  pos lables: 36.0  preds: 2
 training : Acc: 0.90333  F1: 0.00000   Prec: 0.00000   Recall: 0.00000   fbeta: 0.00000  pos lables: 24.0  preds: 5
 training : Acc: 0.88000  F1: 0.00000   Prec: 0.00000   Recall: 0.00000   fbeta: 0.00000  pos lables: 36.0  preds: 0
 training : Acc: 0.81000  F1: 0.00000   Prec: 0.00000   Recall: 0.00000   fbeta: 0.00000  pos lables: 57.0  preds: 0
 training : Acc: 0.91000  F1: 0.00000   Prec: 0.00000   Recall: 0.00000   fbeta: 0.00000  pos lables: 27.0  preds: 0
 training : Acc: 0.90000  F1: 0.00000   Prec: 0.00000   Recall: 0.00000   fbeta: 0.00000  pos lables: 30.0  preds: 0
 training : Acc: 0.87667  F1: 0.00000   Prec: 0.00000   Recall: 0.00000   fbeta: 0.00000  pos lables: 36.0  preds: 1
 training : Acc: 0.85333  F1

# Split Sample Profiles into Training / Validation / Test files

In [ ]:
## del df_all, df_training,df_val, df_test, df_header, df_profiles 


In [9]:
HEADER_FILE =  '../output_11102023/3_sample_profiles/3sample_profiles_1482_header_info.csv'
header_cols = pd.read_csv(HEADER_FILE).columns.to_list()
# df_header = pd.read_csv(HEADER_FILE).columns.to_list()
# header_cols

In [32]:
ALL_FILE        = '../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_all.csv'
TRAINING_FILE   = '../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv'
VALIDATION_FILE = '../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_validation.csv'
TEST_FILE       = '../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv'

ALL_FILE        = '../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_all.csv'
TRAINING_FILE   = '../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_092024.csv'
VALIDATION_FILE = '../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_val_092024.csv'
TEST_FILE       = '../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_test_092024.csv'


print(ALL_FILE)
print(TRAINING_FILE)
print(VALIDATION_FILE)
print(TEST_FILE)

../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_all.csv
../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_092024.csv
../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_val_092024.csv
../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_test_092024.csv


In [11]:
df_all = pd.read_csv(ALL_FILE, skiprows = 0)  # names = header_cols)
df_all.shape
df_all.info()
df_all.head(10)
df_all.tail(10)

(346542, 1482)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346542 entries, 0 to 346541
Columns: 1482 entries, Metadata_Source to Nuclei_Texture_SumVariance_RNA_10_01_256
dtypes: float64(1474), int64(3), object(5)
memory usage: 3.8+ GB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,0.443768,0.062235,-0.648585,0.308744,0.073046,-0.245728,0.099481,-0.107570,-0.558575,0.174041
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,1.705322,3.242936,-1.327198,1.111929,2.102121,1.878933,3.540586,2.529848,-1.022145,0.753343
2,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,-0.408875,-0.214203,-0.092149,-0.658746,-0.453523,5.058665,-0.418815,-0.455386,0.699883,-0.680918
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.438654,-0.627846,-0.707734,-0.504685,-0.398547,-0.303720,-0.389571,0.922415,-0.724165,-0.202291
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,0.683232,-0.350120,0.300466,-0.384838,0.766679,-0.162195,-0.055417,-0.199495,4.481964,-0.165720
5,source_10,2021_08_23_U2OS_48_hr_run18,Dest210823-174733,I13,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.385001,-0.620632,-0.703596,-0.348666,-0.692979,0.238075,-0.492878,-0.662979,-0.251163,-0.245822
6,source_1,Batch6_20221102,UL000587,O34,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-1.358390,-0.948076,-1.779347,-1.717747,-1.499519,-0.395883,-0.842981,-0.349826,-0.925387,-0.687635
7,source_6,p211109CPU2OS48hw384exp035JUMP,110000296167,H17,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-0.309675,0.356243,-0.550914,-0.129928,-0.289333,0.247451,-0.187486,-0.203569,-0.008298,-0.340577
8,source_3,CP_26_all_Phenix1,J12455b,H17,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,0.270542,-0.265563,-0.349706,-0.242253,0.212501,-0.225538,-0.052160,-0.232956,-0.158716,-0.185053
9,source_2,20210712_Batch_5,1053597875,K10,JCP2022_038591,-9222975330808494972,4,51.22,3.936130,1.709440,...,0.919958,1.159541,0.540352,0.923058,1.803371,0.638770,1.548086,0.621993,-0.278182,0.397897


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
346532,source_2,20210712_Batch_5,1053597806,C20,JCP2022_110705,9222418181275619136,0,38.33,3.646233,1.583539,...,0.458833,1.084517,0.896921,1.067972,0.454446,0.557440,0.466269,0.781128,-0.128855,0.797625
346533,source_5,JUMPCPE-20210902-Run26_20210903_010341,ATSJUM405,K21,JCP2022_017353,9222499262440709243,11,62.06,4.128102,1.792812,...,-0.206507,0.696719,-0.028039,0.392982,-0.003498,0.206295,-0.083914,0.108957,0.091652,0.008775
346534,source_11,Batch5,EC000075,O21,JCP2022_017353,9222499262440709243,11,62.06,4.128102,1.792812,...,-0.142481,-0.154578,1.187859,-0.244756,0.161605,-0.142326,-0.244117,-0.084062,0.328968,-0.157261
346535,source_9,20210914-Run9,GR00003303,AA45,JCP2022_017353,9222499262440709243,11,62.06,4.128102,1.792812,...,-0.488080,-0.578261,0.505306,-0.271835,-1.154452,-0.144179,-0.415379,-0.200171,-0.037033,-0.133708
346536,source_11,Batch2,EC000057,M20,JCP2022_040466,9222528730267146426,10,98.00,4.584967,1.991226,...,0.253976,-0.440405,-0.318161,-0.819331,0.108661,-1.795652,-0.072785,-1.915761,0.023169,-1.966859
346537,source_9,20211103-Run16,GR00004412,B19,JCP2022_040466,9222528730267146426,10,98.00,4.584967,1.991226,...,0.391301,0.998376,-0.097661,0.216977,2.249028,-0.260359,1.257899,1.648227,0.278188,1.119104
346538,source_9,20211103-Run16,GR00004410,R43,JCP2022_040466,9222528730267146426,10,98.00,4.584967,1.991226,...,-0.418996,0.571904,1.048403,0.718896,-0.463099,0.251642,-0.235765,0.266329,0.812662,0.611824
346539,source_6,p211109CPU2OS48hw384exp035JUMP,110000296165,N13,JCP2022_102583,9223213570900537648,0,56.14,4.027849,1.749272,...,0.484977,1.845734,1.085721,1.085987,0.374079,-0.449345,-0.147684,-0.348851,1.703237,0.074853
346540,source_1,Batch4_20221012,UL000093,AB26,JCP2022_102583,9223213570900537648,0,56.14,4.027849,1.749272,...,-0.614314,-0.009914,-1.063452,0.065249,-0.804197,0.016692,-0.550138,-0.725893,-1.315111,-0.610961
346541,source_2,20210712_Batch_5,1053597950,L04,JCP2022_102583,9223213570900537648,0,56.14,4.027849,1.749272,...,-0.431079,-1.173641,0.612221,-1.371075,-0.430376,-1.031749,-0.372571,-1.229382,-0.304055,-1.199587


In [13]:
del df_all

## Write to training, val, and test files

In [14]:
print(f" Write to {TRAINING_FILE}")

 Write to ../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv


In [15]:
# df_all.loc[0:277_199].to_csv(TRAINING_FILE, mode = 'w', index = False, header = True )

NameError: name 'df_all' is not defined

In [111]:
print(f" Write to {VALIDATION_FILE}")

 Write to ./output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_validation.csv


In [16]:
# df_all.loc[277_200:298_799].to_csv(VALIDATION_FILE, mode = 'w', index = False, header = True )

In [113]:
print(f" Write to {TEST_FILE}")

 Write to ./output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv


In [17]:
# df_all.loc[298_800:311_399].to_csv(TEST_FILE, mode = 'w', index = False, header = True )

## Read training, validation and test files

#### Read training file

In [33]:
print(f" Read {TRAINING_FILE}")

 Read ../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_092024.csv


In [34]:
df_training =  pd.read_csv(TRAINING_FILE, skiprows = 0)

In [35]:
df_training.shape
df_training.info()
df_training.head(5)
df_training.tail(10)
df_training.loc[277196:277205].Metadata_Source

(312000, 1482)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312000 entries, 0 to 311999
Columns: 1482 entries, Metadata_Source to Nuclei_Texture_SumVariance_RNA_10_01_256
dtypes: float64(1474), int64(3), object(5)
memory usage: 3.4+ GB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,0.443768,0.062235,-0.648585,0.308744,0.073046,-0.245728,0.099481,-0.107570,-0.558575,0.174041
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,1.705322,3.242936,-1.327198,1.111929,2.102121,1.878933,3.540586,2.529848,-1.022145,0.753343
2,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,-0.408875,-0.214203,-0.092149,-0.658746,-0.453523,5.058665,-0.418815,-0.455386,0.699883,-0.680918
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.438654,-0.627846,-0.707734,-0.504685,-0.398547,-0.303720,-0.389571,0.922415,-0.724165,-0.202291
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,0.683232,-0.350120,0.300466,-0.384838,0.766679,-0.162195,-0.055417,-0.199495,4.481964,-0.165720


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
311990,source_10,2021_06_07_U2OS_48_hr_run3,Dest210607-135426,I16,JCP2022_066346,7405561427507332376,8,51.89,3.949126,1.715084,...,-0.345951,-1.118323,-0.161556,-1.257248,-0.559063,-0.574682,-0.069938,-0.905871,-0.095335,-1.502679
311991,source_6,p211004CPU2OS48hw384exp031JUMP,110000295568,N18,JCP2022_072764,7405669563461892666,10,46.61,3.841815,1.668479,...,1.112032,-0.148448,-0.221544,-0.322298,1.159897,-0.276275,0.684799,-0.194555,0.087227,-0.086279
311992,source_3,CP_31_all_Phenix1,B040103c,N18,JCP2022_072764,7405669563461892666,10,46.61,3.841815,1.668479,...,-0.209607,-0.189005,0.021124,-0.088952,0.320085,0.476401,0.183413,0.102997,-0.055001,0.159067
311993,source_1,Batch5_20221030,UL001797,AB35,JCP2022_072764,7405669563461892666,10,46.61,3.841815,1.668479,...,1.146718,-0.549063,-1.305281,-0.393090,0.846342,2.117684,0.992264,0.160027,-0.998341,0.095647
311994,source_3,CP_36_all_Phenix1,BAY5876c,N12,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,...,0.161690,-0.173305,-0.247879,-0.238383,0.212531,-0.141207,-0.022377,-0.028891,-0.084800,-0.083831
311995,source_6,p211004CPU2OS48hw384exp031JUMP,110000296294,N12,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,...,-0.144887,-0.309169,-0.394016,-0.165068,0.035574,-0.069452,-0.392102,-0.454153,-0.337819,-0.324504
311996,source_2,20210823_Batch_10,1086292051,L21,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,...,-0.168412,-0.532963,-0.403206,-0.538526,-0.126008,-0.286082,-0.109849,-0.224999,-0.335858,-0.206278
311997,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143006,A07,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,...,-0.298448,1.689098,-2.980865,2.315933,-0.134850,2.129299,0.192902,1.009330,-1.570815,3.307488
311998,source_6,p210928CPU2OS48hw384exp030JUMP,110000296370,A15,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,...,0.662118,0.773286,-0.972214,0.670786,0.539195,0.083646,1.035482,0.213815,-0.202446,1.495020
311999,source_3,CP_32_all_Phenix1,B40503aW,A15,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,...,-0.509759,0.321496,1.837183,0.893738,-1.431595,1.137122,-0.684118,-0.185220,3.029860,0.015757


277196    source_10
277197     source_5
277198    source_11
277199     source_5
277200     source_2
277201    source_10
277202     source_8
277203     source_9
277204     source_9
277205     source_5
Name: Metadata_Source, dtype: object

In [36]:
del df_training

#### Read validation file

In [37]:
print(f" Read {VALIDATION_FILE}")

 Read ../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_val_092024.csv


In [38]:
df_val = pd.read_csv(VALIDATION_FILE, skiprows = 0)  ## ,  names = header_cols, header = None)

In [39]:
df_val.shape
df_val.info()
print('------------------------------------------------------------')
df_val.head(10)
print('------------------------------------------------------------')
df_val.tail(10)


(21600, 1482)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21600 entries, 0 to 21599
Columns: 1482 entries, source_5 to -1.4244772340330674
dtypes: float64(1474), int64(3), object(5)
memory usage: 244.2+ MB
------------------------------------------------------------


,source_5,JUMPCPE-20210706-Run06_20210706_235916,APTJUM213,I14,JCP2022_100922,5548998150937919747,3,79.96,4.381526509632199,1.90287278544608,...,0.5757217663700753,-0.6271727707836247,-1.0280613013769504,-2.295289099046197,0.1197730209767307,0.5777658099004659,0.6410443379749473,0.361388674433214,-0.855551244645328,-1.4244772340330674
0,source_2,20210607_Batch_2,1053601824,N20,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,-0.441871,0.758631,2.348621,-0.241090,-0.050882,-1.244791,-0.143035,1.014616,0.268188,0.182243
1,source_10,2021_06_15_U2OS_48_hr_run6,Dest210615-151512,N20,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,1.958334,0.681298,-0.397469,-0.866183,1.856061,0.810315,1.929989,1.048665,-0.410391,0.546589
2,source_8,J3,A1170508,G11,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,0.925959,0.195602,0.364828,-0.488673,1.078078,0.276447,0.358473,0.605105,0.045399,0.377695
3,source_9,20210915-Run10,GR00003318,W40,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,-0.319566,0.279319,-0.138141,0.480616,0.308487,0.125694,0.042493,-0.047722,-0.189558,0.052328
4,source_9,20210915-Run10,GR00003316,G16,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,-0.278550,0.298789,0.031301,0.547154,-0.291327,-0.043937,-0.054705,0.069806,-0.173805,0.233578
5,source_5,JUMPCPE-20210730-Run15_20210801_205335,ATSJUM216,G16,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,0.875016,0.516222,-0.200050,-0.777410,0.810594,-0.268878,0.192989,0.857453,-0.193188,-0.209808
6,source_1,Batch6_20221102,UL000595,I07,JCP2022_042631,5549260080151258999,7,76.07,4.331654,1.881213,...,0.179737,0.309325,-1.033849,0.639444,0.012746,-0.142948,-0.043858,0.327219,-0.647443,0.237846
7,source_3,CP_29_all_Phenix1,SP28P31a,E04,JCP2022_042631,5549260080151258999,7,76.07,4.331654,1.881213,...,0.110372,0.829789,-0.438280,0.978006,0.826556,0.041168,-0.069486,0.009252,-0.161340,0.004362
8,source_2,20210726_Batch_7,1086293515,I05,JCP2022_042631,5549260080151258999,7,76.07,4.331654,1.881213,...,-0.087732,0.333011,-0.995919,0.599676,-0.476525,-0.084013,-0.077100,0.375175,-0.253507,0.689125
9,source_8,J3,A1170507,O12,JCP2022_076833,5549671971685925262,14,58.34,4.066288,1.765966,...,-0.282802,0.058105,-0.260677,0.043434,-0.148979,-0.381363,-0.181686,-0.186671,-0.213016,-0.080367


------------------------------------------------------------


,source_5,JUMPCPE-20210706-Run06_20210706_235916,APTJUM213,I14,JCP2022_100922,5548998150937919747,3,79.96,4.381526509632199,1.90287278544608,...,0.5757217663700753,-0.6271727707836247,-1.0280613013769504,-2.295289099046197,0.1197730209767307,0.5777658099004659,0.6410443379749473,0.361388674433214,-0.855551244645328,-1.4244772340330674
21590,source_5,JUMPCPE-20210812-Run20_20210815_062625,AEOJUM204,G03,JCP2022_008895,6702970690958368233,9,109.00,4.691348,2.037426,...,-0.604553,-0.983824,0.214036,-0.822490,-0.317892,-0.763961,-0.763044,-0.825475,0.084190,-0.893306
21591,source_1,Batch3_20221010,UL000091,G43,JCP2022_027954,6703000930786455825,1,79.09,4.370586,1.898122,...,-0.513554,0.367436,1.949631,-0.973198,-1.461577,-0.594070,-1.563581,-0.055157,0.802170,-1.598731
21592,source_3,CP_26_all_Phenix1,J12440a,D22,JCP2022_027954,6703000930786455825,1,79.09,4.370586,1.898122,...,1.353366,0.389084,1.643581,0.113494,1.247632,-0.619674,0.187634,-0.365947,0.211438,-1.899679
21593,source_6,p210906CPU2OS48hw384exp025JUMP,110000294882,D22,JCP2022_027954,6703000930786455825,1,79.09,4.370586,1.898122,...,0.847335,0.275720,0.859933,0.188536,0.896707,-0.266526,-0.065982,-0.139518,0.120872,-0.040728
21594,source_2,20210830_Batch_11,1086290668,N21,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-0.278313,-0.175301,0.118574,-0.149149,-0.142567,-0.485178,-0.128521,-0.297253,-0.130395,-0.334945
21595,source_8,J3,A1170495,G22,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,0.556524,-0.547697,-0.243381,-0.920549,0.703350,-0.404463,0.005729,-0.287962,-0.141280,-0.552511
21596,source_6,p211109CPU2OS48hw384exp035JUMP,110000295493,G22,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-0.045905,-0.185008,-0.169419,-0.440330,0.089256,-0.106695,-0.075618,-0.197659,-0.119509,-0.126529
21597,source_9,20211103-Run16,GR00004419,R42,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,0.538210,-0.078134,-0.439562,0.079730,0.535180,-0.372400,-0.006128,-0.236739,-0.417071,-0.225268
21598,source_9,20211103-Run16,GR00004418,B42,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,-0.498170,-1.326272,-2.079787,-1.927362,-1.250412,-0.154330,-0.630758,-0.136336,-1.295034,-0.720645
21599,source_9,20211103-Run16,GR00004421,B18,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,-0.245970,-0.257799,-0.263613,-0.266980,-1.092559,0.638992,0.048592,0.261487,0.070192,0.553269


In [40]:
del df_val

#### Read test file

In [41]:
print(f" Read {TEST_FILE}")
df_test = pd.read_csv(TEST_FILE, skiprows = 0)  ## , names = header_cols)

 Read ../output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_test_092024.csv


In [42]:
df_test.shape
df_test.info()
df_test.head(10)
df_test.tail(10)

(12600, 1482)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Columns: 1482 entries, source_9 to 0.5532689852091409
dtypes: float64(1474), int64(3), object(5)
memory usage: 142.5+ MB


,source_9,20211103-Run16,GR00004421,B18,JCP2022_040230,6703257770840297119,15,79.29,4.373112017284307,1.899218417851373,...,-0.2459700989852401,-0.2577985228394074,-0.2636134100011204,-0.2669804996152689,-1.092558869602004,0.6389922074864952,0.048591573171269,0.2614874980530011,0.0701921035574996,0.5532689852091409
0,source_9,20210914-Run9,GR00003302,H45,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-0.896670,1.450820,1.152851,-1.278530,1.418223,0.021354,-0.141206,0.868969,0.193294,0.005227
1,source_9,20210914-Run9,GR00003301,H21,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-2.831087,0.228118,-0.683563,-1.849235,-0.919059,0.182213,-0.887957,0.667862,-0.120462,-0.251829
2,source_9,20210914-Run9,GR00003303,X45,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-1.743976,1.145396,0.208649,-1.137074,0.149952,0.528899,-0.171473,1.336097,0.096783,0.456798
3,source_3,CP_32_all_Phenix1,B40803aW,P21,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,0.343824,-0.960700,0.310768,-0.827690,0.089760,-0.492758,-0.630063,-0.921455,-0.279790,-0.952676
4,source_6,p210928CPU2OS48hw384exp030JUMP,110000296383,P21,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,0.164401,-0.457574,0.001981,0.241172,0.594398,0.351269,0.024534,-0.786780,-0.169006,0.893691
5,source_1,Batch5_20221030,UL001783,AE41,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,-1.413608,-1.442370,-0.206235,-0.468183,-1.048844,-0.416958,-0.869591,-0.903433,-0.192858,-0.487136
6,source_6,p210927CPU2OS48hw384exp029JUMP,110000296325,G11,JCP2022_015241,6704729456769442241,1,62.22,4.130676,1.793930,...,-0.843803,-0.399426,-0.808531,-0.786130,-0.878699,-0.081571,-0.629274,-0.448377,-1.327741,-0.970775
7,source_3,CP_31_all_Phenix1,B040903d,G11,JCP2022_015241,6704729456769442241,1,62.22,4.130676,1.793930,...,1.002020,0.561065,0.303877,0.646050,0.387289,0.036483,-0.128751,0.210215,-0.099087,0.258716
8,source_1,Batch5_20221030,UL001781,N22,JCP2022_015241,6704729456769442241,1,62.22,4.130676,1.793930,...,1.299634,1.102926,1.035011,0.218380,0.939852,0.365044,1.681466,0.824675,1.187973,0.234408
9,source_6,p211123CPU2OS48hw384exp036JUMP,110000297126,I14,JCP2022_071726,6704991968420949852,12,76.96,4.343286,1.886265,...,-0.669285,-0.473370,-0.063660,-0.645018,-0.976869,-0.271563,-0.181059,-0.547519,-0.100093,-0.410136


,source_9,20211103-Run16,GR00004421,B18,JCP2022_040230,6703257770840297119,15,79.29,4.373112017284307,1.899218417851373,...,-0.2459700989852401,-0.2577985228394074,-0.2636134100011204,-0.2669804996152689,-1.092558869602004,0.6389922074864952,0.048591573171269,0.2614874980530011,0.0701921035574996,0.5532689852091409
12590,source_10,2021_06_07_U2OS_48_hr_run3,Dest210607-135241,K20,JCP2022_058635,7378615000324315629,13,66.22,4.192983,1.820989,...,1.844088,-0.503675,1.201386,-0.820239,2.240627,-0.563419,0.228868,-0.337481,0.294806,-0.624532
12591,source_9,20210825_Run6,GR00003412,I35,JCP2022_106565,7378620454050468236,12,113.84,4.734794,2.056295,...,-0.415434,-0.243352,-0.718413,-0.684339,-0.160416,-0.263503,0.051314,-0.130027,-0.313828,-0.314547
12592,source_9,20210825_Run6,GR00003413,Y35,JCP2022_106565,7378620454050468236,12,113.84,4.734794,2.056295,...,-0.123415,-0.196720,-0.185392,-0.497761,0.020929,-0.071133,-0.169595,-0.063637,-0.100382,-0.229789
12593,source_11,Batch4,EC000118,G19,JCP2022_106565,7378620454050468236,12,113.84,4.734794,2.056295,...,-0.354073,-0.516344,-0.325743,-0.373612,-0.348973,-0.148328,-0.255002,-0.180020,-0.033692,-0.098056
12594,source_10,2021_08_20_U2OS_48_hr_run17,Dest210810-174829,P09,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,-0.325415,0.877647,1.370634,-0.030661,-0.310835,-0.862696,-0.465788,0.114074,-0.597466,-0.395239
12595,source_1,Batch5_20221030,UL001791,AF11,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,-1.421420,1.191942,3.111663,0.740742,-1.882465,0.040255,-1.197548,0.768332,2.902781,0.549842
12596,source_3,CP_32_all_Phenix1,B40703cW,P06,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,-1.857668,-0.644382,1.188705,-0.482810,-1.500339,-0.396200,-0.536321,-0.346985,0.259447,-0.151251
12597,source_10,2021_08_17_U2OS_48_hr_run16,Dest210809-141809,L22,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,2.056177,-0.139923,1.344993,0.994875,1.954301,1.160509,2.162598,-0.239546,-0.006793,1.017170
12598,source_1,Batch3_20221010,UL000091,AB44,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,-0.588323,0.413319,-0.718256,0.170423,-0.407126,0.370998,-0.286843,0.673749,-0.368921,0.440563
12599,source_6,p210906CPU2OS48hw384exp025JUMP,110000294885,N22,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,0.903210,0.759851,1.091060,0.477611,0.934681,-0.064500,-0.156278,0.083262,0.294936,0.796720


In [136]:
del df_test

# Split Pharamacophore file into Training / Validation / Test files

In [6]:
## del df_all, df_training,df_val, df_test, df_header, df_profiles 

In [7]:
# HEADER_FILE =  './output_11102023/3_sample_profiles/3sample_profiles_1482_header_info.csv'
# header_cols = pd.read_csv(HEADER_FILE).columns.to_list()
# df_header = pd.read_csv(HEADER_FILE).columns.to_list()
# header_cols

In [8]:
FINGERPRINT_LENGTH = 13740
FINGERPRINT_SUFFIX = '-A'
NUM_SAMPLES = 3
PP_LENGTH = str(FINGERPRINT_LENGTH) + FINGERPRINT_SUFFIX
TYPE = 'selcols0570'
ORDER='HashOrder_all'

compoundPharmacophoreSamplePickleFile = ds.compoundPharmacophoreSamplePickleFile.format(num_samples=NUM_SAMPLES,pp_length = PP_LENGTH, type = TYPE, order = ORDER )
compoundPharmacophoreSampleCSVFile    = ds.compoundPharmacophoreSampleCSVFile.format(num_samples=NUM_SAMPLES,   pp_length = PP_LENGTH, type = TYPE, order = ORDER )
compoundPharmacophoreSampleCSVZipFile = ds.compoundPharmacophoreSampleCSVZipFile.format(num_samples=NUM_SAMPLES,pp_length = PP_LENGTH, type = TYPE, order = ORDER )

print(f" compound Pharmacophore File    : {ds.compoundPharmacophoreSamplePickleFile}")
print()
print(f" compoundPharmacophoreSamplePickleFile : {compoundPharmacophoreSamplePickleFile}")
print(f" compoundPharmacophoreSampleCSVFile    : {compoundPharmacophoreSampleCSVFile}")
print(f" compoundPharmacophoreSampleCSVZipFile : {compoundPharmacophoreSampleCSVZipFile}")

 compound Pharmacophore File    : ../output_11102023/{num_samples}sample_pharmacophores_{pp_length}_{type}_{order}.pkl

 compoundPharmacophoreSamplePickleFile : ../output_11102023/3sample_pharmacophores_13740-A_selcols0570_HashOrder_all.pkl
 compoundPharmacophoreSampleCSVFile    : ../output_11102023/3sample_pharmacophores_13740-A_selcols0570_HashOrder_all.csv
 compoundPharmacophoreSampleCSVZipFile : ../output_11102023/3sample_pharmacophores_13740-A_selcols0570_HashOrder_all.gz


In [14]:
PP_ALL_FILE        = f'../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740{FINGERPRINT_SUFFIX}_{TYPE}_HashOrder_all.csv'
PP_TRAINING_FILE   = f'../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740{FINGERPRINT_SUFFIX}_{TYPE}_HashOrder_training.csv'
PP_VALIDATION_FILE = f'../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740{FINGERPRINT_SUFFIX}_{TYPE}_HashOrder_validation.csv'
PP_TEST_FILE       = f'../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740{FINGERPRINT_SUFFIX}_{TYPE}_HashOrder_test.csv'

print(PP_ALL_FILE)
print(PP_TRAINING_FILE)
print(PP_VALIDATION_FILE)
print(PP_TEST_FILE)

../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-A_selcols0570_HashOrder_all.csv
../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-A_selcols0570_HashOrder_training.csv
../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-A_selcols0570_HashOrder_validation.csv
../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-A_selcols0570_HashOrder_test.csv


In [15]:
df_pp_all = pd.read_csv(PP_ALL_FILE, skiprows = 0)  ##  names = header_cols)

In [16]:
df_pp_all.shape
df_pp_all.info()
df_pp_all.head(10)
df_pp_all.tail(10)

(346542, 536)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346542 entries, 0 to 346541
Columns: 536 entries, Metadata_JCP2022 to 10688
dtypes: int64(535), object(1)
memory usage: 1.4+ GB


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,1,2,3,6,7,8,12,...,10165,10190,10234,10236,10447,10452,10457,10462,10678,10688
0,JCP2022_006020,-9223347314827979542,10,0,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,1,0
1,JCP2022_006020,-9223347314827979542,10,0,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2,JCP2022_006020,-9223347314827979542,10,0,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,1,0
3,JCP2022_058905,-9223229703754907651,13,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,JCP2022_058905,-9223229703754907651,13,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,JCP2022_058905,-9223229703754907651,13,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,JCP2022_051160,-9223005963369866647,9,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
7,JCP2022_051160,-9223005963369866647,9,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8,JCP2022_051160,-9223005963369866647,9,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9,JCP2022_038591,-9222975330808494972,4,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,1,2,3,6,7,8,12,...,10165,10190,10234,10236,10447,10452,10457,10462,10678,10688
346532,JCP2022_110705,9222418181275619136,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
346533,JCP2022_017353,9222499262440709243,11,1,1,0,1,1,1,0,...,0,0,0,0,0,0,1,0,0,0
346534,JCP2022_017353,9222499262440709243,11,1,1,0,1,1,1,0,...,0,0,0,0,0,0,1,0,0,0
346535,JCP2022_017353,9222499262440709243,11,1,1,0,1,1,1,0,...,0,0,0,0,0,0,1,0,0,0
346536,JCP2022_040466,9222528730267146426,10,0,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
346537,JCP2022_040466,9222528730267146426,10,0,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
346538,JCP2022_040466,9222528730267146426,10,0,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
346539,JCP2022_102583,9223213570900537648,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,1,0,0
346540,JCP2022_102583,9223213570900537648,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,1,0,0
346541,JCP2022_102583,9223213570900537648,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,1,0,0


## Write to training, val, and test files

In [17]:
print(f" Write to {PP_TRAINING_FILE}")

 Write to ../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-A_selcols0570_HashOrder_training.csv


In [18]:
df_pp_all.loc[0:277_199].to_csv(PP_TRAINING_FILE, mode = 'w', index = False, header = True )

In [19]:
print(f" Write to {PP_VALIDATION_FILE}")

 Write to ../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-A_selcols0570_HashOrder_validation.csv


In [20]:
df_pp_all.loc[277_200:298_799].to_csv(PP_VALIDATION_FILE, mode = 'w', index = False, header = True )

In [21]:
print(f" Write to {PP_TEST_FILE}")

 Write to ../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-A_selcols0570_HashOrder_test.csv


In [22]:
df_pp_all.loc[298_800:311_399].to_csv(PP_TEST_FILE, mode = 'w', index = False, header = True )

In [23]:
del df_pp_all

## Read training, val, and test files 

#### Test files

In [24]:
df_pp_training =  pd.read_csv(PP_TRAINING_FILE, skiprows = 0)

In [25]:
df_pp_training.shape
df_pp_training.info()
df_pp_training.head(5)
df_pp_training.tail(10)
df_pp_training.loc[277196:277205].Metadata_JCP2022

(277200, 536)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277200 entries, 0 to 277199
Columns: 536 entries, Metadata_JCP2022 to 10688
dtypes: int64(535), object(1)
memory usage: 1.1+ GB


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,1,2,3,6,7,8,12,...,10165,10190,10234,10236,10447,10452,10457,10462,10678,10688
0,JCP2022_006020,-9223347314827979542,10,0,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,1,0
1,JCP2022_006020,-9223347314827979542,10,0,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2,JCP2022_006020,-9223347314827979542,10,0,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,1,0
3,JCP2022_058905,-9223229703754907651,13,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,JCP2022_058905,-9223229703754907651,13,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,1,2,3,6,7,8,12,...,10165,10190,10234,10236,10447,10452,10457,10462,10678,10688
277190,JCP2022_089792,5548393758141673059,3,0,1,1,0,1,0,1,...,0,0,1,1,0,0,0,0,1,0
277191,JCP2022_054246,5548833810863940256,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
277192,JCP2022_054246,5548833810863940256,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
277193,JCP2022_054246,5548833810863940256,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
277194,JCP2022_040737,5548902807653184645,5,1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
277195,JCP2022_040737,5548902807653184645,5,1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
277196,JCP2022_040737,5548902807653184645,5,1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
277197,JCP2022_100922,5548998150937919747,3,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
277198,JCP2022_100922,5548998150937919747,3,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
277199,JCP2022_100922,5548998150937919747,3,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


277196    JCP2022_040737
277197    JCP2022_100922
277198    JCP2022_100922
277199    JCP2022_100922
Name: Metadata_JCP2022, dtype: object

In [26]:
del df_pp_training

#### validation 

In [27]:
df_pp_val = pd.read_csv(PP_VALIDATION_FILE, skiprows = 0)  ## ,  names = header_cols, header = None)

In [28]:
df_pp_val.shape
df_pp_val.info()
print('------------------------------------------------------------')
df_pp_val.head(10)
print('------------------------------------------------------------')
df_pp_val.tail(10)

(21600, 536)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21600 entries, 0 to 21599
Columns: 536 entries, Metadata_JCP2022 to 10688
dtypes: int64(535), object(1)
memory usage: 88.3+ MB
------------------------------------------------------------


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,1,2,3,6,7,8,12,...,10165,10190,10234,10236,10447,10452,10457,10462,10678,10688
0,JCP2022_101506,5549160564573729245,13,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,JCP2022_101506,5549160564573729245,13,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,JCP2022_101506,5549160564573729245,13,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,JCP2022_010664,5549241656037143871,15,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,JCP2022_010664,5549241656037143871,15,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,JCP2022_010664,5549241656037143871,15,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
6,JCP2022_042631,5549260080151258999,7,0,1,1,1,1,0,1,...,0,0,0,0,1,1,0,0,0,0
7,JCP2022_042631,5549260080151258999,7,0,1,1,1,1,0,1,...,0,0,0,0,1,1,0,0,0,0
8,JCP2022_042631,5549260080151258999,7,0,1,1,1,1,0,1,...,0,0,0,0,1,1,0,0,0,0
9,JCP2022_076833,5549671971685925262,14,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


------------------------------------------------------------


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,1,2,3,6,7,8,12,...,10165,10190,10234,10236,10447,10452,10457,10462,10678,10688
21590,JCP2022_008895,6702970690958368233,9,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
21591,JCP2022_027954,6703000930786455825,1,0,0,1,1,1,1,0,...,0,0,0,0,1,1,1,1,0,0
21592,JCP2022_027954,6703000930786455825,1,0,0,1,1,1,1,0,...,0,0,0,0,1,1,1,1,0,0
21593,JCP2022_027954,6703000930786455825,1,0,0,1,1,1,1,0,...,0,0,0,0,1,1,1,1,0,0
21594,JCP2022_094244,6703113441807921775,15,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
21595,JCP2022_094244,6703113441807921775,15,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
21596,JCP2022_094244,6703113441807921775,15,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
21597,JCP2022_040230,6703257770840297119,15,0,1,0,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
21598,JCP2022_040230,6703257770840297119,15,0,1,0,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
21599,JCP2022_040230,6703257770840297119,15,0,1,0,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0


In [30]:
del df_pp_val

#### Read test file

In [31]:
df_pp_test =  pd.read_csv(PP_TEST_FILE, skiprows = 0)  ## , names = header_cols)


In [32]:
df_pp_test.shape
df_pp_test.info()
df_pp_test.head(10)
df_pp_test.tail(10)

(12600, 536)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Columns: 536 entries, Metadata_JCP2022 to 10688
dtypes: int64(535), object(1)
memory usage: 51.5+ MB


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,1,2,3,6,7,8,12,...,10165,10190,10234,10236,10447,10452,10457,10462,10678,10688
0,JCP2022_004021,6703456398488902534,6,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,JCP2022_004021,6703456398488902534,6,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,JCP2022_004021,6703456398488902534,6,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,JCP2022_102146,6704177726803547665,1,0,0,0,1,1,1,0,...,0,0,1,1,0,0,0,1,1,1
4,JCP2022_102146,6704177726803547665,1,0,0,0,1,1,1,0,...,0,0,1,1,0,0,0,1,1,1
5,JCP2022_102146,6704177726803547665,1,0,0,0,1,1,1,0,...,0,0,1,1,0,0,0,1,1,1
6,JCP2022_015241,6704729456769442241,1,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
7,JCP2022_015241,6704729456769442241,1,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8,JCP2022_015241,6704729456769442241,1,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9,JCP2022_071726,6704991968420949852,12,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,1,2,3,6,7,8,12,...,10165,10190,10234,10236,10447,10452,10457,10462,10678,10688
12590,JCP2022_058635,7378615000324315629,13,0,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
12591,JCP2022_106565,7378620454050468236,12,0,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
12592,JCP2022_106565,7378620454050468236,12,0,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
12593,JCP2022_106565,7378620454050468236,12,0,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
12594,JCP2022_064379,7378624279819671723,11,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
12595,JCP2022_064379,7378624279819671723,11,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
12596,JCP2022_064379,7378624279819671723,11,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
12597,JCP2022_012996,7378937281046818562,2,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
12598,JCP2022_012996,7378937281046818562,2,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
12599,JCP2022_012996,7378937281046818562,2,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
del df_pp_test

# Split EMBEDDING files into Training / Validation / Test files

## setup

In [52]:
## del df_all, df_training,df_val, df_test, df_header, df_profiles 
# HEADER_FILE =  './output_11102023/3_sample_profiles/3sample_profiles_1482_header_info.csv'
# header_cols = pd.read_csv(HEADER_FILE).columns.to_list()


In [44]:
INPUT_PATH = f"/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/"

LATENT_DIM = 150
# LATENT_DIM = 250
METADATA_COLS = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA', 'Metadata_Permiation']
# METADATA_COLS += [f'Feature_{x:03d}' for x in range(LATENT_DIM)]
INPUT_COLS = LATENT_DIM + len(METADATA_COLS)
print(len(METADATA_COLS))


## AUTOENCODER RUNMODE 
AE_RUNMODE = 'base'
AE_DATETIME = "20240923_1943"     # SNNL, CPB = 600, Latent 250, WD = 0.001, SNN Factor 3
# AE_DATETIME = "20240917_2017"     # SNNL, CPB = 600, Latent 250, WD = 0.001, SNN Factor 3
## 
# AE_RUNMODE = "snnl"
# AE_DATETIME = "20240906_2201"     # SNNL, CPB = 600, Latent 150, WD = 0.001, SNN Factor 3
# AE_DATETIME = "20240917_2004"     # SNNL, CPB = 600, Latent 250, WD = 0.001, SNN Factor 3
# AE_DATETIME = "20240924_0146"     # SNNL, CPB = 600, Latent 250, WD = 0.001, SNN Factor 3
##
# AE_CKPTTYPE = "BEST"
AE_CKPTTYPE = "LAST"
# df_header = pd.read_csv(HEADER_FILE).columns.to_list()
# header_cols
print(INPUT_COLS)


11
161


In [71]:
# INPUT_FILE      = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_BEST_train.csv'
INPUT_FILE      = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_ep100_312000.csv'
TRAINING_FILE   = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_ep100_training.csv'
VALIDATION_FILE = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_ep100_validation.csv'
TEST_FILE       = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_ep100_test.csv'

print("INPUT_FILE:      ",INPUT_FILE)
print("TRAINING_FILE:   ",TRAINING_FILE)
print("VALIDATION_FILE: ",VALIDATION_FILE)
print("TEST_FILE:       ",TEST_FILE)

INPUT_FILE:       ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_ep100_312000.csv
TRAINING_FILE:    ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_ep100_training.csv
VALIDATION_FILE:  ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_ep100_validation.csv
TEST_FILE:        ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_ep100_test.csv


In [72]:
df_all =  pd.read_csv(INPUT_FILE, skiprows = 0) ##  names = header_cols)

In [73]:
df_all.shape
df_all.info()
df_all.head(10)
df_all.tail(10)

(312000, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312000 entries, 0 to 311999
Columns: 161 entries, Metadata_Source to Feature_149
dtypes: float64(154), int64(2), object(5)
memory usage: 383.2+ MB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
0,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,-2.552628,-1.763222,-1.624288,1.537165,3.291877,-8.132829,0.324960,0.177446,-7.537000,-1.099817
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,-0.983650,1.990692,2.430754,4.027295,5.188753,-22.717871,-1.344695,1.636317,-20.857174,0.358101
2,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,2.505916,1.747913,-3.292658,-1.645325,2.075963,-10.405005,-0.384916,-0.171505,-8.145843,-3.062205
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.409347,-0.661720,0.277162,1.322708,-1.090364,-3.622480,-0.231125,0.220053,-3.809806,-0.103089
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.628980,-1.565576,-2.727243,-1.396314,-0.392241,-6.166427,-0.162782,0.391458,-8.103096,2.300465
5,source_10,2021_08_23_U2OS_48_hr_run18,Dest210823-174733,I13,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.011784,1.233157,0.230815,-0.840441,-1.659472,-9.966318,0.332215,0.310508,-0.385488,-1.604083
6,source_1,Batch6_20221102,UL000587,O34,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-0.168297,0.531008,0.910452,0.209394,1.485443,-8.791544,-0.448119,0.092065,-7.883686,-0.300299
7,source_6,p211109CPU2OS48hw384exp035JUMP,110000296167,H17,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-1.199364,0.475766,1.765877,1.148466,1.882804,-7.334637,-0.663661,-0.219054,-10.892461,0.223176
8,source_3,CP_26_all_Phenix1,J12455b,H17,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-2.249634,-0.771401,-0.431551,1.534302,2.095980,-4.494537,-0.710201,-0.526205,-4.837973,0.818884
9,source_2,20210712_Batch_5,1053597875,K10,JCP2022_038591,-9222975330808494972,4,51.22,3.936130,1.709440,...,-1.477407,-1.406402,-1.973514,3.221596,2.807266,-9.649069,-0.401196,0.042179,-10.125250,0.137456


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
311990,source_10,2021_06_07_U2OS_48_hr_run3,Dest210607-135426,I16,JCP2022_066346,7405561427507332376,8,51.89,3.949126,1.715084,...,0.969487,1.315196,-1.123860,-0.780301,-1.417080,-10.608778,0.366750,0.730038,-3.696644,-0.907943
311991,source_6,p211004CPU2OS48hw384exp031JUMP,110000295568,N18,JCP2022_072764,7405669563461892666,10,46.61,3.841815,1.668479,...,-0.631172,0.393714,-2.376329,1.477560,-1.164792,-9.294556,-0.152118,0.000458,-5.950371,0.783036
311992,source_3,CP_31_all_Phenix1,B040103c,N18,JCP2022_072764,7405669563461892666,10,46.61,3.841815,1.668479,...,-0.249563,0.921156,-2.819873,-0.745044,-0.080471,-8.393060,0.135103,-0.188110,-6.538145,-3.068340
311993,source_1,Batch5_20221030,UL001797,AB35,JCP2022_072764,7405669563461892666,10,46.61,3.841815,1.668479,...,-1.022588,0.133211,0.059847,1.002802,3.863717,-6.505188,-0.232071,0.052878,-3.164107,0.916859
311994,source_3,CP_36_all_Phenix1,BAY5876c,N12,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,...,0.487670,0.470133,-2.036996,0.380579,-1.119922,-5.016595,-0.019964,0.071612,-6.743359,-0.841140
311995,source_6,p211004CPU2OS48hw384exp031JUMP,110000296294,N12,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,...,0.667473,0.127919,-0.598324,0.747411,-0.685237,-3.072840,-0.229081,-0.097470,-1.037957,-0.208890
311996,source_2,20210823_Batch_10,1086292051,L21,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,...,-1.171456,0.022704,-0.791719,0.899509,0.209843,-4.686642,-0.187199,-0.065107,-2.625167,-1.318921
311997,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143006,A07,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,...,-1.689094,-4.232093,4.596641,3.633204,3.525940,-8.386950,-0.562791,0.945299,-17.425137,1.206067
311998,source_6,p210928CPU2OS48hw384exp030JUMP,110000296370,A15,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,...,-1.905051,0.806726,-0.906716,0.979464,2.238329,-7.226177,-0.254954,0.101953,-7.851573,-0.992448
311999,source_3,CP_32_all_Phenix1,B40503aW,A15,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,...,5.764628,1.530352,-3.235603,0.203709,-5.806979,-17.204670,-0.022028,-0.587385,-9.218459,-2.491747


## Write to training, val, and test files

In [74]:
print(f" Write to {TRAINING_FILE}")

 Write to ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_ep100_training.csv


In [75]:
df_all.loc[0:277_199].to_csv(TRAINING_FILE, mode = 'w', index = False, header = True )

In [76]:
print(f" Write to {VALIDATION_FILE}")

 Write to ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_ep100_validation.csv


In [77]:
df_all.loc[277_200:298_799].to_csv(VALIDATION_FILE, mode = 'w', index = False, header = True )

In [78]:
print(f" Write to {TEST_FILE}")

 Write to ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_ep100_test.csv


In [79]:
df_all.loc[298_800:311_399].to_csv(TEST_FILE, mode = 'w', index = False, header = True )

## Read files

#### Training file

In [80]:
df_training =  pd.read_csv(TRAINING_FILE, skiprows = 0)

In [81]:
df_training.shape
df_training.info()
df_training.head(5)
df_training.tail(10)
df_training.loc[277196:277205].Metadata_Source

(277200, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277200 entries, 0 to 277199
Columns: 161 entries, Metadata_Source to Feature_149
dtypes: float64(154), int64(2), object(5)
memory usage: 340.5+ MB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
0,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,-2.552628,-1.763222,-1.624288,1.537165,3.291877,-8.132829,0.324960,0.177446,-7.537000,-1.099817
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,-0.983650,1.990692,2.430754,4.027295,5.188753,-22.717871,-1.344695,1.636317,-20.857174,0.358101
2,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,2.505916,1.747913,-3.292658,-1.645325,2.075963,-10.405005,-0.384916,-0.171505,-8.145843,-3.062205
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.409347,-0.661720,0.277162,1.322708,-1.090364,-3.622480,-0.231125,0.220053,-3.809806,-0.103089
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.628980,-1.565576,-2.727243,-1.396314,-0.392241,-6.166427,-0.162782,0.391458,-8.103096,2.300465


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
277190,source_9,20211102-Run15,GR00004398,J35,JCP2022_089792,5548393758141673059,3,100.28,4.607966,2.001214,...,-3.014333,-1.215319,-1.840532,0.283918,3.120132,-9.164672,0.180519,0.277175,-7.725730,-1.379068
277191,source_3,CP_29_all_Phenix1,SP28P31d,L08,JCP2022_054246,5548833810863940256,0,83.10,4.420045,1.919601,...,0.361262,0.580920,0.626291,0.705489,1.267296,-5.307832,-0.178564,-0.071705,-4.202373,0.558450
277192,source_1,Batch6_20221102,UL000595,X16,JCP2022_054246,5548833810863940256,0,83.10,4.420045,1.919601,...,0.793171,1.343941,-0.260161,-0.673947,0.093889,-4.868979,0.042260,0.226557,-0.371595,-0.436463
277193,source_10,2021_08_23_U2OS_48_hr_run18,Dest210823-175355,G13,JCP2022_054246,5548833810863940256,0,83.10,4.420045,1.919601,...,-2.245047,-0.522922,0.166780,0.105426,-0.019724,-11.559787,-0.039445,0.257081,-4.577524,-0.055358
277194,source_6,p211109CPU2OS48hw384exp035JUMP,110000295497,G21,JCP2022_040737,5548902807653184645,5,55.55,4.017284,1.744684,...,-0.756909,0.401381,-0.444990,0.391934,1.026336,-2.510413,-0.015877,0.287872,-3.469492,-0.526720
277195,source_8,J3,A1170503,G21,JCP2022_040737,5548902807653184645,5,55.55,4.017284,1.744684,...,2.465074,0.965515,-0.145447,1.393077,0.499561,-5.349358,-0.225319,-0.048897,-4.613613,-0.055692
277196,source_10,2021_08_17_U2OS_48_hr_run16,Dest210809-140929,N19,JCP2022_040737,5548902807653184645,5,55.55,4.017284,1.744684,...,4.911559,0.521616,-0.866976,-0.175840,-0.017351,-9.922095,-0.426921,-0.570364,-7.931996,1.266966
277197,source_5,JUMPCPE-20211001-Run34_20211003_121618,APTJUM513,I14,JCP2022_100922,5548998150937919747,3,79.96,4.381527,1.902873,...,-1.297288,-2.617470,-0.843620,1.263859,0.633624,-8.586201,0.058447,0.104808,-8.513761,2.836743
277198,source_11,Batch1,EC000029,J07,JCP2022_100922,5548998150937919747,3,79.96,4.381527,1.902873,...,-2.043449,1.372288,0.959388,3.760435,0.065238,-11.110103,0.075404,0.374823,-8.813631,-1.209635
277199,source_5,JUMPCPE-20210706-Run06_20210706_235916,APTJUM213,I14,JCP2022_100922,5548998150937919747,3,79.96,4.381527,1.902873,...,-0.702555,2.175448,-2.956066,-2.020852,7.273649,-12.012573,-0.305002,0.870421,-13.440047,-1.271739


277196    source_10
277197     source_5
277198    source_11
277199     source_5
Name: Metadata_Source, dtype: object

In [82]:
del df_training

#### Read validation file

In [83]:
df_val =  pd.read_csv(VALIDATION_FILE, skiprows = 0)  ## ,  names = header_cols, header = None)

In [84]:
df_val.shape
df_val.info()
df_val.head(10)
df_val.tail(10)

(21600, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21600 entries, 0 to 21599
Columns: 161 entries, Metadata_Source to Feature_149
dtypes: float64(154), int64(2), object(5)
memory usage: 26.5+ MB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
0,source_2,20210607_Batch_2,1053601824,N20,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,-3.271144,-9.151258,-8.550510,4.027408,1.058650,-23.396250,1.205750,2.423736,-44.710579,4.860898
1,source_10,2021_06_15_U2OS_48_hr_run6,Dest210615-151512,N20,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,6.776818,-6.288204,-12.732674,0.032091,-3.526956,-21.306322,-0.339397,2.836556,-41.835403,7.866678
2,source_8,J3,A1170508,G11,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,-3.117678,-9.825349,-10.647116,-0.201180,1.348574,-15.938127,-0.216501,0.457205,-37.806820,3.792253
3,source_9,20210915-Run10,GR00003318,W40,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,-0.850171,-0.501989,-0.652266,0.931933,-0.028567,-3.011659,-0.332797,0.102578,-3.597339,-0.040119
4,source_9,20210915-Run10,GR00003316,G16,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,-1.157325,-0.467260,1.030365,0.534840,1.965488,-5.460619,-0.334805,0.097699,-2.961359,-0.206294
5,source_5,JUMPCPE-20210730-Run15_20210801_205335,ATSJUM216,G16,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,-1.506061,1.273258,-1.779230,0.733574,1.484896,-9.219390,-0.667666,0.325607,-5.921163,-0.704697
6,source_1,Batch6_20221102,UL000595,I07,JCP2022_042631,5549260080151258999,7,76.07,4.331654,1.881213,...,-1.682392,0.626171,1.469981,0.585741,2.208004,-6.815588,-0.453294,0.014098,-2.793020,-0.489978
7,source_3,CP_29_all_Phenix1,SP28P31a,E04,JCP2022_042631,5549260080151258999,7,76.07,4.331654,1.881213,...,-0.100374,-0.113979,-0.624808,-0.220691,0.763050,-5.282200,-0.037718,0.429025,-2.490625,-0.428564
8,source_2,20210726_Batch_7,1086293515,I05,JCP2022_042631,5549260080151258999,7,76.07,4.331654,1.881213,...,0.401566,-0.271495,0.115744,0.523646,-0.376371,-4.337296,-0.300159,0.024116,-2.043522,-0.219609
9,source_8,J3,A1170507,O12,JCP2022_076833,5549671971685925262,14,58.34,4.066288,1.765966,...,-0.423850,0.081613,0.643789,0.033416,-0.912188,-2.396454,-0.150339,-0.063765,-1.861728,-0.110365


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
21590,source_5,JUMPCPE-20210812-Run20_20210815_062625,AEOJUM204,G03,JCP2022_008895,6702970690958368233,9,109.00,4.691348,2.037426,...,-1.961073,-0.619872,-1.010140,-0.520003,2.570905,-7.224851,-0.195091,0.264809,-5.488086,-1.345322
21591,source_1,Batch3_20221010,UL000091,G43,JCP2022_027954,6703000930786455825,1,79.09,4.370586,1.898122,...,6.332401,3.736740,2.195881,-0.517344,-5.509088,-17.668198,-0.454334,-0.270607,-10.375038,0.244154
21592,source_3,CP_26_all_Phenix1,J12440a,D22,JCP2022_027954,6703000930786455825,1,79.09,4.370586,1.898122,...,6.090743,3.378679,0.135698,-0.446835,-5.459666,-12.088471,0.321631,0.319501,-5.387634,0.731342
21593,source_6,p210906CPU2OS48hw384exp025JUMP,110000294882,D22,JCP2022_027954,6703000930786455825,1,79.09,4.370586,1.898122,...,1.416903,2.126688,0.809993,-1.501835,-0.272585,-7.874771,0.311936,0.109160,-2.852084,0.503643
21594,source_2,20210830_Batch_11,1086290668,N21,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,0.081671,-0.196157,-0.736156,0.869219,-0.521360,-3.032590,-0.141598,-0.231654,-2.354809,-0.132796
21595,source_8,J3,A1170495,G22,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-0.629628,0.276551,-0.859049,0.414772,1.079770,-4.289495,-0.306479,0.224219,-4.057531,0.242262
21596,source_6,p211109CPU2OS48hw384exp035JUMP,110000295493,G22,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-0.561523,0.479018,-1.304662,0.430854,0.157790,-4.350481,-0.134111,0.252412,-4.306442,-0.707823
21597,source_9,20211103-Run16,GR00004419,R42,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,-0.535846,-0.490052,-0.417134,0.317036,-0.421300,-6.055983,-0.150349,-0.200863,-1.438397,0.417129
21598,source_9,20211103-Run16,GR00004418,B42,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,1.951243,-2.095703,0.740252,4.117397,0.318896,-8.398430,-0.936792,-0.608298,-18.761026,-1.655976
21599,source_9,20211103-Run16,GR00004421,B18,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,-0.093694,-0.953900,-0.718689,-0.640695,-1.424198,-2.799984,-0.197789,-0.003732,-6.069263,-0.885951


In [85]:
del df_val

#### Read test file

In [86]:
df_test =  pd.read_csv(TEST_FILE, skiprows = 0)  ## , names = header_cols)

In [87]:
df_test.shape
df_test.info()
df_test.head(10)
df_test.tail(10)

(12600, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Columns: 161 entries, Metadata_Source to Feature_149
dtypes: float64(154), int64(2), object(5)
memory usage: 15.5+ MB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
0,source_9,20210914-Run9,GR00003302,H45,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-8.584577,-2.549284,-14.389825,8.053354,3.698846,-29.142130,-1.390845,-0.019289,-59.721710,-2.561406
1,source_9,20210914-Run9,GR00003301,H21,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-7.877361,-5.206586,-9.055118,3.658077,7.019308,-25.132833,-0.839879,-0.231124,-58.813179,-0.599193
2,source_9,20210914-Run9,GR00003303,X45,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-6.282897,-4.337790,-7.773513,5.959421,2.878678,-29.837185,-1.042768,-0.356013,-56.541813,-1.827621
3,source_3,CP_32_all_Phenix1,B40803aW,P21,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,2.692098,1.221261,-0.059358,-2.164765,-3.813182,-10.335291,0.014976,0.334896,-1.662589,0.603296
4,source_6,p210928CPU2OS48hw384exp030JUMP,110000296383,P21,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,0.876389,0.207340,-0.905392,-1.302434,-0.196218,-5.037147,0.489360,0.147495,-2.039498,-0.751584
5,source_1,Batch5_20221030,UL001783,AE41,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,-2.109472,-0.676401,-0.443937,0.832075,1.784650,-4.642450,0.031325,-0.322575,-3.436913,-0.587018
6,source_6,p210927CPU2OS48hw384exp029JUMP,110000296325,G11,JCP2022_015241,6704729456769442241,1,62.22,4.130676,1.793930,...,1.640406,0.480600,1.068831,0.159028,-0.051895,-5.898269,-0.222642,0.093199,-4.724836,-0.752703
7,source_3,CP_31_all_Phenix1,B040903d,G11,JCP2022_015241,6704729456769442241,1,62.22,4.130676,1.793930,...,1.765544,-0.127611,-1.549160,0.890610,-2.859351,-7.446344,-0.461567,-0.190149,-2.713564,1.425465
8,source_1,Batch5_20221030,UL001781,N22,JCP2022_015241,6704729456769442241,1,62.22,4.130676,1.793930,...,2.708364,1.861815,0.112553,-0.672287,0.101188,-11.333173,-0.103554,0.395255,-4.158037,-0.119725
9,source_6,p211123CPU2OS48hw384exp036JUMP,110000297126,I14,JCP2022_071726,6704991968420949852,12,76.96,4.343286,1.886265,...,-0.380334,-0.467424,-0.189049,0.844495,1.554514,-4.326290,0.050614,-0.240059,-5.271820,-1.509296


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
12590,source_10,2021_06_07_U2OS_48_hr_run3,Dest210607-135241,K20,JCP2022_058635,7378615000324315629,13,66.22,4.192983,1.820989,...,2.800491,0.188683,-1.175913,0.148319,-2.167508,-8.733515,0.287856,0.061322,-5.249257,1.906846
12591,source_9,20210825_Run6,GR00003412,I35,JCP2022_106565,7378620454050468236,12,113.84,4.734794,2.056295,...,-2.641854,-1.071615,-1.732349,1.277083,2.801852,-7.104240,-0.074693,0.277539,-7.140350,-1.397056
12592,source_9,20210825_Run6,GR00003413,Y35,JCP2022_106565,7378620454050468236,12,113.84,4.734794,2.056295,...,-0.185455,1.298899,0.261662,-0.498887,0.014747,-4.242375,-0.162296,-0.025041,-1.746776,-0.756124
12593,source_11,Batch4,EC000118,G19,JCP2022_106565,7378620454050468236,12,113.84,4.734794,2.056295,...,-1.476977,-0.839138,-1.125794,0.350573,-0.058340,-2.234104,-0.221824,-0.219424,-2.427887,-0.156677
12594,source_10,2021_08_20_U2OS_48_hr_run17,Dest210810-174829,P09,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,1.436478,0.201428,-0.075862,-0.790810,-2.171621,-8.267819,-0.486865,0.032218,-4.680921,-0.827356
12595,source_1,Batch5_20221030,UL001791,AF11,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,2.639573,2.514009,1.215656,-2.306093,-6.864563,-17.781103,-0.482374,0.282003,-9.769746,-0.648618
12596,source_3,CP_32_all_Phenix1,B40703cW,P06,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,0.324546,-0.430807,-2.542788,-2.075148,0.557032,-8.146994,0.144866,0.213334,-9.071682,-2.234688
12597,source_10,2021_08_17_U2OS_48_hr_run16,Dest210809-141809,L22,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,3.998431,-0.770758,-3.839563,1.545455,2.312152,-11.327177,-0.291132,0.200411,-5.000613,1.645215
12598,source_1,Batch3_20221010,UL000091,AB44,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,-3.353591,0.134484,1.470768,0.988587,1.287441,-8.929701,-0.048152,-0.021745,-4.687459,-1.884815
12599,source_6,p210906CPU2OS48hw384exp025JUMP,110000294885,N22,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,-1.165434,-0.014599,-2.914070,2.054106,-1.979200,-12.200453,-0.086589,-0.160835,-7.989376,1.157567


In [88]:
del df_test

# Verify Pharmacophore All/Train/Val/Test Files matches in Compound order to corresponding embedded files

## Read pharmcophore files

In [20]:
PP_ALL_FILE        = '../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-B_selcols0570_HashOrder_all.csv'
PP_TRAINING_FILE   = '../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-B_selcols0570_HashOrder_training.csv'
PP_VALIDATION_FILE = '../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-B_selcols0570_HashOrder_validation.csv'
PP_TEST_FILE       = '../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-B_selcols0570_HashOrder_test.csv'

print(PP_ALL_FILE)
print(PP_TRAINING_FILE)
print(PP_VALIDATION_FILE)
print(PP_TEST_FILE)

../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-B_selcols0570_HashOrder_all.csv
../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-B_selcols0570_HashOrder_training.csv
../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-B_selcols0570_HashOrder_validation.csv
../output_11102023/3_sample_pharmacophores/3sample_pharmacophores_13740-B_selcols0570_HashOrder_test.csv


In [21]:
df_pp_all =  pd.read_csv(PP_ALL_FILE, skiprows = 0) ##  names = header_cols)

In [22]:
df_pp_all.shape
df_pp_all.info()
df_pp_all.head(5)
df_pp_all.tail(5)

(346542, 806)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346542 entries, 0 to 346541
Columns: 806 entries, Metadata_JCP2022 to 10375
dtypes: int64(805), object(1)
memory usage: 2.1+ GB


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,0,2,3,4,8,9,10,...,9921,9926,9927,9932,9933,9950,9956,10127,10370,10375
0,JCP2022_006020,-9223347314827979542,10,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,JCP2022_006020,-9223347314827979542,10,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,JCP2022_006020,-9223347314827979542,10,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,JCP2022_058905,-9223229703754907651,13,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,JCP2022_058905,-9223229703754907651,13,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,0,2,3,4,8,9,10,...,9921,9926,9927,9932,9933,9950,9956,10127,10370,10375
346537,JCP2022_040466,9222528730267146426,10,1,1,1,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
346538,JCP2022_040466,9222528730267146426,10,1,1,1,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
346539,JCP2022_102583,9223213570900537648,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
346540,JCP2022_102583,9223213570900537648,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
346541,JCP2022_102583,9223213570900537648,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [97]:
# df_pp_training =  pd.read_csv(PP_TRAINING_FILE, skiprows = 0)

# df_pp_val = pd.read_csv(PP_VALIDATION_FILE, skiprows = 0)  ## ,  names = header_cols, header = None)

# df_pp_test =  pd.read_csv(PP_TEST_FILE, skiprows = 0)  ## , names = header_cols)


## Read Profiles files

In [23]:
INPUT_PATH = f"/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/"

LATENT_DIM = 150
# LATENT_DIM = 250
METADATA_COLS = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA', 'Metadata_Permiation']
INPUT_COLS = LATENT_DIM + len(METADATA_COLS)
print(len(METADATA_COLS))

## AUTOENCODER RUNMODE
AE_RUNMODE = 'base'
AE_DATETIME = "20240923_1943"     # SNNL, CPB = 600, Latent 250, WD = 0.001, SNN Factor 3
# AE_DATETIME = "20240917_2017"     # SNNL, CPB = 600, Latent 250, WD = 0.001, SNN Factor 3
## 
# AE_RUNMODE = "snnl"
# AE_DATETIME = "20240906_2201"     # SNNL, CPB = 600, Latent 150, WD = 0.001, SNN Factor 3
# AE_DATETIME = "20240917_2004"     # SNNL, CPB = 600, Latent 250, WD = 0.001, SNN Factor 3
# AE_DATETIME = "20240924_0146"     # SNNL, CPB = 600, Latent 250, WD = 0.001, SNN Factor 3
##
AE_CKPTTYPE = "BEST"
# AE_CKPTTYPE = "LAST"
# AE_CKPTTYPE = "ep100"
# df_header = pd.read_csv(HEADER_FILE).columns.to_list()
# header_cols
print(INPUT_COLS)


# INPUT_FILE      = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_BEST_train.csv'
PROFILES_INPUT_FILE     = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_{AE_CKPTTYPE}_312000.csv'
PROFILES_TRAINING_FILE   = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_{AE_CKPTTYPE}_training.csv'
PROFILES_VALIDATION_FILE = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_{AE_CKPTTYPE}_validation.csv'
PROFILES_TEST_FILE       = f'../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_{INPUT_COLS}_HashOrder_{AE_RUNMODE}_{AE_DATETIME}_{AE_CKPTTYPE}_test.csv'

print("INPUT_FILE:      ",PROFILES_INPUT_FILE)
print("TRAINING_FILE:   ",PROFILES_TRAINING_FILE)
print("VALIDATION_FILE: ",PROFILES_VALIDATION_FILE)
print("TEST_FILE:       ",PROFILES_TEST_FILE)

11
161
INPUT_FILE:       ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_BEST_312000.csv
TRAINING_FILE:    ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_BEST_training.csv
VALIDATION_FILE:  ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_BEST_validation.csv
TEST_FILE:        ../output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_BEST_test.csv


#### All file

In [24]:
df_all =  pd.read_csv(PROFILES_INPUT_FILE, skiprows = 0) ##  names = header_cols)
df_all.shape
df_all.info()
df_all.head(10)
df_all.tail(10)

(312000, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312000 entries, 0 to 311999
Columns: 161 entries, Metadata_Source to Feature_149
dtypes: float64(154), int64(2), object(5)
memory usage: 383.2+ MB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
0,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,-0.766995,0.014673,-0.800139,0.578975,3.133392,-1.437044,-0.005154,6.372777e-07,-0.620331,0.002278
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,0.611691,0.014401,-1.709864,0.125061,4.774752,5.228751,-0.009253,1.953434e-06,-1.609306,0.001220
2,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,0.547850,0.022825,-2.193591,1.296923,2.558569,0.306822,-0.004226,9.856348e-07,-0.878720,0.003899
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.081542,0.008125,-0.380961,0.126596,-0.937269,1.704932,0.002017,6.639099e-07,0.258712,-0.001949
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-1.222203,0.012126,-0.561924,-1.161146,0.029847,0.268880,-0.010465,6.803231e-07,-2.683861,-0.003365
5,source_10,2021_08_23_U2OS_48_hr_run18,Dest210823-174733,I13,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,1.183027,0.020648,0.358211,0.003128,-0.686916,-4.484452,-0.002296,7.002995e-07,6.986052,-0.001466
6,source_1,Batch6_20221102,UL000587,O34,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-0.568593,0.017441,0.559327,0.134560,2.440967,-0.486743,-0.005490,1.109426e-06,0.560920,-0.003220
7,source_6,p211109CPU2OS48hw384exp035JUMP,110000296167,H17,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-1.668347,0.011494,0.744067,-0.313809,2.240428,0.315052,-0.016546,7.096034e-07,-5.568783,0.002506
8,source_3,CP_26_all_Phenix1,J12455b,H17,JCP2022_051160,-9223005963369866647,9,68.54,4.227418,1.835944,...,-2.417409,0.006985,-0.446477,0.292908,1.907219,0.762607,-0.001733,5.024277e-07,-0.682162,0.002751
9,source_2,20210712_Batch_5,1053597875,K10,JCP2022_038591,-9222975330808494972,4,51.22,3.936130,1.709440,...,-0.452951,0.013479,-0.537652,1.064820,2.279218,0.928250,-0.004790,7.720323e-07,-4.281958,0.004652


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
311990,source_10,2021_06_07_U2OS_48_hr_run3,Dest210607-135426,I16,JCP2022_066346,7405561427507332376,8,51.89,3.949126,1.715084,...,2.201070,0.014548,-0.169960,-0.088875,-1.814717,-3.430669,-0.006432,6.545208e-07,3.242729,-0.003399
311991,source_6,p211004CPU2OS48hw384exp031JUMP,110000295568,N18,JCP2022_072764,7405669563461892666,10,46.61,3.841815,1.668479,...,0.860485,0.013882,-0.837915,0.167994,-1.254373,-2.393184,-0.010793,6.282345e-07,-1.035366,0.000678
311992,source_3,CP_31_all_Phenix1,B040103c,N18,JCP2022_072764,7405669563461892666,10,46.61,3.841815,1.668479,...,-0.701097,0.019679,-0.959447,0.508901,0.611261,0.320144,-0.002099,7.854250e-07,-1.737941,0.004572
311993,source_1,Batch5_20221030,UL001797,AB35,JCP2022_072764,7405669563461892666,10,46.61,3.841815,1.668479,...,0.040267,0.015149,0.062102,0.123532,5.034752,0.127079,-0.010946,5.945178e-07,3.712693,-0.003120
311994,source_3,CP_36_all_Phenix1,BAY5876c,N12,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,...,0.188203,0.016523,-0.668554,0.052714,-0.926004,1.040517,-0.011061,5.595895e-07,-3.091601,0.001174
311995,source_6,p211004CPU2OS48hw384exp031JUMP,110000296294,N12,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,...,0.567462,0.009049,-0.379262,0.295149,-0.159171,0.090503,0.000025,4.147512e-07,1.996181,-0.001776
311996,source_2,20210823_Batch_10,1086292051,L21,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,...,-0.996463,0.007953,-0.244910,0.085320,0.812196,-1.092298,-0.002329,4.276991e-07,1.721518,0.000278
311997,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143006,A07,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,...,-2.260948,0.027182,0.110926,-0.708681,3.685269,13.634201,-0.006055,1.858297e-06,-5.645097,-0.000870
311998,source_6,p210928CPU2OS48hw384exp030JUMP,110000296370,A15,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,...,-1.010672,0.020899,-0.948283,-0.348785,2.389482,2.562848,-0.014979,7.622336e-07,-0.882611,0.003532
311999,source_3,CP_32_all_Phenix1,B40503aW,A15,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,...,4.761042,0.024457,-1.428297,1.061771,-6.218274,-0.627262,-0.005533,1.476338e-06,1.091232,0.001255


In [25]:
df_length = len(df_all)
df_pp_len = len(df_pp_all)
df_length, df_pp_len
df_all.iloc[0,4], df_pp_all.iloc[0,0]
df_all.iloc[0,5], df_pp_all.iloc[0,1]
df_all.iloc[0,6], df_pp_all.iloc[0,2]

(312000, 346542)

('JCP2022_006020', 'JCP2022_006020')

(-9223347314827979542, -9223347314827979542)

(10, 10)

In [26]:
ttl_match, ttl_non_match = 0,0
for i in range(df_length):
    if (df_all.iloc[i,4] == df_pp_all.iloc[i, 0]) and \
       (df_all.iloc[i,5] == df_pp_all.iloc[i, 1]) and \
       (df_all.iloc[i,6] == df_pp_all.iloc[i, 2]):
        ttl_match += 1
    else:
        ttl_non_match += 1

print(f" Total matched rows: {ttl_match}")
print(f" Total unmatched rows: {ttl_non_match}")

 Total matched rows: 312000
 Total unmatched rows: 0


In [27]:
del df_all, df_pp_all

#### Training file

In [28]:
df_pp_training =  pd.read_csv(PP_TRAINING_FILE, skiprows = 0)
df_pp_training.shape
df_pp_training.info()
df_pp_training.head(5)
df_pp_training.tail(10)

(277200, 806)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277200 entries, 0 to 277199
Columns: 806 entries, Metadata_JCP2022 to 10375
dtypes: int64(805), object(1)
memory usage: 1.7+ GB


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,0,2,3,4,8,9,10,...,9921,9926,9927,9932,9933,9950,9956,10127,10370,10375
0,JCP2022_006020,-9223347314827979542,10,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,JCP2022_006020,-9223347314827979542,10,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,JCP2022_006020,-9223347314827979542,10,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,JCP2022_058905,-9223229703754907651,13,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,JCP2022_058905,-9223229703754907651,13,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,0,2,3,4,8,9,10,...,9921,9926,9927,9932,9933,9950,9956,10127,10370,10375
277190,JCP2022_089792,5548393758141673059,3,1,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
277191,JCP2022_054246,5548833810863940256,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
277192,JCP2022_054246,5548833810863940256,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
277193,JCP2022_054246,5548833810863940256,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
277194,JCP2022_040737,5548902807653184645,5,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
277195,JCP2022_040737,5548902807653184645,5,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
277196,JCP2022_040737,5548902807653184645,5,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
277197,JCP2022_100922,5548998150937919747,3,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
277198,JCP2022_100922,5548998150937919747,3,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
277199,JCP2022_100922,5548998150937919747,3,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df_training = pd.read_csv(PROFILES_TRAINING_FILE, skiprows = 0)
df_training.shape
df_training.info()
df_training.head(5)
df_training.tail(10)
df_training.loc[277196:277205].Metadata_Source

(277200, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277200 entries, 0 to 277199
Columns: 161 entries, Metadata_Source to Feature_149
dtypes: float64(154), int64(2), object(5)
memory usage: 340.5+ MB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
0,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,-0.766995,0.014673,-0.800139,0.578975,3.133392,-1.437044,-0.005154,6.372777e-07,-0.620331,0.002278
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,0.611691,0.014401,-1.709864,0.125061,4.774752,5.228751,-0.009253,1.953434e-06,-1.609306,0.001220
2,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,...,0.547850,0.022825,-2.193591,1.296923,2.558569,0.306822,-0.004226,9.856348e-07,-0.878720,0.003899
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-0.081542,0.008125,-0.380961,0.126596,-0.937269,1.704932,0.002017,6.639099e-07,0.258712,-0.001949
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,...,-1.222203,0.012126,-0.561924,-1.161146,0.029847,0.268880,-0.010465,6.803231e-07,-2.683861,-0.003365


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
277190,source_9,20211102-Run15,GR00004398,J35,JCP2022_089792,5548393758141673059,3,100.28,4.607966,2.001214,...,-0.891894,0.015222,-1.257538,-0.833911,3.591903,-3.465932,-0.011691,6.681608e-07,-0.387147,-0.000635
277191,source_3,CP_29_all_Phenix1,SP28P31d,L08,JCP2022_054246,5548833810863940256,0,83.10,4.420045,1.919601,...,-1.109190,0.013911,0.462122,0.106805,1.026845,0.950131,-0.004305,6.619060e-07,0.808615,0.001707
277192,source_1,Batch6_20221102,UL000595,X16,JCP2022_054246,5548833810863940256,0,83.10,4.420045,1.919601,...,0.901602,0.010900,-0.502601,-0.142243,0.075448,0.067430,-0.005580,4.201894e-07,3.864306,-0.000615
277193,source_10,2021_08_23_U2OS_48_hr_run18,Dest210823-175355,G13,JCP2022_054246,5548833810863940256,0,83.10,4.420045,1.919601,...,0.676368,0.010665,1.403911,-0.316352,-0.204800,-5.588204,0.000008,6.868480e-07,2.229469,0.002284
277194,source_6,p211109CPU2OS48hw384exp035JUMP,110000295497,G21,JCP2022_040737,5548902807653184645,5,55.55,4.017284,1.744684,...,-0.658957,0.005685,0.212769,-0.269108,1.392122,0.308340,-0.003104,3.273515e-07,-1.106692,-0.000335
277195,source_8,J3,A1170503,G21,JCP2022_040737,5548902807653184645,5,55.55,4.017284,1.744684,...,1.735407,0.012612,-0.045320,0.243171,0.671047,-0.646722,-0.007726,5.766574e-07,1.592328,-0.001583
277196,source_10,2021_08_17_U2OS_48_hr_run16,Dest210809-140929,N19,JCP2022_040737,5548902807653184645,5,55.55,4.017284,1.744684,...,2.493746,0.017869,-0.235858,0.000158,0.294992,-2.660300,-0.017990,7.961061e-07,0.312739,-0.001225
277197,source_5,JUMPCPE-20211001-Run34_20211003_121618,APTJUM513,I14,JCP2022_100922,5548998150937919747,3,79.96,4.381527,1.902873,...,-0.351440,0.011190,1.859087,0.810238,0.198592,1.974035,-0.006640,7.616013e-07,-2.027458,0.001649
277198,source_11,Batch1,EC000029,J07,JCP2022_100922,5548998150937919747,3,79.96,4.381527,1.902873,...,-0.037638,0.030781,0.225563,0.576570,-0.996557,2.590239,-0.016402,1.021047e-06,-0.002854,0.004836
277199,source_5,JUMPCPE-20210706-Run06_20210706_235916,APTJUM213,I14,JCP2022_100922,5548998150937919747,3,79.96,4.381527,1.902873,...,0.958456,0.020998,-1.304987,-0.741198,9.816779,-1.327597,-0.016591,9.813371e-07,-3.367221,-0.000484


277196    source_10
277197     source_5
277198    source_11
277199     source_5
Name: Metadata_Source, dtype: object

In [30]:
df_length = len(df_training)
df_pp_len = len(df_pp_training)
df_length, df_pp_len
df_training.iloc[0,4], df_pp_training.iloc[0,0]
df_training.iloc[0,5], df_pp_training.iloc[0,1]
df_training.iloc[0,6], df_pp_training.iloc[0,2]

(277200, 277200)

('JCP2022_006020', 'JCP2022_006020')

(-9223347314827979542, -9223347314827979542)

(10, 10)

In [31]:
ttl_match, ttl_non_match = 0,0
for i in range(df_length):
    if (df_training.iloc[i,4] == df_pp_training.iloc[i, 0]) and \
       (df_training.iloc[i,5] == df_pp_training.iloc[i, 1]) and \
       (df_training.iloc[i,6] == df_pp_training.iloc[i, 2]):
        ttl_match += 1
    else:
        ttl_non_match += 1

print(f" Total matched rows: {ttl_match}")
print(f" Total unmatched rows: {ttl_non_match}")

 Total matched rows: 277200
 Total unmatched rows: 0


In [32]:
del df_training, df_pp_training

#### Read validation file

In [33]:
df_pp_val = pd.read_csv(PP_VALIDATION_FILE, skiprows = 0)  ## ,  names = header_cols, header = None)
df_pp_val.columns
df_pp_val.shape
df_pp_val.info()
df_pp_val.head(10)
df_pp_val.tail(10)

Index(['Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', '0', '2', '3', '4', '8', '9', '10',
       ...
       '9921', '9926', '9927', '9932', '9933', '9950', '9956', '10127', '10370', '10375'],
      dtype='object', length=806)

(21600, 806)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21600 entries, 0 to 21599
Columns: 806 entries, Metadata_JCP2022 to 10375
dtypes: int64(805), object(1)
memory usage: 132.8+ MB


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,0,2,3,4,8,9,10,...,9921,9926,9927,9932,9933,9950,9956,10127,10370,10375
0,JCP2022_101506,5549160564573729245,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,JCP2022_101506,5549160564573729245,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,JCP2022_101506,5549160564573729245,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,JCP2022_010664,5549241656037143871,15,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,JCP2022_010664,5549241656037143871,15,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,JCP2022_010664,5549241656037143871,15,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,JCP2022_042631,5549260080151258999,7,1,1,0,0,0,0,1,...,1,1,0,0,0,1,0,0,1,0
7,JCP2022_042631,5549260080151258999,7,1,1,0,0,0,0,1,...,1,1,0,0,0,1,0,0,1,0
8,JCP2022_042631,5549260080151258999,7,1,1,0,0,0,0,1,...,1,1,0,0,0,1,0,0,1,0
9,JCP2022_076833,5549671971685925262,14,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,0,2,3,4,8,9,10,...,9921,9926,9927,9932,9933,9950,9956,10127,10370,10375
21590,JCP2022_008895,6702970690958368233,9,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
21591,JCP2022_027954,6703000930786455825,1,0,0,0,0,1,0,0,...,0,1,1,1,1,1,1,0,0,0
21592,JCP2022_027954,6703000930786455825,1,0,0,0,0,1,0,0,...,0,1,1,1,1,1,1,0,0,0
21593,JCP2022_027954,6703000930786455825,1,0,0,0,0,1,0,0,...,0,1,1,1,1,1,1,0,0,0
21594,JCP2022_094244,6703113441807921775,15,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
21595,JCP2022_094244,6703113441807921775,15,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
21596,JCP2022_094244,6703113441807921775,15,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
21597,JCP2022_040230,6703257770840297119,15,1,1,1,0,1,1,1,...,0,0,0,0,1,0,1,0,0,0
21598,JCP2022_040230,6703257770840297119,15,1,1,1,0,1,1,1,...,0,0,0,0,1,0,1,0,0,0
21599,JCP2022_040230,6703257770840297119,15,1,1,1,0,1,1,1,...,0,0,0,0,1,0,1,0,0,0


In [34]:
df_val = pd.read_csv(PROFILES_VALIDATION_FILE, skiprows = 0)  ## ,  names = header_cols, header = None)
df_val.columns
df_val.shape
df_val.info()
df_val.head(10)
df_val.tail(10)

Index(['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin',
       'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA',
       ...
       'Feature_140', 'Feature_141', 'Feature_142', 'Feature_143', 'Feature_144', 'Feature_145', 'Feature_146', 'Feature_147',
       'Feature_148', 'Feature_149'],
      dtype='object', length=161)

(21600, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21600 entries, 0 to 21599
Columns: 161 entries, Metadata_Source to Feature_149
dtypes: float64(154), int64(2), object(5)
memory usage: 26.5+ MB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
0,source_2,20210607_Batch_2,1053601824,N20,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,-3.725241,0.035587,1.151401,-1.682419,1.115334,10.616944,-0.047067,2.667295e-06,-30.374733,-0.004683
1,source_10,2021_06_15_U2OS_48_hr_run6,Dest210615-151512,N20,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,3.537362,0.045151,-4.826554,-3.532433,1.080575,16.857992,-0.070492,2.864283e-06,-26.010828,-0.023570
2,source_8,J3,A1170508,G11,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,-6.562871,0.033947,-3.468480,-2.141973,5.308029,14.113111,-0.050354,2.270743e-06,-25.699251,-0.009998
3,source_9,20210915-Run10,GR00003318,W40,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,-1.015490,0.005167,-0.482003,0.217316,0.178134,1.514248,0.000233,3.182251e-07,-1.041412,0.000663
4,source_9,20210915-Run10,GR00003316,G16,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,-1.058542,0.004742,-0.373149,-0.008277,1.382519,-0.973441,0.003197,4.148939e-07,1.312948,0.002285
5,source_5,JUMPCPE-20210730-Run15_20210801_205335,ATSJUM216,G16,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,0.617887,0.006613,-1.223345,0.531935,1.121791,-3.338016,-0.000107,4.779483e-07,-0.466309,0.002921
6,source_1,Batch6_20221102,UL000595,I07,JCP2022_042631,5549260080151258999,7,76.07,4.331654,1.881213,...,-1.952502,0.010852,0.491809,0.247910,2.398642,-0.791524,0.003076,6.418766e-07,3.929215,0.002685
7,source_3,CP_29_all_Phenix1,SP28P31a,E04,JCP2022_042631,5549260080151258999,7,76.07,4.331654,1.881213,...,0.498358,0.012608,-0.769725,-0.369199,0.989697,-0.657078,-0.005128,4.519632e-07,2.045289,-0.000662
8,source_2,20210726_Batch_7,1086293515,I05,JCP2022_042631,5549260080151258999,7,76.07,4.331654,1.881213,...,1.102820,0.012800,-0.348906,0.129962,-0.481835,0.222491,-0.003976,4.389858e-07,1.218522,0.000852
9,source_8,J3,A1170507,O12,JCP2022_076833,5549671971685925262,14,58.34,4.066288,1.765966,...,-0.325361,0.004968,0.050010,-0.184485,-0.947841,0.729610,0.000213,4.091584e-07,1.008752,-0.000676


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
21590,source_5,JUMPCPE-20210812-Run20_20210815_062625,AEOJUM204,G03,JCP2022_008895,6702970690958368233,9,109.00,4.691348,2.037426,...,-1.861755,0.007400,-0.114016,-0.434690,3.120278,-2.578972,-0.001198,4.542935e-07,-0.498911,0.001321
21591,source_1,Batch3_20221010,UL000091,G43,JCP2022_027954,6703000930786455825,1,79.09,4.370586,1.898122,...,5.538791,0.017023,0.834009,1.391182,-6.567818,-2.104539,-0.000388,1.680923e-06,3.080817,-0.000330
21592,source_3,CP_26_all_Phenix1,J12440a,D22,JCP2022_027954,6703000930786455825,1,79.09,4.370586,1.898122,...,7.209506,0.022679,1.306062,0.366506,-5.610929,-1.721339,-0.005504,1.252931e-06,5.537314,-0.005735
21593,source_6,p210906CPU2OS48hw384exp025JUMP,110000294882,D22,JCP2022_027954,6703000930786455825,1,79.09,4.370586,1.898122,...,1.843124,0.012435,0.890134,-0.076146,-0.523715,-1.541798,0.002479,7.427523e-07,4.629504,-0.000046
21594,source_2,20210830_Batch_11,1086290668,N21,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-0.159453,0.004899,-0.141224,0.325938,-0.523688,-0.154702,-0.001160,3.761675e-07,0.618713,-0.000774
21595,source_8,J3,A1170495,G22,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-0.093747,0.006786,-0.408210,-0.093295,1.672522,-0.765932,-0.001401,4.373320e-07,-0.362496,-0.001280
21596,source_6,p211109CPU2OS48hw384exp035JUMP,110000295493,G22,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-0.119667,0.007582,0.029360,-0.017161,0.624752,-0.953376,-0.002586,4.124904e-07,-1.255258,0.000217
21597,source_9,20211103-Run16,GR00004419,R42,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,0.402903,0.008930,0.106090,0.210580,0.425021,-2.257501,-0.000105,4.425803e-07,2.820224,-0.001114
21598,source_9,20211103-Run16,GR00004418,B42,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,-0.055556,0.037215,-0.046159,0.188240,4.083176,7.762218,-0.021854,1.866700e-06,-8.252466,-0.001563
21599,source_9,20211103-Run16,GR00004421,B18,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,-0.654169,0.011018,-0.458332,-0.463053,-0.839896,3.992070,-0.007733,6.811076e-07,-2.166077,-0.001582


In [35]:
df_length = len(df_val)
df_pp_len = len(df_pp_val)
df_length, df_pp_len
df_val.iloc[0,4] , df_pp_val.iloc[0,0]
df_val.iloc[0,5] , df_pp_val.iloc[0,1]
df_val.iloc[0,6] , df_pp_val.iloc[0,2]

(21600, 21600)

('JCP2022_101506', 'JCP2022_101506')

(5549160564573729245, 5549160564573729245)

(13, 13)

In [36]:
ttl_match, ttl_non_match = 0,0
for i in range(df_length):
    if (df_val.iloc[i,4] == df_pp_val.iloc[i, 0]) and \
       (df_val.iloc[i,5] == df_pp_val.iloc[i, 1]) and \
       (df_val.iloc[i,6] == df_pp_val.iloc[i, 2]):
        ttl_match += 1
    else:
        ttl_non_match += 1

print(f" Total matched rows: {ttl_match}")
print(f" Total unmatched rows: {ttl_non_match}")

 Total matched rows: 21600
 Total unmatched rows: 0


In [37]:
del df_val, df_pp_val

#### Read test file

In [38]:
df_pp_test =  pd.read_csv(PP_TEST_FILE, skiprows = 0)  ## , names = header_cols)
df_pp_test.shape
df_pp_test.info()
df_pp_test.head(10)
df_pp_test.tail(10)

(12600, 806)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Columns: 806 entries, Metadata_JCP2022 to 10375
dtypes: int64(805), object(1)
memory usage: 77.5+ MB


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,0,2,3,4,8,9,10,...,9921,9926,9927,9932,9933,9950,9956,10127,10370,10375
0,JCP2022_004021,6703456398488902534,6,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,JCP2022_004021,6703456398488902534,6,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,JCP2022_004021,6703456398488902534,6,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,JCP2022_102146,6704177726803547665,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,JCP2022_102146,6704177726803547665,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,JCP2022_102146,6704177726803547665,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,JCP2022_015241,6704729456769442241,1,1,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
7,JCP2022_015241,6704729456769442241,1,1,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
8,JCP2022_015241,6704729456769442241,1,1,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,JCP2022_071726,6704991968420949852,12,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,0,2,3,4,8,9,10,...,9921,9926,9927,9932,9933,9950,9956,10127,10370,10375
12590,JCP2022_058635,7378615000324315629,13,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
12591,JCP2022_106565,7378620454050468236,12,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
12592,JCP2022_106565,7378620454050468236,12,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
12593,JCP2022_106565,7378620454050468236,12,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
12594,JCP2022_064379,7378624279819671723,11,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
12595,JCP2022_064379,7378624279819671723,11,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
12596,JCP2022_064379,7378624279819671723,11,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
12597,JCP2022_012996,7378937281046818562,2,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
12598,JCP2022_012996,7378937281046818562,2,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
12599,JCP2022_012996,7378937281046818562,2,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df_test = pd.read_csv(PROFILES_TEST_FILE, skiprows = 0)  ## , names = header_cols)
df_test.shape
df_test.info()
df_test.head(10)
df_test.tail(10)

(12600, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Columns: 161 entries, Metadata_Source to Feature_149
dtypes: float64(154), int64(2), object(5)
memory usage: 15.5+ MB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
0,source_9,20210914-Run9,GR00003302,H45,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-3.327352,0.047957,-5.146083,0.133483,7.258646,4.004018,-0.052651,2.564651e-06,-47.471184,0.020332
1,source_9,20210914-Run9,GR00003301,H21,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-2.712915,0.040120,-4.229829,-1.301542,10.483354,3.721997,-0.053937,2.532294e-06,-49.760448,0.016169
2,source_9,20210914-Run9,GR00003303,X45,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-3.817876,0.040926,-2.812590,-0.477595,7.014334,1.155897,-0.038983,2.649987e-06,-43.797977,0.018047
3,source_3,CP_32_all_Phenix1,B40803aW,P21,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,0.662736,0.033059,0.982629,-0.070947,-3.409516,1.021382,-0.003060,1.490827e-06,11.092926,-0.006313
4,source_6,p210928CPU2OS48hw384exp030JUMP,110000296383,P21,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,1.643660,0.017068,-1.317735,-0.000594,-0.204664,1.816564,-0.000245,7.096640e-07,4.869806,-0.001466
5,source_1,Batch5_20221030,UL001783,AE41,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,-3.379498,0.014683,0.078032,0.391880,1.668460,2.384432,-0.003760,7.669784e-07,3.671083,-0.001564
6,source_6,p210927CPU2OS48hw384exp029JUMP,110000296325,G11,JCP2022_015241,6704729456769442241,1,62.22,4.130676,1.793930,...,1.459078,0.012785,0.225619,0.539256,0.869810,-0.115279,-0.000246,6.821750e-07,0.040992,-0.002430
7,source_3,CP_31_all_Phenix1,B040903d,G11,JCP2022_015241,6704729456769442241,1,62.22,4.130676,1.793930,...,2.268785,0.016546,-0.865848,0.349065,-3.118612,-1.439584,-0.008098,5.918533e-07,3.096774,-0.000765
8,source_1,Batch5_20221030,UL001781,N22,JCP2022_015241,6704729456769442241,1,62.22,4.130676,1.793930,...,3.584120,0.009897,-0.883304,-0.050701,-0.870741,-2.643478,-0.002136,8.769338e-07,5.087862,-0.000349
9,source_6,p211123CPU2OS48hw384exp036JUMP,110000297126,I14,JCP2022_071726,6704991968420949852,12,76.96,4.343286,1.886265,...,-0.530850,0.008340,0.570113,0.573650,1.774152,-0.138683,-0.002314,5.440163e-07,-1.283157,0.000937


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Feature_140,Feature_141,Feature_142,Feature_143,Feature_144,Feature_145,Feature_146,Feature_147,Feature_148,Feature_149
12590,source_10,2021_06_07_U2OS_48_hr_run3,Dest210607-135241,K20,JCP2022_058635,7378615000324315629,13,66.22,4.192983,1.820989,...,3.019578,0.015101,0.824221,-0.378767,-1.250879,-2.019894,-0.012036,7.221733e-07,1.919275,-0.003441
12591,source_9,20210825_Run6,GR00003412,I35,JCP2022_106565,7378620454050468236,12,113.84,4.734794,2.056295,...,-1.206482,0.011832,-1.318972,0.228564,3.224792,-2.056447,0.000446,5.968393e-07,-2.114779,0.001228
12592,source_9,20210825_Run6,GR00003413,Y35,JCP2022_106565,7378620454050468236,12,113.84,4.734794,2.056295,...,-0.082789,0.004604,-0.072054,0.040112,0.417809,-1.126571,-0.001626,3.589462e-07,1.752626,-0.000938
12593,source_11,Batch4,EC000118,G19,JCP2022_106565,7378620454050468236,12,113.84,4.734794,2.056295,...,-2.060350,0.007820,-0.655703,0.090007,0.263418,1.159805,0.000513,4.555681e-07,0.632528,-0.000307
12594,source_10,2021_08_20_U2OS_48_hr_run17,Dest210810-174829,P09,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,1.601922,0.007995,-1.213057,-0.323118,-2.783082,-0.350303,0.003494,9.139573e-07,2.852537,-0.000331
12595,source_1,Batch5_20221030,UL001791,AF11,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,1.508342,0.004148,-0.399711,-1.065345,-8.670267,-1.937182,-0.001078,1.432617e-06,2.855982,0.000839
12596,source_3,CP_32_all_Phenix1,B40703cW,P06,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,-0.764604,0.011587,-1.286446,-0.889321,0.865635,2.475070,-0.010083,8.733644e-07,-1.249884,-0.001450
12597,source_10,2021_08_17_U2OS_48_hr_run16,Dest210809-141809,L22,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,3.019087,0.024101,-1.660990,1.069352,1.727055,0.946899,-0.007892,1.010004e-06,5.498997,-0.000881
12598,source_1,Batch3_20221010,UL000091,AB44,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,-3.011322,0.010392,0.344499,0.737810,0.477564,0.507359,0.005572,8.722242e-07,2.708362,0.002181
12599,source_6,p210906CPU2OS48hw384exp025JUMP,110000294885,N22,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,0.762565,0.012253,-0.518212,0.329818,-2.960701,-2.140489,-0.008443,7.789209e-07,-0.510535,0.003786


In [40]:
df_length = len(df_test)
df_pp_len = len(df_pp_test)
df_length, df_pp_len
df_test.iloc[0,4] , df_pp_test.iloc[0,0]
df_test.iloc[0,5] , df_pp_test.iloc[0,1]
df_test.iloc[0,6] , df_pp_test.iloc[0,2]

(12600, 12600)

('JCP2022_004021', 'JCP2022_004021')

(6703456398488902534, 6703456398488902534)

(6, 6)

In [41]:
ttl_match, ttl_non_match = 0,0
for i in range(df_length):
    if (df_test.iloc[i,4] == df_pp_test.iloc[i, 0]) and \
       (df_test.iloc[i,5] == df_pp_test.iloc[i, 1]) and \
       (df_test.iloc[i,6] == df_pp_test.iloc[i, 2]):
        ttl_match += 1
    else:
        ttl_non_match += 1

print(f" Total matched rows: {ttl_match}")
print(f" Total unmatched rows: {ttl_non_match}")

 Total matched rows: 12600
 Total unmatched rows: 0


In [42]:
del df_test, df_pp_test

# Write profiles to separate files based on `Metadata_Bin` 

  ### Used for XGBoost and Dask Clusters 

## Create Dask Clusters 

In [45]:
n_workers = 3
n_threads = 2
cuda_visible_devices = [0,1,2]

try:
    cluster.close()
    del cluster
except Exception as e:
    print(" *** Cluster close failed ***")
else:
    print(" *** Cluster close succeeded ***\n")
finally:
    # cluster = LocalCUDACluster( CUDA_VISIBLE_DEVICES=[0,1,2], n_workers=n_workers, threads_per_worker= n_threads, silence_logs = logging.WARNING)
    cluster = LocalCluster(n_workers=n_workers, threads_per_worker= n_threads, silence_logs = logging.WARNING)
    print(f"\n *** Cluster created - Cluster dashboard link : {cluster.dashboard_link}")

 *** Cluster close succeeded ***



NameError: name 'cluster' is not defined

In [44]:
try:
    client.close()
    del client
except Exception as e:
    print(" *** Client close failed *** ")
else:
    print(" *** Client close succeeded *** ")
finally:
    client = Client(cluster)    
    print("\n *** Client created *** ")
    client

 *** Client close succeeded *** 

 *** Client created *** 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:45785/status,
Dashboard: http://127.0.0.1:45785/status,Workers: 3
Total threads: 6,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36569,Workers: 3
Dashboard: http://127.0.0.1:45785/status,Total threads: 6
Started: 4 minutes ago,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:38879,Total threads: 2
Dashboard: http://127.0.0.1:39267/status,Memory: 21.33 GiB
Nanny: tcp://127.0.0.1:39665,


## Create n = 12 output files for the 12 output buckets:

In [38]:
NUMBER_OF_BINS = 12
input_files = []
output_files = []
for i in range(NUMBER_OF_BINS):
    # inputFile = os.path.join('./output_11102023/', '3_sample_profiles', f"3sample_profiles_{i:03d}.csv")
    outputFile = os.path.join('./output_11102023/', '3_sample_profiles', f"3sample_profiles_1482_bin_{i:03d}.csv")
    # print(outputFile)
    # input_files.append(inputFile)
    output_files.append(outputFile)
for i in range(NUMBER_OF_BINS):
    print(f"  -->    {output_files[i]:65s}")
# print(output_files)

  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_000.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_001.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_002.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_003.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_004.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_005.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_006.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_007.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_008.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_009.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_010.csv
  -->    ./output_11102023/3_sample_profiles/3sample_profiles_1482_bin_011.csv


## Read input files 

In [39]:
# profileInputFiles = './output_11102023/profiles/compound_profiles_2samples_001.csv'
# profileInputFiles = './output_11102023/profiles/compound_profiles_2samples_002.csv'
# MetadataColumns
profileInputFiles = './output_11102023/3_sample_profiles/3sample_profiles_1482_Unordered.csv'
category_conversions  = {"Metadata_Source":'category', "Metadata_Batch":'category' ,"Metadata_Plate":'category' , "Metadata_Well":'category'}
string_conversions    = {"Metadata_Source": 'string', "Metadata_Batch":'string' ,"Metadata_Plate":'string' , "Metadata_Well":'string'}

In [40]:
# del df_profiles
# df_profiles = read_cell_profiles(profileInputFiles)
logging.info(f" Read profiles file ...")
df_profiles = read_cell_profiles(profileInputFiles, 
                                 # names = COMPOUND_PROFILE_COLUMNS, 
                                 dtype= string_conversions,
                                )
# df_profiles = read_cell_profiles(profileInputFiles, rows = 200000, names = all_columns, usecols = Xy_columns, dtype = Xy_columns_dtype)
logging.info(f" Read profiles file ... complete")

df_profiles.head(3)

2024-04-23 21:24:40,927 - INFO: -  Read profiles file ...
2024-04-23 21:24:41,100 - INFO: -  Read profiles file ... complete


 Reading cell profiles file :  ./output_11102023/3_sample_profiles/3sample_profiles_1482_Unordered.csv    {}
 Reading ALL  rows - Number of partitions:  156   



/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,source_1,Batch1_20221004,UL001641,AA40,JCP2022_000082,-1920040449369071690,6,64.89,4.172694,1.812178,...,0.438577,-0.051575,0.279409,0.767113,0.143125,0.498506,-0.147239,-0.072483,0.059082,0.547492
1,source_1,Batch1_20221004,UL001641,Z22,JCP2022_000137,-9070688483616626497,15,60.28,4.099000,1.780173,...,0.522905,0.206919,-0.737207,0.050593,0.889717,-0.436981,0.109206,-0.024600,-0.490239,-0.262859
2,source_1,Batch1_20221004,UL001641,Z32,JCP2022_000428,4317235005617559002,10,94.53,4.548917,1.975570,...,0.258570,0.235806,0.430072,0.565591,0.588245,-0.338327,-0.014979,0.087962,-0.005913,0.229194


In [41]:
df_profiles.shape
df_profiles.Metadata_Bin.value_counts()
# df_profiles.info(verbose = True)
# df_profiles.memory_usage(deep= True).compute()

(Delayed('int-9068b106-62f5-44b7-b64a-65d0267846c9'), 1482)

Dask Series Structure:
npartitions=1
    int64
      ...
Name: count, dtype: int64
Dask Name: value-counts-agg, 4 graph layers

In [ ]:
# df_profiles.memory_usage(deep=True)[:10]
#  #   Column               Non-Null Count  Dtype
# ---  ------               --------------  -----
#  0   Metadata_Source      231017 non-null      string
#  1   Metadata_Batch       231017 non-null      string
#  2   Metadata_Plate       231017 non-null      string
#  3   Metadata_Well        231017 non-null      string
#  4   Metadata_JCP2022     231017 non-null      object
#  5   Metadata_Hash        231017 non-null      int64
#  6   Metadata_Bin         231017 non-null      int64
#  7   Metadata_TPSA        231017 non-null      float64
#  8   Metadata_lnTPSA      231017 non-null      float64
#  9   Metadata_log10TPSA   231017 non-null      float64
# 10   Metadata_Permiation  231017 non-null      int64
# dtypes: object(1), float64(3), int64(3), string(4)


#  #   Column               Dtype          Size(Categorical)    Size(string)
# ---  ------               -------         ---------------- ----------------
#      Index                                       33,396           33,396
#  0   Metadata_Source      category/string       249,067       15,057,278
#  1   Metadata_Batch       category/string       271,226       17,233,446
#  2   Metadata_Plate       category/string       429,214       15,503,673
#  3   Metadata_Well        category/string     9,562,119       13,872,452
#  4   Metadata_JCP2022     object             16,402,207       16,402,207
#  5   Metadata_Hash        int64               1,848,136        1,848,136
#  6   Metadata_Bin         int64               1,848,136        1,848,136
#  7   Metadata_TPSA        float64             1,848,136        1,848,136
#  8   Metadata_lnTPSA      float64             1,848,136        1,848,136
#  9   Metadata_log10TPSA   float64             1,848,136        1,848,136
# 10   Metadata_Permiation  int64               1,848,136        1,848,136
 

**Memory Usage:**
 |No |Field name| Type | categorical | string[python] | 
 |---|-----------|-----|-----------------:|------------------:
 |    | Index               |                 |       33,396  |         33,396 
 | 0  | Metadata_Source     | category/string |      249,067  |     15,057,278 
 | 1  | Metadata_Batch      | category/string |      271,226  |     17,233,446 
 | 2  | Metadata_Plate      | category/string |      429,214  |     15,503,673 
 | 3  | Metadata_Well       | category/string |    9,562,119  |     13,872,452 
 | 4  | Metadata_JCP2022    | object          |   16,402,207  |     16,402,207 
 | 5  | Metadata_Hash       | int64           |    1,848,136  |      1,848,136 
 | 6  | Metadata_Bin        | int64           |    1,848,136  |      1,848,136 
 | 7  | Metadata_TPSA       | float64         |    1,848,136  |      1,848,136 
 | 8  | Metadata_lnTPSA     | float64         |    1,848,136  |      1,848,136 
 | 9  | Metadata_log10TPSA  | float64         |    1,848,136  |      1,848,136 
| 10  | Metadata_Permiation | int64           |    1,848,136  |      1,848,136 

In [ ]:
# df_profiles.Metadata_Source.dtype
# df_profiles.Metadata_Source.memory_usage(deep=True).compute()

## Group by Bin Number

In [42]:
df_grouped_profiles = df_profiles.groupby('Metadata_Bin')

### Count num of rows in each bin

In [43]:

df_profiles_bin = df_profiles['Metadata_Bin']
type(df_profiles_bin)
df_profiles_bin.npartitions

s1 = df_profiles_bin.value_counts().compute()
s1.sort_index()
s1.sum()

# Metadata_Bin
# 0     19279
# 1     18815
# 2     19048
# 3     19421
# 4     19368
# 5     19332
# 6     19480
# 7     19298
# 8     19246
# 9     19189
# 10    19183
# 11    19358
# Name: count, dtype: int64


dask.dataframe.core.Series

156

/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


Metadata_Bin
0     21780
1     21435
2     21804
3     21714
4     21756
5     21315
6     21351
7     21834
8     21291
9     21606
10    21783
11    21645
12    22014
13    21654
14    21633
15    21927
Name: count, dtype: int64

346542

## Read Each bin group ansd write to separte file

In [35]:
# BIN_NO = 2

for BIN_NO in range(12): ## [0,1,2,3,4,5,7, 8, 9 , 10, 11]:
    
    logging.info(f" Get bin group: {BIN_NO} from df_grouped_profiles")    
    
    grp = df_grouped_profiles.get_group((BIN_NO,))
    logging.info(f" Number of partitions :  {grp.npartitions}")
    grp.head(3)
    
    # ss = grp.map_partitions(len).compute()
    # ss
    # ss.sum()
    
    logging.info(f" Write bin {BIN_NO} to  {output_files[BIN_NO]}")
    
    grp.to_csv(output_files[BIN_NO], index = False, single_file = True)
    
    logging.info(f" Writing bin {BIN_NO} profiles file ... complete")

2024-04-23 17:22:35,758 - INFO: -  Get bin group: 0 from df_grouped_profiles
2024-04-23 17:22:36,165 - INFO: -  Number of partitions :  156
2024-04-23 21:22:54,198 - ERROR: - Task exception was never retrieved
future: <Task finished name='Task-214751' coro=<Client._gather.<locals>.wait() done, defined at /home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit


KeyboardInterrupt: 

### Read Each bin and verify count

In [ ]:
# BIN_READ = 2

for BIN_READ in  range(12) : ## [7, 8, 9 , 10, 11]:
    logging.info(f" Read from  {output_files[BIN_READ]}  for bin {BIN_READ} ")
    grp =  dd.read_csv(output_files[BIN_READ], usecols = ['Metadata_JCP2022', 'Metadata_Bin'],)
    grp.npartitions  
    grp.head()
    ss = grp.map_partitions(len).compute()
    logging.info(f" Total rows in bin {BIN_READ} :   {ss.sum()}")


# Build Profiles Metadata file `profiles_metadata.pkl`

In [ ]:
# print(f" Features select file:  {profilesMetadataFile,}")

# with open(profilesMetadataFile,, 'rb') as f: 
#     x = pickle.load(f)
    
# for i in x:
#     print(f" {i:20s}    {len(x[i])} ")

metadata_columns = set(["Metadata_Source", "Metadata_Batch", "Metadata_Plate", "Metadata_Well", "Metadata_JCP2022"])
metadata_columns_dtype = {x: pd.StringDtype()  for x in metadata_columns} 

label_columns =  set(["Metadata_TPSA", "Metadata_lnTPSA", "Metadata_log10TPSA"])
label_columns_dtype = {x: np.dtype('float32') for x in label_columns}
label_columns_dtype["Metadata_permiation"] = np.dtype('int8')

X_columns = x['selected']
X_columns_conversion = {x: np.dtype('float32') for x in X_columns}

y_columns = set(["Metadata_permiation"])
y_columns_dtype = {x: np.dtype('int8') for x in y_columns} ## "Metadata_log10TPSA":np.dtype('float64')}

# y_columns = set(["Metadata_log10TPSA"])
# y_columns_dtype = {x: np.dtype('float32') for x in y_columns} ## "Metadata_log10TPSA":np.dtype('float64')}

Xy_columns = y_columns | X_columns
Xy_columns_dtype = y_columns_dtype | X_columns_dtype

all_columns = pd.read_csv( './output_11102023/profiles/compound_profiles_2samples_001.csv', nrows = 0).columns.tolist()
# all_columns = metadata_columns | label_columns | X_columns
# all_columns_dtype = metadata_columns_dtype | label_columns_dtype | X_columns_dtype

print(f" len(metadata_columms)        : {len(metadata_columns)} ")
print(f" len(label_columms)           : {len(label_columns)} ")
print(f" len(X_columms)               : {len(X_columns)}")
print(f" len(y_columms)               : {len(y_columns)}")
print(f" len(Xy_columms)              : {len(Xy_columns)}")
print(f" len(all_columms)             : {len(all_columns)} ")

# print(f" len(metadata_columms_dtype)  : {len(metadata_columns_dtype)} ")
# print(f" len(label_columms_dtype)     : {len(label_columns_dtype)} ")
# print(f" len(X_columms_dtype)         : {len(X_columns_dtype)}")
# print(f" len(y_columms_dtype)         : {len(y_columns_dtype)}")
# print(f" len(Xy_columms_dtype)        : {len(Xy_columns_dtype)}")
# print(f" len(all_columms_dtype)       : {len(all_columns_dtype)} ")
# del all_columns_dtype
# for k  in sorted(all_columns_dtype.keys()):
#     print(f" {k:60s}  {all_columns_dtype[k]}")

In [ ]:
feature_selection_columns = { 'selected': ttl_selected_cols,
                              'dropped_correlation': ttl_corr_dropped_cols,
                              'dropped_variance': set()
                              'dropped_nans': set()}
# with open("./metadata/feature_selection_columns.pkl", 'wb') as f: 
#     pickle.dump(feature_selection_columns,f)

In [ ]:
 
# all_profile_columns = df_profiles.columns.to_list()

In [ ]:
MetadataOriginalNames      = ["Metadata_Source", "Metadata_Batch" ,"Metadata_Plate" , "Metadata_Well", "Metadata_JCP2022", "Metadata_Hash", 
                              "TPSA"  , "lnTPSA"  , "log10TPSA", "permiation"]
                        
MetadataNames              = ["Metadata_Source", "Metadata_Batch" , "Metadata_Plate" , "Metadata_Well"     , "Metadata_JCP2022", "Metadata_Hash", 
                              "Metadata_Bin"   , "Metadata_TPSA"  , "Metadata_lnTPSA", "Metadata_log10TPSA", "Metadata_Permiation"]

MetadataCategoricalColumns = {"Metadata_Source":'category', "Metadata_Batch":'category' ,"Metadata_Plate":'category' , "Metadata_Well":'category'}

MetadataFloatColumns       = {"Metadata_TPSA":np.float32, "Metadata_lnTPSA":np.float32, "Metadata_log10TPSA":np.float32}

MetadataIntegerColumns     = {"Metadata_Hash": np.int64, "Metadata_Bin":np.int8, "Metadata_Permiation":np.int8}

MetadataNumericColumns     =  MetadataFloatColumns | MetadataIntegerColumns

metadata_columns = {"MetadataOriginalNames"      : MetadataOriginalNames,
                    "MetadataNames"              : MetadataNames,
                    "MetadataCategoricalColumns" : MetadataCategoricalColumns,
                    "MetadataNumericColumns"     : MetadataNumericColumns,
                    "MetadataFloatColumns"       : MetadataFloatColumns,
                    "MetadataIntegerColumns"     : MetadataIntegerColumns,
                   }


pickle_data['metadata_columns'] = metadata_columns

## Read Profile Metadata file

In [ ]:
print(f" Features selection file:  {profileMetadataFile}")

with open(profileMetadataFile, 'rb') as f: 
     pickle_data = pickle.load(f)
pickle_data.keys()

In [ ]:
disp_metadata_file(pickle_data)
 

In [ ]:
# pickle_data['selected_columns'].keys()
# pickle_data['selected_columns']['dropped_nans']  = {'Cells_AreaShape_FormFactor',
#                                                      'Cells_Neighbors_AngleBetweenNeighbors_5',
#                                                      'Cells_Neighbors_AngleBetweenNeighbors_Adjacent',
#                                                      'Cells_Neighbors_FirstClosestDistance_5',
#                                                      'Cells_Neighbors_FirstClosestDistance_Adjacent',
#                                                      'Cells_Neighbors_SecondClosestDistance_5',
#                                                      'Cells_Neighbors_SecondClosestDistance_Adjacent',
#                                                      'Cytoplasm_AreaShape_FormFactor',
#                                                      'Nuclei_AreaShape_FormFactor',
#                                                      'Nuclei_Neighbors_AngleBetweenNeighbors_1',
#                                                      'Nuclei_Neighbors_FirstClosestDistance_1',
#                                                      'Nuclei_Neighbors_SecondClosestDistance_1'}


# pickle_data['selected_columns']['dropped_image_features'] = set(list(ParquetColumns['Image'].keys()))
# len(pickle_data['selected_columns']['dropped_image_features'])
# pickle_data['selected_columns']['dropped_metadata'] = set(list(ParquetColumns['Metadata'].keys()))
# len(pickle_data['selected_columns']['dropped_metadata'])

In [ ]:
# for i in ['Cells_AreaShape_FormFactor', 'Cytoplasm_AreaShape_FormFactor', 'Nuclei_AreaShape_FormFactor'
#           'Cells_Neighbors_AngleBetweenNeighbors_5',
#           'Cells_Neighbors_AngleBetweenNeighbors_Adjacent',
#           'Cells_Neighbors_FirstClosestDistance_5',
#           'Cells_Neighbors_FirstClosestDistance_Adjacent',
#           'Cells_Neighbors_SecondClosestDistance_5',
#           'Cells_Neighbors_SecondClosestDistance_Adjacent',
#           'Nuclei_Neighbors_AngleBetweenNeighbors_1',
#           'Nuclei_Neighbors_FirstClosestDistance_1',
#           'Nuclei_Neighbors_SecondClosestDistance_1']:

# for i in pickle_data["selected_columns"]["dropped_nans"]:
    
#     if i in pickle_data['all_profile_columns']: 
#     # if i in pickle_data['selected_columns']['dropped_correlation']: 
#         # del parquetNumericColumns [x]
#         print(f" FOUND in pickle_data['all_profile_columns'], deleted -- {i:35s}")
#     else:
#         print(f" NOT FOUND in pickle_data['all_profile_columns'], skipped -- {i:35s}")
# len(pickle_data['all_profile_columns'])
# t = set(pickle_data['all_profile_columns'])  - set(pickle_data['metadata_columns']['MetadataColumns'])
# len(t)
# # t = set(pickle_data['all_profile_columns'])  - set(pickle_data['selected_columns']['selected'])
# # len(t)
# t1 = t -set(pickle_data['selected_columns']['dropped_correlation'])
# len(t1)
# t1

In [ ]:
# ttl_bad_columns =  {'Cells_Neighbors_AngleBetweenNeighbors_Adjacent', 
#                     'Cells_Neighbors_AngleBetweenNeighbors_5', 
#                     'Cells_Neighbors_FirstClosestDistance_Adjacent', 
#                     'Cells_Neighbors_SecondClosestDistance_Adjacent',
#                     'Cells_Neighbors_FirstClosestDistance_5', 
#                     'Cells_Neighbors_SecondClosestDistance_5',
#                     'Nuclei_Neighbors_AngleBetweenNeighbors_1', 
#                     'Nuclei_Neighbors_SecondClosestDistance_1', 
#                     'Nuclei_Neighbors_FirstClosestDistance_1', 
#                    }
# len(ttl_bad_columns)
# pickle_data['selected_columns']['dropped_nans'] = ttl_bad_columns | {'Cells_AreaShape_FormFactor', 'Cytoplasm_AreaShape_FormFactor', 'Nuclei_AreaShape_FormFactor'}
# len(pickle_data['selected_columns']['dropped_nans']) 
# pickle_data['selected_columns']['dropped_nans']  

In [ ]:
# tmp_list = []
# check_in = pickle_data['all_profile_columns']
# check_in = pickle_data['selected_columns']['dropped_correlation']

# for i in ttl_bad_columns: 
#     if i in check_in:
#         print(f"{i} is in DROPPED CORRELATION" )
#         # pickle_data['selected_columns']['dropped_correlation']
#         # print(f"{i} is in 'all_profile_columns'" )
#         # pickle_data['all_profile_columns'].remove(i)
#         tmp_list.append(i)        
#     else:
#         # print(f" ***** {i} is NOT in all_profile_columns'" )
#         # print(f"{i} is NOT in DROPPED CORRELATION" )
#         # if i in pickle_data['selected_columns']['dropped_correlation']:
#             # print(f"{i} is in DROPPED CORRELATION" )
#         # else :
#         pass
# tmp_list         
# pickle_data['selected_columns']['dropped_correlation'] = pickle_data['selected_columns']['dropped_correlation'] - set(tmp_list)

In [ ]:
# dump_data = { 'all_profile_columns'   : COMPOUND_PROFILE_COLUMNS,  ## pickle_data['all_profile_columns'],
#                 'metadata_columns'    : pickle_data['metadata_columns'],
#                 'selected_columns'    : pickle_data['selected_columns'],
#                 'parquet_columns'     : pickle_data['parquet_columns'] }

In [ ]:

with open(profileMetadataFile, 'wb') as f: 
     pickle.dump(pickle_data,f)


In [ ]:

with open(profileMetadataFile, 'rb') as f: 
    pickle_data = pickle.load(f)
    

In [ ]:
disp_metadata_file(pickle_data)

# Build Metadata File of Parquet column names (if not existing)

Build dictionary of Parquet data columns - write columns to pickle

This list of column names will allow us to identify & control the columns we want to load from parquet files

In [ ]:
with open("./metadata/parquet_columns.pkl",'rb') as f:
    ParquetColumns = pickle.load(f)

for k,v in ParquetColumns.items():
    print(f" {k:20s}   items: {len(v)}")
# print(ParquetColumns['Cytoplasm'])

parquetNumericColumns  =  ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei']

print(f"\n\n all numeric columns (Cells + Cytoplasm + Nuclei) : {len(parquetNumericColumns )}")


In [ ]:
# # keys = ('source_1', 'Batch1_20221004', 'UL001641')
# keys = ('source_6' , 'p210920CPU2OS48hw384exp028JUMP',   '110000296345') 

# dframe = read_profiles_from_local(keys, local_formatter,  columns = None)
# cols = dframe.columns.to_list()
# print(f" rows: {len(dframe)}  columns {len(cols)}")      
# cat_cols = cat_columns(dframe)

# tt = [] 
# for k in cols:
#     if (dframe[k].dtype in [np.int64]):
#         print(f" {k:50s}   {dframe[k].dtype}")
#         tt.append(k)
# len(tt)


In [ ]:
# ParquetColumns = { 'Metadata' : {x: dframe[x].dtype for x in cat_cols['Metadata']},
#                    'Cells'    : {x: dframe[x].dtype for x in cat_cols['Cells']},
#                    'Cytoplasm': {x: dframe[x].dtype for x in cat_cols['Cytoplasm']},
#                     'Nuclei'  : {x: dframe[x].dtype for x in cat_cols['Nuclei']},
#                     'Image'   : {x: dframe[x].dtype for x in (cat_cols['Image_Granularity']+ cat_cols['Image_ImageQuality'] + 
#                                                               cat_cols['Image_Intensity']  + cat_cols['Image_Texture']      + 
#                                                               cat_cols['Image_Threshold'])},
#                  }

# print(len(floatConversion) + len(intConversion))
# import pickle
# with open("./metadata/parquet_columns.pkl", 'wb') as f: 
#     pickle.dump(ParquetColumns,f)

In [ ]:
# del ParquetColumns

# with open("./metadata/parquet_columns.pkl",'rb') as f:
#     ParquetColumns = pickle.load(f)

# for k,v in ParquetColumns.items():
#     print(f" {k:20s}   items: {len(v)}")

# for k,v in ParquetColumns['Cytoplasm'].items():
#     print(f" {k:50s}   type: {v}")


# `Nan` detection on Compound / Profiles dataset - Development

In [ ]:
check_fld_grps = {'_'.join(x.split('_')[:2]) for x in ParquetColumns['Cells'] }
# ParquetColumns['Cells']
# ParquetColumns['Nuclei']

In [ ]:
import re
pattern = re.compile("Cells_Neighbors*")
cell_neighbor_cols = list(filter(pattern.match, list(ParquetColumns['Cells'].keys())))
cell_neighbor_cols

pattern = re.compile("Nuclei_Neighbors*")
nuclei_neighbor_cols = list(filter(pattern.match, list(ParquetColumns['Nuclei'].keys())))
nuclei_neighbor_cols

check_cols = cell_neighbor_cols + nuclei_neighbor_cols
check_cols

## Load Extended metadata

The following files contain the metadata information for the entire dataset.
The schema is [here](metadata/README.md).

In [13]:
CompoundExtendedMetadata2SampleFile
CompoundExtendedMetadata3SampleFile

INPUT_FILE = CompoundExtendedMetadata3SampleFile
print(f" Input file: {INPUT_FILE}")
# MetadataNames
MetadataCategoricalColumns

'./output_11102023/compound_extended_metadata_2samples.csv'

'./output_11102023/compound_extended_metadata_3samples.csv'

 Input file: ./output_11102023/compound_extended_metadata_3samples.csv


NameError: name 'MetadataCategoricalColumns' is not defined

In [ ]:
ext_metadata = pd.read_csv(INPUT_FILE ,
                           # usecols = [0,1,2,3,4,5,  9,10,11,12], 
                           # usecols = MetadataNames, 
                           # names=extMetadataNames,  
                           header=0,
                          )
ext_metadata.sort_values(by=list(MetadataCategoricalColumns), inplace=True)

ext_metadata.info()
# ext_metadata.head(5)
# ext_metadata.tail(5)
ext_metadata.reset_index(drop=True, inplace = True)

# print('---------------------------------------------------------------')
# print(' After Index Reset')
# print('---------------------------------------------------------------')
# ext_metadata.info()
# ext_metadata.head(3)
# ext_metadata.tail(3)

ext_metadata.iloc[199995:200005,:6]
ext_metadata.memory_usage(deep=True) 


## Group  Metadata by Source, Batch, and Plate Id

In [ ]:
grouped_metadata = ext_metadata.groupby(by = ['Metadata_Source','Metadata_Batch','Metadata_Plate'])
type(grouped_metadata)
len(grouped_metadata)

for ctr, (keys, group_data) in enumerate(grouped_metadata):
    print(f"{ctr:5d}     {keys}") 
    if ctr > 10:
        break

## For each Metadata group (Plate) read matching parquet profile

In [ ]:
# keys =  ('source_1', 'Batch6_20221102', 'UL000597')
# keys =  ('source_10', '2021_06_08_U2OS_48_hr_run4', 'Dest210608-152921') 
# keys =  ('source_10', '2021_08_23_U2OS_48_hr_run18', 'Dest210823-174556')
# keys =  ('source_11', 'Batch3', 'EC000136')
keys = ('source_11',	'Batch3',	'EC000157'	)


In [ ]:
group_data = ext_metadata.query('Metadata_Source == @keys[0] & Metadata_Batch == @keys[1] & Metadata_Plate == @keys[2]')
type(group_data)
group_data.columns
group_data
# group_data[[check_cols].isna().any()
indices_to_change = group_data.index
indices_to_change

In [ ]:
# ext_metadata.loc[indices_to_change, 'Metadata_Plate']  = 'Kevin'
# ext_metadata.loc[indices_to_change] 

In [ ]:
 # 4  key: ('source_1', 'Batch6_20221102', 'UL000595')   1
 # 5  key: ('source_1', 'Batch6_20221102', 'UL000597')   388
 # 6  key: ('source_10', '2021_06_08_U2OS_48_hr_run4', 'Dest210608-152921')   1
 # 7  key: ('source_10', '2021_08_23_U2OS_48_hr_run18', 'Dest210823-174556')   1
 # 8  key: ('source_11', 'Batch3', 'EC000136')   1
 # 9  key: ('source_11', 'Batch3', 'EC000157')   129

In [ ]:
# keys =  ('source_1', 'Batch6_20221102', 'UL000595')
# keys =  ('source_1', 'Batch6_20221102', 'UL000597')
# keys =  ('source_11', 'Batch3', 'EC000136')
# keys = ('source_11',	'Batch3',	'EC000157'	)
# keys

# source_path = local_formatter.format(root_folder, keys[0], keys[1], keys[2])
# source_path 

# dframe = read_profiles_from_local(root_folder, keys, local_formatter,  READ_PARQUET_COLUMNS)
# dframe

# indices_to_change = dframe.index
# indices_to_change

# dframe['Metadata_Plate'] = 'EC000157'
# dframe

In [ ]:
# dframe.to_parquet(source_path)

In [ ]:
# df_join = pd.merge(group_data, dframe, how='left', on=['Metadata_Source', 'Metadata_Plate', 'Metadata_Well'])
# df_join

## Run Nan detection routine against Metadata/Profile join

In [ ]:
# del na_rows, rows_read
df_profiles_list = []
group_id = 0
ttl_meta_rows = 0
ttl_rows_read = 0
ttl_na_rows = 0 
ttl_bad_columns = set()
df_Nans = pd.DataFrame()

In [ ]:
START_GROUP = 200
END_GROUP  = 9999

for group_id, (keys, group_data) in enumerate(grouped_metadata,1):
    if group_id <= START_GROUP: 
        continue
        
    df_profile = read_profiles_from_local(root_folder, keys, local_formatter,  READ_PARQUET_COLUMNS)
    df_join = pd.merge(group_data, df_profile, how='left', on=['Metadata_Source', 'Metadata_Plate', 'Metadata_Well'])
    
    print(f" group: {group_id}  key: {keys}  metadata size: {len(group_data)}    profile size: {len(df_profile )}    join_size: {len(df_join)}")
    ttl_rows_read, ttl_na_rows, ttl_bad_columns, df_Nans = check_df_for_nans(df_join, group_id, ttl_rows_read , ttl_na_rows, ttl_bad_columns, df_Nans)    

    ttl_meta_rows  += len(group_data)
 
    if group_id >= END_GROUP:
        break

print("\n")
print(f" # of groups: {group_id} \n")
print(f" ttl_meta_rows - total number of metadata rows: {ttl_meta_rows}")
print(f" ttl_rows_read - total join rows read   : {ttl_rows_read}")  
print(f" ttl_na_rows   - total invalid rows     : {ttl_na_rows}")
print(f"\n Bad columns: {ttl_bad_columns}")
print("\n")


In [ ]:
df_Nans.info()
# list(df_Nans.index)
# df_Nans.columns

df_Nans_T = df_Nans.transpose()

In [ ]:
df_Nans_T.info()
df_Nans_T

In [ ]:
for group_id, (keys, group_data) in enumerate(df_Nans_T.groupby(by = ['Metadata_Source','Metadata_Batch','Metadata_Plate'])):
    print(f" {group_id}  key: {keys}   {len(group_data)}")

In [ ]:
check_cols

In [ ]:
for i in check_cols: 
    df_Nans_T[i].isna().value_counts()

In [ ]:
c = 100
df_Nans_T.iloc[195:201][['Metadata_JCP2022']+check_cols]

In [ ]:
df_join.head(4)
# df_profiles['Metadata_Plate'].unique()

row_num, na_rows, bad_columns = check_df_for_nans(df_join)
print(f" Rows read  : {row_num}   invalid rows : {na_rows}")
print(f" Bad columns: {bad_columns}")

In [ ]:
# COMPOUND_PROFILE_COLUMNS = df_profiles.columns.to_list()
# len(COMPOUND_PROFILE_COLUMNS)
len(READ_PARQUET_COLUMNS)
# col_dict = cat_columns(df_profiles)
# col_dict['Nuclei'][1000:]
JOIN_COLUMNS[:12]

# Normalize by Plate|

**`normalize Parameters()`**

- **profiles :** pandas.core.frame.DataFrame or path\
    Either a pandas DataFrame or a file that stores profile data
- **features :** list\
    A list of strings corresponding to feature measurement column names in the `profiles` DataFrame. All features listed must be found in `profiles`.\
    Defaults to `"infer"`. If `"infer"`, then assume cell painting features are those prefixed with `"Cells"`, `"Nuclei"`, or `"Cytoplasm"`.
- **image_features:** bool, default False\
    Whether the profiles contain image features.
- **meta_features :** list\
    A list of strings corresponding to metadata column names in the `profiles`DataFrame. All features listed must be found in `profiles`. Defaults to "infer".
    If `"infer"`, then assume metadata features are those prefixed with `"Metadata"`
- **samples :** tsr, default `allr\
    The metadata column values to use as a normalization reference. We often us econtrol samples. The function uses a `pd.query()` function, so you should\
    structure samples in this fashion. An example is"Metadata_treatment == 'control'" (include all quotes). Defaults to "all".
- **method :** str  `'standardize'` | `'robustize'` | `'mad_robustize'` | `'spherize'` \
    How to normalize the dataframe. Defaults to "standardize". Check avail_method sfor available normalization methods.
- **output_file :** str, optional\
    If provided, will write normalized profiles to file. If not specified, will return the normalized profiles as outpu.t\
    We recommend that this output file b  suffixed with "_normalized.csv".
- **output_type :** str, optional\
    If provided, will write normalized profiles as a specified file type (either CSV or parquet).
    If not specified and output_file is provided, then the file will be outputed as CSV as default.
- **compression_options :** str or dict, optional\
    Contains compression options as input to pd.DataFrame.to_csv(compression=compression_options). pandas version >= 1.2.
- **float_format :** str, optional\
    Decimal precision to use in writing output file as input to
    pd.DataFrame.to_csv(float_format=float_format). For example, use "%.3g" for 3
    decimal precision.
- **mad_robustize_epsilon:** float, optional\
    The mad_robustize fudge factor parameter. The function only uses
    this variable if method = "mad_robustize". Set this to 0 if
    mad_robustize generates features with large values.
- **spherize_center :** bool\
    If the function should center data before sphering (aka whitening). The
    function only uses this variable if method = "spherize". Defaults to True.
- **spherize_method :** str\
    The sphering (aka whitening) normalization selection. The function only uses
    this variable if method = "spherize". Defaults to "ZCA-corr". See
    :py:func:`pycytominer.operations.transform` for available spherize methods.
- **spherize_epsilon :** float, default 1e-6.\
    The sphering (aka whitening) fudge factor parameter. The function only uses
    this variable if method = "spherize".
  
**Returns**
 
- **normalized :** pandas.core.frame.DataFrame, optional\
    The normalized profile DataFrame. If output_file=None, then return the
    DataFrame. If you specify output_file, then write to file and do not return
    data.
, then write to file and do not return
        data
.

In [ ]:
try:
    df_normalized = normalize(
        profiles=df_join,
        features="infer",
        meta_features="infer",
        samples="all",
        method="standardize",
        # method="robustize",
        # method="mad_robustize",
        # method="spherize",
        # output_file=None,
        # mad_robustize_epsilon = 0 , 
        # compression_options=compression_options,
    )    
except Exception as e:
    print(f" exception:  {e} occurred at {keys}")

In [ ]:
df_normalized.head(5)

In [ ]:
# df_normalized[:10]
# df_normalized.isna().any(axis=1)

In [ ]:
disp_stats(df_normalized, col_dict['Cells'][:20])

In [ ]:
disp_stats(df_normalized, ['Cells_AreaShape_Area', 'Nuclei_Texture_Variance_RNA_10_03_256'])

## Display and plot normalized columns

In [ ]:
# disp_stats(df_normalized_robust, ['Cells_AreaShape_Area', 'Nuclei_Texture_Variance_RNA_10_03_256'])
# disp_stats(df_normalized_mad, ['Cells_AreaShape_Area', 'Nuclei_Texture_Variance_RNA_10_03_256'])
# len(col_dict['Cells'])
# col_dict['Cells'][:20]

In [ ]:
# fig = px.box(df_normalized, y=cols_subset[:34], notched=True)
import plotly.graph_objects as go
from plotly.colors import n_colors
# px.violin(df_normalized, y = col_list,  width=1600, height=800  )
 

col_list = col_dict['Cells'][:20]
col_list = col_dict['Cells'][:35]

colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', len(col_list), colortype='rgb')
# del fig
fig = go.Figure();
for data_line, color in zip(col_list , colors):
    col_name = '_'.join(data_line.split('_')[2:])
    fig.add_trace(go.Violin(y=df_normalized[data_line], line_color=color, name = col_name,box_visible=True,));
fig.update_layout(xaxis_showgrid=False, 
                    xaxis_zeroline=True,
                    autosize=False,
                    width=2500,
                    height=750,
                    margin=dict(
                        l=50,
                        r=50,
                        b=20,
                        t=20,
                        pad=4
                    ),
                    paper_bgcolor="LightSteelBlue",
);

In [ ]:
fig.show()

# Read Binned Profile files

In [ ]:
# profilesFile = CompoundProfiles2SampleFile + '_001' + CSV
# oldOutputFile ='./output_11102023/compound_profiles_2samples_old2.csv.bz2'
x=17
x = 1
x = 4
profilesFile =  "./output_11102023/binned_profiles/binned_2sample_profiles_{0:03d}.csv"
print(f" Read from {profilesFile.format(x)}")

In [ ]:
df_profiles =  pd.read_csv(profilesFile, 
                           # nrows = 20,
                           # skiprows = 0, 
                           # header = 0, 
                           names = COMPOUND_PROFILE_COLUMNS, 
                           usecols = MetadataNames, 
                           # usecols = ['Metadata_JCP2022', 'Metadata_permiation', 'Cells_AreaShape_Area']
                           dtype= string_conversions,
                           # dtype = conversionDict,
                          )

In [ ]:
df_profiles.shape
df_profiles.columns
# df_profiles.loc[0]
# df_profiles.memory_usage(deep=True).sum()
# df_profiles.memory_usage(deep=True)[:20]
df_profiles.head(5)
# df_profiles.info(verbose=True)
# HEADER_LIST = df_profiles.columns.to_list()

In [ ]:
# files = [f for f in pathlib.Path().glob(f"{output_path}/profiles/*.csv")]
# files.sort()
# files
# for profilesFile in files:

# for x in range(1,12):
#     profilesFile = CompoundProfiles2SampleFile + f"_{x:03d}" + CSV
#     profilesFile_a = CompoundProfiles2SampleFile + f"_{x:03d}_a" + CSV
    
#     df_iter =  pd.read_csv(profilesFile, 
#                            low_memory = True,
#                            # skiprows = 0,
#                            # nrows = 5,
#                            header = 0,
#                            # chunksize= 10000,  
#                            # iterator = True, 
#                            # dtype = conversionDict,
#                            # names=JOIN_COLUMNS,
#                            # usecols = JOIN_COLUMNS
#                           )
#     # for i, df_profiles in enumerate(df_iter):
#     # print(f"*** Chunk {i}   Starting index : {df_profiles.index[0]}   Ending Index {df_profiles.index[-1]}")
#     # print(f"\t Number of rows {df_profiles.shape[0]}   Size of chunk read: {df_profiles.memory_usage(deep=True).sum():,d} bytes")
#     # inv_cols = check_values(df_profiles,  numeric_columns)
#     print("-"*150)
#     print(f" {profilesFile}   {df_iter.shape}") 
#     print("-"*150)
#     df_iter.head(3)
#     print("\n\n")
    
#     # df_profiles.tail(5)
#     # if x > 13 :
#     break

In [ ]:
# files = [f for f in pathlib.Path().glob(f"{output_path}/profiles/*.csv")]
# files.sort()
# files
# for profilesFile in files:

for x in range(1,6):
    
    # profilesFile = CompoundProfiles2SampleFile + f"_{x:03d}" + CSV
    # profilesFile_a = CompoundProfiles2SampleFile + f"_{x:03d}_a" + CSV
    
    df_iter =  pd.read_csv(profilesFile.format(x), 
                           low_memory = True,
                           # skiprows = 0,
                           # nrows = 5,
                           header = 0,
                           names = COMPOUND_PROFILE_COLUMNS, 
                           # names=JOIN_COLUMNS,
                           usecols = MetadataNames, 
                           # usecols = ['Metadata_JCP2022', 'Metadata_permiation', 'Cells_AreaShape_Area']
                           dtype= string_conversions,
                           # dtype = conversionDict,
                           # chunksize= 10000,  
                           # iterator = True, 
                          )
    # for i, df_profiles in enumerate(df_iter):
    # print(f"*** Chunk {i}   Starting index : {df_profiles.index[0]}   Ending Index {df_profiles.index[-1]}")
    # print(f"\t Number of rows {df_profiles.shape[0]}   Size of chunk read: {df_profiles.memory_usage(deep=True).sum():,d} bytes")
    # inv_cols = check_values(df_profiles,  numeric_columns)
    print("-"*150)
    print(f" {profilesFile.format(x)}   {df_iter.shape}") 
    print("-"*150)
    df_iter.head(4)
    df_iter.tail(4)
    print("\n\n")
    
    # df_profiles.tail(5)
    # if x > 13 :
    # break

# Experiments on reducing the memory foot print 

- converting the Metadata fields from `object` to `category`

In [ ]:
try: 
    del df_profiles2
except Exception as e:
    pass

In [ ]:
outputFile = CompoundProfiles2SampleFile
type = '.bz2'

compoundProfiles = outputFile+type
# del df_profiles_list 
print(f" Read from {compoundProfiles}")

In [ ]:
print(compoundProfiles)

In [ ]:
df_profiles2 =  pd.read_csv(compoundProfiles, usecols = ['Metadata_Source'], nrows=30000)

In [ ]:
df_profiles2.info()

df_profiles2.memory_usage(deep=True)[:6]

In [ ]:
## experiment with converting columns to "category" type to reduce memory footprint

# df_profiles2 = df_profiles.copy()
# del df_profiles2
# df_profiles2["Metadata_Source"] = df_profiles2["Metadata_Source"].astype("category")
# df_profiles2["Metadata_Batch"] = df_profiles2["Metadata_Batch"].astype("category")
# df_profiles2["Metadata_Plate"] = df_profiles2["Metadata_Plate"].astype("category")
# df_profiles2["Metadata_Well"] = df_profiles2["Metadata_Well"].astype("category")

# df_profiles2["Metadata_JCP2022"] = df_profiles2["Metadata_JCP2022"].astype("object")

In [ ]:
df_profiles2.memory_usage(deep=True)[:6]

In [ ]:
del df_profiles2

In [ ]:
df_profiles2.memory_usage(deep=True)[:6]
df_profiles2.head(10)
df_profiles2.tail(10)

**Each row in `dframes` is well-level profile, containing thousands of features (n=4762) averaged over (typically) a couple of thousand cells per well.**

In [ ]:
df_profiles.columns[-5:]

In [ ]:
# df_profiles.drop(columns='_merge', inplace = True)
df_profiles2.columns[:20]
df_profiles2.columns[-5:]
df_profiles2.head(2)
# df_profiles.index.value_counts()
print('---------------------------------------------------------------')
print(' Before Index Reset')
print('---------------------------------------------------------------')
df_profiles2[1552:1557]
print('---------------------------------------------------------------')
d

In [ ]:
df_profiles2.reset_index(drop=True, inplace = True)
df_profiles2.info()
print('---------------------------------------------------------------')
print(' After Index Reset')
print('---------------------------------------------------------------')
df_profiles2.head(5)
df_profiles2.tail(5)
print('---------------------------------------------------------------')

In [ ]:
df_info = df_profiles.info(max_cols = 10,verbose = True, memory_usage = True, show_counts = True)

In [ ]:
st_col, end_col = 0,10
col_list = list(df_profiles.columns)
col_list[st_col:end_col]
df_profiles[col_list[st_col:end_col]].shape
df_profiles[col_list[st_col:end_col]]

In [ ]:
# dframes.drop(index=384, inplace = True)

# Get Plate Level Stats

In [ ]:
def get_stats(df_stats, df_data):
    df_data_cols = df_data.columns.tolist()
    print(df_data_cols[:10])
    if len(df_stats.columns) == 0 :
        print(f" length of df_stats.columns: {len(df_stats.columns)}")
        df_stats = pd.DataFrame(columns = ["Stat"]+(df_data.columns.tolist()) ) 
        df_stats_cols = df_stats.columns.tolist()
        print(df_stats_cols[:10])
    col_data = [df_data.iloc[0,0], df_data.iloc[0,1], "All wells"]
    print(col_data)
    df_stats.loc[len(df_stats)] = ["Minimum"] + col_data + df_data.min(0).tolist()[5:] 
    df_stats.loc[len(df_stats)] = ["Maximum"] + col_data + df_data.max(0).tolist()[5:] 
    # df_stats.loc[len(df_stats)] = ["Mean"]    + col_data + df_data.mean(0, numeric_only=True).tolist()[5:] 
    # df_stats.loc[len(df_stats)] = ["Median"]  + col_data + df_data.median(0, numeric_only=True).tolist()[5:] 
    print(type(df_stats), len(df_stats))
    return df_stats



In [ ]:
del stats
stats = pd.DataFrame()
stats = get_stats(stats, df_profiles)

In [ ]:
print(type(stats), len(stats))
stats

In [ ]:
# for i in dframes.columns[:500]:
#     print(f' {i}')
# stats.columns.to_series().to_csv('./output/measurement_column_names.csv')
stats.to_csv('./output/sample_statistics.csv')

## Join features with Compound and Well metadata

The profiles are annotated with only three columns of metadata (source, plate, well).

Let's add more metadata!

In [ ]:
# wells[(wells.Metadata_Source == "source_3") & (wells.Metadata_Plate == "BR5874c3") & (wells.Metadata_JCP2022 == "JCP2022_999999")][20:50]

In [ ]:
wells.head(3)
plates.head(3)

In [ ]:
metadata = wells.merge(plates, on=["Metadata_Source", "Metadata_Plate"])

In [ ]:
metadata.info()
metadata.head()
metadata.Metadata_PlateType.value_counts()
metadata2.Metadata_JCP2022.value_counts()

In [ ]:
compound.head(3)

In [ ]:
metadata2 = metadata.merge(compound, on="Metadata_JCP2022")
metadata2.info()
metadata2.head()
metadata2.Metadata_PlateType.value_counts()
metadata2.Metadata_JCP2022.value_counts()

In [ ]:
jcp_999999_idxs = metadata2[ metadata2.Metadata_JCP2022 == "JCP2022_999999"].index
len(jcp_999999_idxs)

In [ ]:
jcp_033924_idxs = metadata2[ metadata2.Metadata_JCP2022 == "JCP2022_033924"].index
len(jcp_033924_idxs)

In [ ]:
# jcp_all_idxs = jcp_999999_idxs + jcp_033924_idx
jcp_bad_idxs = pd.Index.append(jcp_999999_idxs,jcp_033924_idxs)
len(jcp_bad_idxs)

In [ ]:
metadata3 = metadata2.drop(jcp_bad_idxs)

In [ ]:

# metadata2 = compound.merge(wells, on="Metadata_JCP2022")
# metadata2.info()
# metadata2.head()
# metadata2.Metadata_JCP2022.value_counts()

# metadata3 = metadata2.merge(plates, on=["Metadata_Source", "Metadata_Plate"])
metadata3.info()
metadata3.head()
metadata3.Metadata_JCP2022.value_counts()[:20]

### Save as `compound_well_plate.csv`

In [ ]:
compoundWellPlateFile         = f"./output/cwp_metadata_clean.csv"
metadata3.to_csv(compoundWellPlateFile)

In [ ]:
ann_dframe = metadata3.merge(    
    dframes, on=["Metadata_Source", "Metadata_Plate", "Metadata_Well"]
)

We now know a little bit more about each profile:

In [ ]:
ann_dframe.info()
ann_dframe.head()
ann_dframe.Metadata_PlateType.value_counts()
ann_dframe.Metadata_JCP2022.value_counts()[:20]

In [ ]:
# ann_dframe.sample(2, random_state=42)
ann_dframe.sort_values("Metadata_JCP2022")

More metadata information will be added in the future. 

# Plot features

The scatterplot below contains every well in the sampled dataset.

In the interactive plot (see settings for `pio.renderers.default` above), you can hover over the points to see the JCP ID and the InChiKey for a given compound.

<div class="alert alert-warning">
NOTE: Because these are raw, unnormalized features, you will notice discernable clusters corresponding to each source due to batch effects.
Upcoming data releases will included normalized features, where these effects are mitigated to some extent. 
</div>

In [ ]:
 # pio.renderers.default = 'jupyterl`ab'
# pio.renderers.default = 'notebook_connected'
pio.renderers.default = "browser"

In [ ]:
from pickle import FALSE, TRUE
del fig
fig = px.scatter(
    df_profiles,
    x="Cells_AreaShape_Eccentricity",
    y="Cells_AreaShape_EulerNumber",
    color="Metadata_Source",
    hover_name="Metadata_JCP2022",
    hover_data=["Metadata_Source"],
)

fig.show()

### **Use DASH**

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Analysis of Iris data using scatter matrix'),
    dcc.Dropdown(
        id="dropdown",
        options=useColumns,
        value=[useColumns[0], "Cells_AreaShape_Eccentricity"],
        multi=True
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))
def update_bar_chart(dims):
    # df = px.data.iris() # replace with your own data source
    # fig = px.scatter_matrix(
    #     df_profiles, dimensions=dims, color="species")
    print(value)
    fig = px.scatter(
            df_profiles,
            x="Cells_AreaShape_Eccentricity",
            y="Cells_AreaShape_EulerNumber",
            color="Metadata_Source",
            hover_name="Metadata_JCP2022",
            hover_data=["Metadata_Source"],
        )
    return fig


app.run_server(debug=True,port=8051)

So that's just a couple of (raw) measurements from the sentinel plates for 10/13 of the sources, for the principal dataset alone. 